In [ ]:
print()

### setup packages

In [ ]:
# import packages
from glob import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scanpy as sc
import scipy.stats as ss
import seaborn as sns
sc.settings.set_figure_params(dpi=100)

### genetic constraints

In [ ]:
# gnomad v4.1, pLI=0.853
# constraint metrics based on MANE Select transcript (ENST00000424100.2)
mut = pd.DataFrame(index=['syn','mis','plof'], columns=['expected_SNVs','observed_SNVs','z_score','obs/exp','lbound','ubound'])
mut.loc['syn'] = [173.725, 167, 0.278, 0.961, 0.847, 1.093]
mut.loc['mis'] = [467.904, 299, 0, 0.639, 0.581, 0.703]
mut.loc['plof'] = [22.128, 7, 0.853, 0.316, 0.179, 0.594]  # right is LOEUF

# create forest plots
fig, ax = plt.subplots(figsize=[4, 2.5]); ax.grid(False)
colors = ['tab:red','tab:orange','tab:green']
for idx_, idx in enumerate(mut.index[::-1]):
    ax.scatter(mut.loc[idx, 'obs/exp'], idx, color=colors[idx_], s=75)
    ax.plot([mut.loc[idx, 'lbound'], mut.loc[idx, 'ubound']], [idx]*2, color=colors[idx_], lw=2.5)
    ax.plot([mut.loc[idx, 'lbound']]*2, [idx_-0.1, idx_+0.1], color=colors[idx_], lw=2.5)
    ax.plot([mut.loc[idx, 'ubound']]*2, [idx_-0.1, idx_+0.1], color=colors[idx_], lw=2.5)
ax.set_xlim(0.1, 1.15)
ax.plot([1]*2, [-1, 3], linestyle='--', color='k')
ax.set_ylim(-1, 3)
ax.set_yticklabels(['Synonmous','Missense','pLOF'][::-1])
ax.set_xlabel('Observed / Expected')

### compile spatial transcriptomic datasets

In [ ]:
# read in the mapping file for GPCRdb to filter for GPCRs
gpcr_mapping = pd.read_table('gpcrdb_mapping.txt', header=None, sep=' ')
gpcr_mapping.columns = ['uniprot','name']
gpcr_mapping[['name','species']] = gpcr_mapping['name'].str.split('_', expand=True)
# keep human uniprot IDs
uniprots = gpcr_mapping.loc[gpcr_mapping['species'] == 'human', 'uniprot'].tolist()

In [ ]:
from glob import glob
# compile the complete list of visiums to run
adatas = []

In [ ]:
# process heart atlas
for fn in glob('/path/to/data/external/heartcellatlas/*.h5ad'):
    # read in the file
    adata = sc.read_h5ad(fn)
    # define the spatial coordinates
    adata.obsm['spatial'] = adata.obs[['array_row','array_col']].values
    # add on the file id
    adata.obs['file_id'] = fn.split('/')[-1]
    # split by sanger id
    for id in adata.obs['sangerID'].unique():
        adatas.append(adata[adata.obs['sangerID'] == id].copy())

In [ ]:
# process fetal lung atlas
for fn in glob('/path/to/data/external/fetallung/*.h5ad'):
    # read in the file
    adata = sc.read_h5ad(fn)
    # define the spatial coordinates
    adata.obsm['spatial'] = adata.obs[['array_row','array_col']].values
    # add on the file id
    adata.obs['file_id'] = fn.split('/')[-1]
    # each sample is a separate file
    adatas.append(adata)
    assert len(list(adata.uns['spatial'].keys())) == 1

In [ ]:
# process fetal immune atlas
for fn in glob('/path/to/data/external/fetalimmune/*.h5ad'):
    # read in the file
    adata = sc.read_h5ad(fn)
    # define the spatial coordinates
    adata.obsm['spatial'] = adata.obs[['array_row','array_col']].values
    # add on the file id
    adata.obs['file_id'] = fn.split('/')[-1]
    # split by sanger id
    for id in adata.obs['sample_id'].unique():
        adatas.append(adata[adata.obs['sample_id'] == id].copy())

In [ ]:
# process kidney atlas
for fn in glob('/path/to/data/external/kidneycellatlas/*.h5ad'):
    # read in the file
    adata = sc.read_h5ad(fn)
    # define the spatial coordinates
    adata.obsm['spatial'] = adata.obs[['array_row','array_col']].values
    # add on the file id
    adata.obs['file_id'] = fn.split('/')[-1]
    # split by sanger id
    for id in adata.obs['sample'].unique():
        adatas.append(adata[adata.obs['sample'] == id].copy())

In [ ]:
# process 10x genomics data
fns = glob('/path/to/data/external/10x_multiorgans/*.h5ad') +\
glob('/path/to/data/external/10x_visium_v1_cancer/*.h5ad')
for fn in fns:
    # read in the file
    adata = sc.read_h5ad(fn)
    # define the spatial coordinates
    adata.obsm['spatial'] = adata.obs[['row','col']].values
    # add on the file id
    adata.obs['file_id'] = fn.split('/')[-1]
    # each sample is a separate file
    adatas.append(adata)

10x genomics derived data was log normalized via bayesspace in R via:
```
library(SingleCellExperiment)
library(BayesSpace)
# read in dataset
dirname <- "<path_to_raw_10x_genomics_data>"
sce <- readVisium(paste0(dirname, "<name_of_dataset>"))
# compute PCA with HVGs
sce <- spatialPreprocess(sce, platform="Visium", n.PCs=15, n.HVGs=2000, log.normalize=TRUE)
# write the data
library(zellkonverter)
writeH5AD(sce, file="<path_to_normalized_h5ad>")
```

In [ ]:
import spatialdm as sdm
# define the method to derive the spatial correlations
def calc_corr(adata):
    try:
        # make sure everything is unique
        adata.var_names = adata.var_names.astype(str)
        sc.pp.filter_genes(adata, min_cells=10)
        adata.var_names_make_unique()
        # define the weight matrix for the kernel
        sdm.weight_matrix(adata, l=1.2, cutoff=0.2, single_cell=False)
        # define the min n-spots filter
        n_spots = 10

        # assemble expression values of complexes
        genes = df_complex.values.flatten().astype(str)[df_complex.values.flatten().astype(str) != 'nan']
        genes = adata.var.index.intersection(genes)
        df_gex = sc.get.obs_df(adata, keys=genes.tolist())
        expr_complexs = []
        for idx in df_complex.index:
            genes = adata.var.index.intersection(df_complex.loc[idx].dropna())
            if len(genes) == 0: continue
            expr_complex = df_gex[genes].mean(1)
            expr_complex.name = idx
            expr_complexs.append(expr_complex)
        expr_complex = pd.concat(expr_complexs, axis=1)
        valid_complexes = expr_complex.columns[(expr_complex > 0).sum(0) >= n_spots]

        # perform rapid filtering (random)
        ligands = valid_rand_pairs['ligand'].unique()
        ligands = adata.var.index.intersection(ligands)
        try: valid_ligands = ligands[(adata[:, ligands].X > 0).sum(0).A1 >= n_spots].tolist()
        except: valid_ligands = ligands[(adata[:, ligands].X > 0).sum(0) >= n_spots].tolist()
        valid_ligands = valid_ligands + [x for x in valid_rand_pairs['ligand'].unique() if x in valid_complexes]
        receptors = valid_rand_pairs['receptor'].unique()
        receptors = adata.var.index.intersection(receptors)
        try: valid_receptors = receptors[(adata[:, receptors].X > 0).sum(0).A1 >= n_spots].tolist()
        except: valid_receptors = receptors[(adata[:, receptors].X > 0).sum(0) >= n_spots].tolist()
        valid_receptors = valid_receptors + [x for x in valid_rand_pairs['receptor'].unique() if x in valid_complexes]
        valid_rand_pairs_ = valid_rand_pairs.loc[valid_rand_pairs['ligand'].isin(valid_ligands) &\
        valid_rand_pairs['receptor'].isin(valid_receptors)].copy()
        # extract and compute Z-scores for the known ligand-receptor pairs
        sdm.extract_lr(adata, 'human', min_cell=10, datahost='user', df_intrxn=valid_rand_pairs_, df_complex=df_complex.copy())
        sdm.spatialdm_global(adata, specified_ind=None, method='z-score', nproc=1)
        zpval_rand, z_rand = adata.uns['global_res']['z_pval'], adata.uns['global_res']['z']

        # perform rapid filtering (known)
        ligands = df_intrxn_gpcrvsall['ligand'].unique()
        ligands = adata.var.index.intersection(ligands)
        try: valid_ligands = ligands[(adata[:, ligands].X > 0).sum(0).A1 >= n_spots].tolist()
        except: valid_ligands = ligands[(adata[:, ligands].X > 0).sum(0) >= n_spots].tolist()
        valid_ligands = valid_ligands + [x for x in df_intrxn_gpcrvsall['ligand'].unique() if x in valid_complexes]
        receptors = df_intrxn_gpcrvsall['receptor'].unique()
        receptors = adata.var.index.intersection(receptors)
        try: valid_receptors = receptors[(adata[:, receptors].X > 0).sum(0).A1 >= n_spots].tolist()
        except: valid_receptors = receptors[(adata[:, receptors].X > 0).sum(0) >= n_spots].tolist()
        valid_receptors = valid_receptors + [x for x in df_intrxn_gpcrvsall['receptor'].unique() if x in valid_complexes]
        df_intrxn_gpcrvsall_ = df_intrxn_gpcrvsall.loc[df_intrxn_gpcrvsall['ligand'].isin(valid_ligands) &\
        df_intrxn_gpcrvsall['receptor'].isin(valid_receptors)].copy()
        # extract and compute Z-scores for the known ligand-receptor pairs
        sdm.extract_lr(adata, 'human', min_cell=10, datahost='user', df_intrxn=df_intrxn_gpcrvsall_, df_complex=df_complex.copy())
        sdm.spatialdm_global(adata, specified_ind=None, method='z-score', nproc=1)
        zpval_known, z_known = adata.uns['global_res']['z_pval'], adata.uns['global_res']['z']
        
        # perform rapid filtering (orphan)
        ligands = df_intrxn_orphan['ligand'].unique()
        ligands = adata.var.index.intersection(ligands)
        try: valid_ligands = ligands[(adata[:, ligands].X > 0).sum(0).A1 >= n_spots].tolist()
        except: valid_ligands = ligands[(adata[:, ligands].X > 0).sum(0) >= n_spots].tolist()
        valid_ligands = valid_ligands + [x for x in df_intrxn_orphan['ligand'].unique() if x in valid_complexes]
        receptors = df_intrxn_orphan['receptor'].unique()
        receptors = adata.var.index.intersection(receptors)
        try: valid_receptors = receptors[(adata[:, receptors].X > 0).sum(0).A1 >= n_spots].tolist()
        except: valid_receptors = receptors[(adata[:, receptors].X > 0).sum(0) >= n_spots].tolist()
        valid_receptors = valid_receptors + [x for x in df_intrxn_orphan['receptor'].unique() if x in valid_complexes]
        df_intrxn_orphan_ = df_intrxn_orphan.loc[df_intrxn_orphan['ligand'].isin(valid_ligands) &\
        df_intrxn_orphan['receptor'].isin(valid_receptors)].copy()
        # extract and compute Z-scores for the orphan ligand-receptor pairs
        sdm.extract_lr(adata, 'human', min_cell=10, datahost='user', df_intrxn=df_intrxn_orphan_, df_complex=df_complex.copy())
        sdm.spatialdm_global(adata, specified_ind=None, method='z-score', nproc=1)
        zpval_orphan, z_orphan = adata.uns['global_res']['z_pval'], adata.uns['global_res']['z']
        
        # assemble and return result
        result = {adata.obs['file_id'][0]: {'rand': {'z_pval': zpval_rand, 'z': z_rand}, 
                                            'known': {'z_pval': zpval_known, 'z': z_known}, 
                                            'orphan': {'z_pval': zpval_orphan, 'z': z_orphan}}}
        return result
    except Exception as e:
        print('!!!' + adata.obs['file_id'][0] + str(e))

In [ ]:
# retrieve uniprot to gene name mapping
df_conv = pd.read_csv('/home/dchen2/PACKAGES/cellphonedb-data-5.0.0/data/gene_input.csv')
df_conv = df_conv[['uniprot','gene_name']].value_counts().reset_index()
assert df_conv['uniprot'].value_counts().max() == 1
uniprot2genename = df_conv.set_index('uniprot')['gene_name']
# read in and process the complex
df_complex = pd.read_csv('/home/dchen2/PACKAGES/cellphonedb-data-5.0.0/data/complex_input.csv', index_col=0)
assert df_complex.index.value_counts().max() == 1; df_complex.index.name = None
df_complex = df_complex.loc[:, df_complex.columns.str.startswith('uniprot')]
df_complex.columns = df_complex.columns.str.replace('uniprot','subunit')
for col in df_complex.columns: df_complex[col] = df_complex[col].map(uniprot2genename)
# read in and process the interaction pairings
df_intrxn = pd.read_csv('/home/dchen2/PACKAGES/cellphonedb-data-5.0.0/data/interaction_input.csv')
for col, new_col in zip(['partner_a','partner_b'], ['ligand','receptor']):
    conv = df_intrxn[col].map(uniprot2genename)
    conv[conv.isna()] = df_intrxn.loc[conv.isna(), col]
    df_intrxn[new_col] = conv
df_intrxn['interaction_name'] = df_intrxn[['ligand','receptor']].agg(':'.join, axis=1)
df_intrxn.index = df_intrxn['interaction_name']
df_intrxn = df_intrxn[['interaction_name','ligand','receptor','is_ppi','directionality']]
# identify secreted proteins to differentiate
secreted_proteins = pd.read_csv('/home/dchen2/PACKAGES/cellphonedb-data-5.0.0/data/protein_input.csv', index_col=0)['secreted']
secreted_complexes = pd.read_csv('/home/dchen2/PACKAGES/cellphonedb-data-5.0.0/data/complex_input.csv', index_col=0)['secreted']
secreted_proteins.index = secreted_proteins.index.map(uniprot2genename)
secreted = pd.concat([secreted_proteins, secreted_complexes], axis=0)
secreted = secreted.index[secreted]
df_intrxn['annotation'] = 'No Secretion'
df_intrxn.loc[df_intrxn['ligand'].isin(secreted) | df_intrxn['receptor'].isin(secreted), 'annotation'] = 'Secretion'
# reset name of the index
df_intrxn.index.name = None
# adjust to remove duplicates
df_intrxn = df_intrxn.value_counts().reset_index().iloc[:, :-1]
df_intrxn.index = df_intrxn['interaction_name'].values
# derive a GPCR only subset
assert df_complex.index[df_complex.isin(uniprots).any(axis=1)].shape[0] == 0
df_conv_gpcrs = df_conv.loc[df_conv['uniprot'].isin(uniprots)]
# derive a list of the interactions with GPCRs
mask = df_intrxn['receptor'].isin(df_conv_gpcrs['gene_name'].unique())
df_intrxn_gpcrs = df_intrxn.loc[mask]

In [ ]:
from tqdm import tqdm
# get a list of the unique ligands from the entire database, we care about known for paralog
ligands = df_intrxn['ligand'].unique()
# receptors are known GPCRs, read in from GPCRdb
receptors = df_intrxn_gpcrs['receptor'].unique()
# fill in the possible interactions beyond known interactions
df_intrxn_gpcrvsall = pd.DataFrame(columns=['ligand','receptor'])
for receptor in tqdm(receptors):
    for ligand in ligands:
        df_intrxn_gpcrvsall.loc[df_intrxn_gpcrvsall.shape[0]] = ligand, receptor
df_intrxn_gpcrvsall.insert(0, 'interaction_name', df_intrxn_gpcrvsall[['ligand','receptor']].agg(':'.join, axis=1))
df_intrxn_gpcrvsall.index = df_intrxn_gpcrvsall['interaction_name']
df_intrxn_gpcrvsall['annotation'] = 'No Secretion'
df_intrxn_gpcrvsall.loc[df_intrxn_gpcrvsall['ligand'].isin(secreted), 'annotation'] = 'Secretion'
# reset name of the index
df_intrxn_gpcrvsall.index.name = None
# adjust to remove duplicates
df_intrxn_gpcrvsall = df_intrxn_gpcrvsall.value_counts().reset_index().iloc[:, :-1]
df_intrxn_gpcrvsall.index = df_intrxn_gpcrvsall['interaction_name'].values

In [ ]:
# repeat for orphans from GPCRdb (class A and class c)
with open('/path/to/orphans.all.txt', 'rt') as f:
    receptors = [x.strip() for x in f.readlines()]
# fill in the possible orphan interactions
df_intrxn_orphan = pd.DataFrame(columns=['ligand','receptor'])
for receptor in tqdm(receptors):
    for ligand in ligands:
        df_intrxn_orphan.loc[df_intrxn_orphan.shape[0]] = ligand, receptor
df_intrxn_orphan.insert(0, 'interaction_name', df_intrxn_orphan[['ligand','receptor']].agg(':'.join, axis=1))
df_intrxn_orphan.index = df_intrxn_orphan['interaction_name']
df_intrxn_orphan['annotation'] = 'No Secretion'
df_intrxn_orphan.loc[df_intrxn_orphan['ligand'].isin(secreted), 'annotation'] = 'Secretion'
# reset name of the index
df_intrxn_orphan.index.name = None
# adjust to remove duplicates
df_intrxn_orphan = df_intrxn_orphan.value_counts().reset_index().iloc[:, :-1]
df_intrxn_orphan.index = df_intrxn_orphan['interaction_name'].values

In [ ]:
# generate random pairs for the GPCR subset of the known
n_needed = df_intrxn_gpcrs.shape[0]
np.random.seed(0)
valid_rand_pairs = []; running_pairs = []
while n_needed > 0:
    # grab the data
    ligands = np.random.choice(df_intrxn_gpcrs['ligand'].unique(), size=n_needed, replace=True)
    receptors = np.random.choice(df_intrxn_gpcrs['receptor'].unique(), size=n_needed, replace=True)
    rand_pairs = pd.Series(np.unique(ligands + '=X=' + receptors))
    rand_pairs_intrxn_format = pd.Series(np.unique(ligands + ':' + receptors))
    mask = (~rand_pairs_intrxn_format.isin(df_intrxn_gpcrs.index)) & (~rand_pairs.isin(running_pairs))
    rand_pairs = rand_pairs[mask]
    # create the dataframe and deduplicate
    rand_pairs_split = rand_pairs.str.split('=X=', expand=True)
    valid_rand_pairs.append(rand_pairs_split)
    running_pairs += rand_pairs.tolist()
    n_needed -= len(rand_pairs)
valid_rand_pairs = pd.concat(valid_rand_pairs, axis=0)
valid_rand_pairs.columns = ['ligand','receptor']

In [ ]:
# add on the proper annotation
valid_rand_pairs = valid_rand_pairs.reset_index().iloc[:, 1:]
valid_rand_pairs['annotation'] = 'No Secretion'
valid_rand_pairs.loc[valid_rand_pairs['ligand'].isin(secreted) | valid_rand_pairs['receptor'].isin(secreted), 'annotation'] = 'Secretion'
valid_rand_pairs['interaction_name'] = valid_rand_pairs[['ligand','receptor']].agg(':'.join, axis=1)
valid_rand_pairs.index = valid_rand_pairs['interaction_name']
valid_rand_pairs = valid_rand_pairs[['interaction_name','ligand','receptor','annotation']]

In [ ]:
import multiprocessing as mp
from tqdm import tqdm
# leverage multiprocessing to clear the predictions
results = []
with mp.Pool(processes=36) as pool:
    for result in tqdm(pool.map(calc_corr, adatas), total=len(adatas)):
        results.append(result)

In [ ]:
# read in the annotations
anno = pd.read_excel('/path/to/241111_visium_annotations.xlsx', sheet_name=0)
# conversion for 10x
file2sample = anno[['file_id','sample_id']].value_counts().reset_index().set_index('file_id')['sample_id']
anno['tag'] = anno[['file_id','sample_id']].agg(':'.join, axis=1)
# create an age category
anno['age_cat'] = 'Unknown (likely adult)'
anno.loc[anno['age'].astype(str).str.endswith('pcw'), 'age_cat'] = 'Fetal'
anno.loc[anno['age'].isin(['55-60','45-50','40-45','50-55','20-25','51-60','41-50']), 'age_cat'] = 'Adult (<60)'
anno.loc[anno['age'].isin(['60-70','73','70-75','65-70','60-65','71-80','71-70']), 'age_cat'] = 'Adult (≥60)'

In [ ]:
# compile the information
df_info = pd.DataFrame(columns=['file_id','sample_id'])
for adata in adatas:
    file_id = adata.obs['file_id'].iloc[0]
    sample_col = 'sangerID'
    if sample_col not in adata.obs: sample_col = 'sample_id'
    if sample_col not in adata.obs: sample_col = 'sample'
    if sample_col not in adata.obs:
        sample_id = file2sample[file_id]
    else:
        sample_id = adata.obs[sample_col].iloc[0]
    df_info.loc[df_info.shape[0]] = file_id, sample_id
df_info['tag'] = df_info[['file_id','sample_id']].agg(':'.join, axis=1)

In [ ]:
# compile the results for z-score of known LRs with rand LRs as control
df_zs = []
for result, tag in zip(results, df_info['tag']):
    if result is None: continue
    # grab the file id
    file_id = list(result.keys())[0]
    # grab the result object
    result = result[file_id]
    # retrieve the proper dataframe
    known, rand = result['known']['z'].copy(), result['rand']['z'].copy()
    known.index += '}{KNOWN'; rand.index += '}{RAND'
    df_z = pd.concat([known, rand], axis=0).copy()
    df_z.name = tag
    df_zs.append(df_z)

In [ ]:
# compute the expression profiles
df_z = pd.concat(df_zs, axis=1).T.fillna(0)
df_z['organ'] = df_z.index.map(anno[['tag','organ']].value_counts().reset_index().set_index('tag')['organ'])
df_z['organ'] += ':' + df_z.index.map(anno[['tag','age_cat']].value_counts().reset_index().set_index('tag')['age_cat'])
df_z = df_z.melt(id_vars='organ').dropna().reset_index().iloc[:, 1:]
df_z.columns = ['organ','variable','value']
df_z[['variable','test']] = df_z['variable'].str.split('}{', expand=True, regex=False)
df_z['secreted_vs_not'] = df_z['variable'].map(df_intrxn['annotation'])
df_z['receptor_type'] = df_z['variable'].map(df_intrxn['directionality'])
df_z['protein_vs_metabolite'] = df_z['variable'].map(df_intrxn['is_ppi']).map({True:'Protein (Secreted)', False:'Metabolite (Secreted)'})
assert df_z.loc[df_z['secreted_vs_not'] == 'No Secretion', 'protein_vs_metabolite'].value_counts().index.tolist() == ['Protein (Secreted)']
assert df_z.loc[df_z['receptor_type'] == 'Adhesion-Adhesion', 'protein_vs_metabolite'].value_counts().index.tolist() == ['Protein (Secreted)']
# > gap junctions are not GPCRs so they are excluded from this analysis
# assert df_z.loc[df_z['receptor_type'] == 'Gap-Gap', 'protein_vs_metabolite'].value_counts().index.tolist() == ['Protein (Secreted)']
df_z.loc[df_z['secreted_vs_not'] == 'No Secretion', 'protein_vs_metabolite'] = 'Protein (Non-Secreted)'
df_z.loc[df_z['receptor_type'] == 'Adhesion-Adhesion', 'protein_vs_metabolite'] = 'Protein (Adhesion)'
df_z.loc[df_z['receptor_type'] == 'Gap-Gap', 'protein_vs_metabolite'] = 'Protein (Gap Junction)'
df_z['protein_vs_metabolite'] = df_z['protein_vs_metabolite'].fillna('Random')
df_z['organ_simple'] = df_z['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
# only look at known for now
df_z_plot = df_z.loc[df_z['variable'].isin(df_intrxn.index) | (df_z['test'] == 'RAND')]

In [ ]:
# get the color of the different organs
order = sorted(df_z_plot['organ_simple'].unique())
order1 = [x for x in order if x.startswith('cns')]
order2 = [x for x in order if x.startswith('heart')]
order3 = [x for x in order if x not in order1 and x not in order2]
hue_order = ['Metabolite (Secreted)','Protein (Secreted)','Protein (Non-Secreted)','Protein (Adhesion)', 'Random']
# this is too long we have to separately plot
# > cns
fig, ax = plt.subplots(figsize=[6, 4]); ax.grid(False)
sns.barplot(x='organ_simple', y='value', data=df_z_plot, hue='protein_vs_metabolite', order=order1,
            ci=68, errwidth=1.5, capsize=0.3, edgecolor='k', errcolor='k', linewidth=1.5, hue_order=hue_order,
            saturation=1, palette=['#cc78bc', '#56b4e9', '#de8f05', '#0173b2','lightgray'])
ax.tick_params(axis='x', labelrotation=90); ax.set_xlim(-1, len(order1)); ax.legend(bbox_to_anchor=(1.05, .99), bbox_transform=ax.transAxes, frameon=False, loc='upper left')
ax.set_ylim(0, 60)
# > heart
fig, ax = plt.subplots(figsize=[11, 4]); ax.grid(False)
sns.barplot(x='organ_simple', y='value', data=df_z_plot, hue='protein_vs_metabolite', order=order2,
            ci=68, errwidth=1.5, capsize=0.3, edgecolor='k', errcolor='k', linewidth=1.5, hue_order=hue_order,
            saturation=1, palette=['#cc78bc', '#56b4e9', '#de8f05', '#0173b2','lightgray'])
ax.tick_params(axis='x', labelrotation=90); ax.set_xlim(-1, len(order2)); ax.legend(bbox_to_anchor=(1.05, .99), bbox_transform=ax.transAxes, frameon=False, loc='upper left')
ax.set_ylim(0, 60)
# > rest
fig, ax = plt.subplots(figsize=[18, 4]); ax.grid(False)
sns.barplot(x='organ_simple', y='value', data=df_z_plot, hue='protein_vs_metabolite', order=order3,
            ci=68, errwidth=1.5, capsize=0.3, edgecolor='k', errcolor='k', linewidth=1.5, hue_order=hue_order,
            saturation=1, palette=['#cc78bc', '#56b4e9', '#de8f05', '#0173b2','lightgray'])
ax.tick_params(axis='x', labelrotation=90); ax.set_xlim(-1, len(order3)); ax.legend(bbox_to_anchor=(1.05, .99), bbox_transform=ax.transAxes, frameon=False, loc='upper left')
ax.set_ylim(0, 60)

In [ ]:
# average per organ in the random data to get a baseline to normalize by
df_z_tmp = df_z_plot.loc[df_z_plot['test'] == 'RAND'].copy()
df_z_tmp['organ'] = df_z_tmp['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
df_z_tmp = df_z_tmp.groupby('organ').mean(numeric_only=True)
# derive a baseline based on random
baseline_random = df_z_tmp.mean(1)
baseline_random_organ = baseline_random.copy()

In [ ]:
# utilize this baseline to normalize all relevant values
df_z_nrm = df_z_plot.loc[df_z_plot['test'] == 'KNOWN'].copy()
df_z_nrm['organ'] = df_z_nrm['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
df_z_nrm['value'] /= df_z_nrm['organ'].map(baseline_random)
df_z_nrm[['ligand','receptor']] = df_z_nrm['variable'].str.split(':', expand=True)
assert df_z_nrm['variable'].str.split(':', expand=True).shape[1] == 2
df_z_nrm_plot = df_z_nrm.pivot_table(index='organ', columns='variable', values='value')

In [ ]:
# calculate linkages
from scipy.cluster.hierarchy import linkage, fcluster
print('calculating linkages for rows')
Z_row = linkage(df_z_nrm_plot, method='ward')
print('calculating linkages for columns')
Z_col = linkage(df_z_nrm_plot.T, method='ward')
Z_col_organ_known = Z_col.copy()

In [ ]:
from sklearn.metrics import silhouette_score
# define clusters
clusters = fcluster(Z_col_organ_known, criterion='maxclust', t=30)
clusters = pd.Series(clusters, index=df_z_nrm_plot.columns).astype(str)
# define the palette
pal = sns.color_palette('hls', 30).as_hex()
pal = {str(idx+1):color for idx, color in enumerate(pal)}
colors = clusters.map(pal)

In [ ]:
# create clustermap
pal = {str(idx):'r' for idx in range(25, 30+1)}
g = sns.clustermap(df_z_nrm_plot, method='ward', figsize=[25, 15], vmin=-15, vmax=15, cmap='RdBu_r',
                   col_colors=clusters.map(pal).fillna('lightgray'), colors_ratio=(.025),
                   cbar_pos=(0, 1, .01, .1), yticklabels=1, dendrogram_ratio=(.05, .1))
g.ax_heatmap.grid(False); g.ax_col_colors.grid(False)

In [ ]:
# create clustermap, this is the clean version
g = sns.clustermap(df_z_nrm_plot, method='ward', figsize=[25, 15], vmin=-15, vmax=15, cmap='RdBu_r',
                   cbar_pos=(0, 1, .01, .1), yticklabels=1, dendrogram_ratio=(.05, .1))
g.ax_heatmap.grid(False)

In [ ]:
# compare the CNS and non-CNS adult and non-adult
df_z_nrm_melt = df_z_nrm.copy()
df_z_nrm_melt['cat_simple'] = 'CNS'
df_z_nrm_melt.loc[~df_z_nrm_melt['organ'].str.startswith('cns'), 'cat_simple'] = 'Non-CNS'
df_z_nrm_melt.loc[df_z_nrm_melt['organ'].str.endswith('adult'), 'cat_simple'] += ' (Adult)'
df_z_nrm_melt.loc[df_z_nrm_melt['organ'].str.endswith('fetal'), 'cat_simple'] += ' (Fetal)'
# map on the interaction types
df_z_nrm_melt['secreted_vs_not'] = df_z_nrm_melt['variable'].map(df_intrxn['annotation'])
df_z_nrm_melt['receptor_type'] = df_z_nrm_melt['variable'].map(df_intrxn['directionality'])
df_z_nrm_melt['protein_vs_metabolite'] = df_z_nrm_melt['variable'].map(df_intrxn['is_ppi']).map({True:'Protein (Secreted)', False:'Metabolite (Secreted)'})
assert df_z_nrm_melt.loc[df_z_nrm_melt['secreted_vs_not'] == 'No Secretion', 'protein_vs_metabolite'].value_counts().index.tolist() == ['Protein (Secreted)']
assert df_z_nrm_melt.loc[df_z_nrm_melt['receptor_type'] == 'Adhesion-Adhesion', 'protein_vs_metabolite'].value_counts().index.tolist() == ['Protein (Secreted)']
df_z_nrm_melt.loc[df_z_nrm_melt['secreted_vs_not'] == 'No Secretion', 'protein_vs_metabolite'] = 'Protein (Non-Secreted)'
df_z_nrm_melt.loc[df_z_nrm_melt['receptor_type'] == 'Adhesion-Adhesion', 'protein_vs_metabolite'] = 'Protein (Adhesion)'
df_z_nrm_melt.loc[df_z_nrm_melt['receptor_type'] == 'Gap-Gap', 'protein_vs_metabolite'] = 'Protein (Gap Junction)'
# get the color of the different organs
fig, ax = plt.subplots(figsize=[8, 4]); ax.grid(False)
order = sorted(df_z_nrm_melt['cat_simple'].unique())
sns.barplot(x='cat_simple', y='value', data=df_z_nrm_melt, hue='protein_vs_metabolite', order=order,
            ci=68, errwidth=1.5, capsize=0.3, edgecolor='k', errcolor='k', linewidth=1.5,
            hue_order=['Metabolite (Secreted)','Protein (Secreted)','Protein (Non-Secreted)',
                       'Protein (Adhesion)'], saturation=1,
            palette=['#cc78bc', '#56b4e9', '#de8f05', '#0173b2'])
ax.tick_params(axis='x', labelrotation=90)
ax.set_xlim(-1, len(order))
ax.legend(bbox_to_anchor=(1.05, .99), bbox_transform=ax.transAxes, frameon=False, loc='upper left')
ax.set_yscale('log')
ax.set_ylim(0, 100)
# perform all p-value comparisons
df_z_nrm_melt['tag'] = df_z_nrm_melt[['cat_simple', 'protein_vs_metabolite']].agg(':'.join, axis=1)
for tag in sorted(df_z_nrm_melt['tag'].unique()):
    if tag == 'CNS (Adult):Metabolite (Secreted)': continue
    p = ss.mannwhitneyu(df_z_nrm_melt.loc[df_z_nrm_melt['tag'] == 'CNS (Adult):Metabolite (Secreted)', 'value'],
                        df_z_nrm_melt.loc[df_z_nrm_melt['tag'] == tag, 'value'])[1]
    print(tag, p)
for tag in sorted(df_z_nrm_melt['tag'].unique()):
    if tag == 'Non-CNS (Adult):Metabolite (Secreted)': continue
    p = ss.mannwhitneyu(df_z_nrm_melt.loc[df_z_nrm_melt['tag'] == 'Non-CNS (Adult):Metabolite (Secreted)', 'value'],
                        df_z_nrm_melt.loc[df_z_nrm_melt['tag'] == tag, 'value'])[1]
    print(tag, p)
for tag in sorted(df_z_nrm_melt['tag'].unique()):
    if tag == 'Non-CNS (Fetal):Metabolite (Secreted)': continue
    p = ss.mannwhitneyu(df_z_nrm_melt.loc[df_z_nrm_melt['tag'] == 'Non-CNS (Fetal):Metabolite (Secreted)', 'value'],
                        df_z_nrm_melt.loc[df_z_nrm_melt['tag'] == tag, 'value'])[1]
    print(tag, p)

In [ ]:
# copy over the normalized data
df_z_known = df_z_nrm.copy()
# then average per receptor
df_z_stat = df_z_known.groupby('variable').mean(numeric_only=True)['value'].reset_index()
df_z_stat.columns = ['index','mean']
# retrieve other statistics
def custom(x): return ss.skew(x['value'])
df_z_stat['skew'] = df_z_known.groupby('variable').apply(custom).abs().values
df_z_stat['var'] = df_z_known.groupby('variable').var(numeric_only=True)['value'].fillna(0).values
df_z_stat['max'] = df_z_known.groupby('variable').max(numeric_only=True)['value'].fillna(0).values
df_z_stat_known = df_z_stat.copy()

In [ ]:
import statsmodels.api as sm
# model max after mean
model = sm.OLS(exog=sm.add_constant(df_z_stat['mean']), endog=df_z_stat['max'])
model = model.fit()
summary = model.summary()
exp = model.predict(sm.add_constant(df_z_stat['mean']))
obsmexp = df_z_stat['max'] - exp
df_z_stat['max_obsmexp'] = obsmexp

In [ ]:
# mean against max
fig, ax = plt.subplots(figsize=[6, 6]); ax.grid(False)
ax.scatter(df_z_stat['mean'], df_z_stat['max'], s=2.5, alpha=0.5, color='xkcd:bright lilac')
ax.set(xlabel='mean', ylabel='max')
# retrieve axes
xlim, ylim = ax.get_xlim(), ax.get_ylim()
ax.plot([0]*2, ylim, color='k', linestyle='-', lw=1.5)
ax.plot(xlim, [0]*2, color='k', linestyle='-', lw=1.5)
# retrieve predictions
xl = np.linspace(*xlim, 100)
yl = model.predict(sm.add_constant(xl))
ax.plot(xl, yl, color='k', linestyle='--', lw=1.5)
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
# retrieve correlation
print(ss.pearsonr(df_z_stat['mean'], df_z_stat['max']))

In [ ]:
# define the correlations of the statistics
def get_r(xs, ys): return ss.pearsonr(xs, ys)[0]
def get_p(xs, ys): return ss.pearsonr(xs, ys)[1]
mask = df_z_stat.iloc[:, 1:].corr(method=get_p) >= 0.05
for idx in mask.index: mask.loc[idx, idx] = False
g = sns.clustermap(df_z_stat.iloc[:, 1:].corr(method=get_r), vmin=-1, vmax=1, cmap='RdBu_r', annot=True, method='ward',
                   figsize=[6, 6], fmt='.2f', cbar_pos=(0, .9, .015, .12), mask=mask)
g.ax_heatmap.set_facecolor('lightgray'); g.ax_heatmap.grid(False); g.ax_cbar.grid(False)

In [ ]:
# map on the interaction types
df_z_stat['secreted_vs_not'] = df_z_stat['index'].map(df_intrxn['annotation'])
df_z_stat['receptor_type'] = df_z_stat['index'].map(df_intrxn['directionality'])
df_z_stat['protein_vs_metabolite'] = df_z_stat['index'].map(df_intrxn['is_ppi']).map({True:'Protein (Secreted)', False:'Metabolite (Secreted)'})
assert df_z_stat.loc[df_z_stat['secreted_vs_not'] == 'No Secretion', 'protein_vs_metabolite'].value_counts().index.tolist() == ['Protein (Secreted)']
assert df_z_stat.loc[df_z_stat['receptor_type'] == 'Adhesion-Adhesion', 'protein_vs_metabolite'].value_counts().index.tolist() == ['Protein (Secreted)']
df_z_stat.loc[df_z_stat['secreted_vs_not'] == 'No Secretion', 'protein_vs_metabolite'] = 'Protein (Non-Secreted)'
df_z_stat.loc[df_z_stat['receptor_type'] == 'Adhesion-Adhesion', 'protein_vs_metabolite'] = 'Protein (Adhesion)'
df_z_stat.loc[df_z_stat['receptor_type'] == 'Gap-Gap', 'protein_vs_metabolite'] = 'Protein (Gap Junction)'

In [ ]:
# plot those of interest
col = 'mean'
fig, ax = plt.subplots(figsize=[3, 4]); ax.grid(False)
order = ['Metabolite (Secreted)','Protein (Secreted)','Protein (Non-Secreted)',
         'Protein (Adhesion)']
palette = ['#cc78bc', '#56b4e9', '#de8f05', '#0173b2']
sns.barplot(x='protein_vs_metabolite', y=col, data=df_z_stat, palette=palette,
            errwidth=1.5, capsize=0.3, errcolor='k', ci=68,
            linewidth=1.5, edgecolor='k', saturation=1, order=order)
ax.tick_params(axis='x', labelrotation=90)
ylim = ax.get_ylim()
ax.set_xlim(-1, 4)
ax.set_yscale('symlog')
ax.set_ylim(0, 100)
# plot the results
for idx, tag1 in enumerate(df_z_stat['protein_vs_metabolite'].unique()[:-1]):
    for tag2 in df_z_stat['protein_vs_metabolite'].unique()[idx+1:]:
        p = ss.mannwhitneyu(df_z_stat.loc[df_z_stat['protein_vs_metabolite'] == tag1, col],
                            df_z_stat.loc[df_z_stat['protein_vs_metabolite'] == tag2, col])[1]
        print(tag1, tag2, p)

In [ ]:
# plot those of interest
col = 'max_obsmexp'
fig, ax = plt.subplots(figsize=[3, 4]); ax.grid(False)
sns.barplot(x='protein_vs_metabolite', y=col, data=df_z_stat, palette=palette,
            errwidth=1.5, capsize=0.3, errcolor='k', ci=68,
            linewidth=1.5, edgecolor='k', saturation=1, order=order)
ax.tick_params(axis='x', labelrotation=90)
ylim = ax.get_ylim()
ax.set_xlim(-1, 4)
ax.set_yscale('symlog')
ax.set_ylim(-10, 100)
# plot the results
for idx, tag1 in enumerate(df_z_stat['protein_vs_metabolite'].unique()[:-1]):
    for tag2 in df_z_stat['protein_vs_metabolite'].unique()[idx+1:]:
        p = ss.mannwhitneyu(df_z_stat.loc[df_z_stat['protein_vs_metabolite'] == tag1, col],
                            df_z_stat.loc[df_z_stat['protein_vs_metabolite'] == tag2, col])[1]
        print(tag1, tag2, p)

In [ ]:
# get the plot
plot = df_z_nrm_melt.pivot_table(index='organ', columns='protein_vs_metabolite', values='value', aggfunc=np.mean)
# get the plot – standardized
g = sns.clustermap(plot, method='ward', cmap='Reds', figsize=[6, 9], yticklabels=1,
                   dendrogram_ratio=(.2, .05), cbar_pos=(0, 1, .01, .1), standard_scale=0)
g.ax_heatmap.grid(False)
plot.to_csv('/path/to/SI_TABLE7.csv')

In [ ]:
# compile the results for z-score of gpcr screen LRs
df_zs = []
for result, tag in zip(results, df_info['tag']):
    if result is None: continue
    # grab the file id
    file_id = list(result.keys())[0]
    # grab the result object
    result = result[file_id]
    # retrieve the proper dataframe
    df_z = result['known']['z'].copy()
    df_z.name = tag
    df_zs.append(df_z)

In [ ]:
# average per organ
df_z = pd.concat(df_zs, axis=1).T.fillna(0)
df_z['organ'] = df_z.index.map(anno[['tag','organ']].value_counts().reset_index().set_index('tag')['organ'])
df_z['organ'] += ':' + df_z.index.map(anno[['tag','age_cat']].value_counts().reset_index().set_index('tag')['age_cat'])
df_z['organ'] = df_z['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
df_z = df_z.groupby('organ').mean()
df_z = (df_z.T / baseline_random_organ).T

In [ ]:
from tqdm import tqdm
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_curve, auc, precision_recall_curve
# define the data, mean
xs = df_z.mean(0)
ys = 1 * df_z.columns.isin(df_intrxn.index)
xs = np.array(xs).reshape(-1, 1); ys = np.array(ys)
xs_mean = xs.copy()
# setup the data splitter
seed = 0; np.random.seed(seed)
sss = StratifiedKFold(random_state=0, n_splits=10, shuffle=True)
df_stat = pd.DataFrame(columns=['auroc','auprc','auroc_rand','auprc_rand','method'])
# setup the random seed and loop
for idxs, idys in tqdm(sss.split(xs, ys), total=10):
    data = xs[idxs].copy(), ys[idxs].copy()
    # train the model
    clf = LogisticRegression(random_state=seed)
    clf.fit(*data)
    # get the predictions
    preds = clf.predict_proba(xs[idys])
    preds = preds[:, clf.classes_ == 1][:, 0]
    preds_bin = clf.predict(xs[idys])
    auroc = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
    auprc = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
    # retrieve random predictions
    np.random.shuffle(preds)
    np.random.shuffle(preds_bin)
    auroc_rand = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
    auprc_rand = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
    df_stat.loc[df_stat.shape[0]] = auroc, auprc, auroc_rand, auprc_rand, 'mean'

In [ ]:
# define the data, max
xs = df_z.max(0)
ys = 1 * df_z.columns.isin(df_intrxn.index)
xs = np.array(xs).reshape(-1, 1); ys = np.array(ys)
xs_max = xs.copy()
# setup the data splitter
seed = 0; np.random.seed(seed)
sss = StratifiedKFold(random_state=0, n_splits=10, shuffle=True)
# setup the random seed and loop
for idxs, idys in tqdm(sss.split(xs, ys), total=10):
    data = xs[idxs].copy(), ys[idxs].copy()
    # train the model
    clf = LogisticRegression(random_state=seed)
    clf.fit(*data)
    # get the predictions
    preds = clf.predict_proba(xs[idys])
    preds = preds[:, clf.classes_ == 1][:, 0]
    preds_bin = clf.predict(xs[idys])
    auroc = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
    auprc = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
    # retrieve random predictions
    np.random.shuffle(preds)
    np.random.shuffle(preds_bin)
    auroc_rand = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
    auprc_rand = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
    df_stat.loc[df_stat.shape[0]] = auroc, auprc, auroc_rand, auprc_rand, 'max'

In [ ]:
# find the median
df_stat.groupby('method').median()

In [ ]:
# find the average
df_stat.groupby('method').mean()

In [ ]:
# find the standard error
df_stat.groupby('method').std() / np.sqrt(10)

In [ ]:
import statsmodels.api as sm
# define the data, obsmexp
model = sm.OLS(exog=sm.add_constant(df_z.mean(0).reset_index().set_index('index')[0]),
               endog=df_z.max(0).reset_index().set_index('index')[0])
model = model.fit()
summary = model.summary()
exp = model.predict(sm.add_constant(df_z.mean(0).reset_index().set_index('index')[0]))
xs = df_z.max(0).reset_index().set_index('index')[0] - exp
ys = 1 * df_z.columns.isin(df_intrxn.index)
xs = np.array(xs).reshape(-1, 1); ys = np.array(ys)
xs_obsmexp = xs.copy()
# setup the data splitter
seed = 0; np.random.seed(seed)
sss = StratifiedKFold(random_state=0, n_splits=10, shuffle=True)
# setup the random seed and loop
for idxs, idys in tqdm(sss.split(xs, ys), total=10):
    data = xs[idxs].copy(), ys[idxs].copy()
    # train the model
    clf = LogisticRegression(random_state=seed)
    clf.fit(*data)
    # get the predictions
    preds = clf.predict_proba(xs[idys])
    preds = preds[:, clf.classes_ == 1][:, 0]
    preds_bin = clf.predict(xs[idys])
    auroc = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
    auprc = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
    # retrieve random predictions
    np.random.shuffle(preds)
    np.random.shuffle(preds_bin)
    auroc_rand = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
    auprc_rand = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
    df_stat.loc[df_stat.shape[0]] = auroc, auprc, auroc_rand, auprc_rand, 'obsmexp'

In [ ]:
# melt the data and plot accordingly
data = df_stat.melt(id_vars='method')
fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
sns.boxplot(x='method', y='value', hue='variable', data=data, hue_order=['auprc','auprc_rand'],
            order=['mean','max'], palette=['xkcd:bright lilac','lightgray'], saturation=1, linewidth=1.5, linecolor='k')
ax.legend(bbox_to_anchor=(.99, 1.05), bbox_transform=ax.transAxes, loc='upper left', frameon=False)
# add some extra space on top and compute p-values
ylim = ax.get_ylim(); yrange = ylim[1] - ylim[0]; ax.set_ylim(ylim[0], ylim[1]+yrange*0.1)
# compute p-values
for method in ['mean','max']:
    p = ss.mannwhitneyu(data.loc[(data['method'] == method)&(data['variable'] == 'auprc'), 'value'],
                        data.loc[(data['method'] == method)&(data['variable'] == 'auprc_rand'), 'value'])[1]
    print(method, p)
for n in range(2, 4):
    ax.patches[n].set_hatch('//')
ax.set_xticklabels(['Mean','Max'])
ax.set_xlabel('Metric utilized')
ax.set_ylabel('Area under precision-\nrecall curve (AUPRC)')
# change name and color
children = ax.legend_.get_children()[0].get_children()[-1].get_children()[0].get_children()
children[0].get_children()[-1].get_children()[0].set_text('Observed')
children[-1].get_children()[-1].get_children()[0].set_text('Randomly\nshuffled')
children[-1].get_children()[0].get_children()[0].set_hatch('//')

In [ ]:
# melt the data and plot accordingly
data = df_stat.melt(id_vars='method')
fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
sns.boxplot(x='method', y='value', hue='variable', data=data, hue_order=['auroc','auroc_rand'],
            order=['mean','max'], palette=['xkcd:bright lilac','lightgray'], saturation=1, linewidth=1.5, linecolor='k')
ax.legend(bbox_to_anchor=(.99, 1.05), bbox_transform=ax.transAxes, loc='upper left', frameon=False)
# add some extra space on top and compute p-values
ylim = ax.get_ylim(); yrange = ylim[1] - ylim[0]; ax.set_ylim(ylim[0], ylim[1]+yrange*0.1)
# compute p-values
for method in ['mean','max']:
    p = ss.mannwhitneyu(data.loc[(data['method'] == method)&(data['variable'] == 'auroc'), 'value'],
                        data.loc[(data['method'] == method)&(data['variable'] == 'auroc_rand'), 'value'])[1]
    print(method, p)
for n in range(2, 4):
    ax.patches[n].set_hatch('//')
ax.set_xticklabels(['Mean','Max'])
ax.set_xlabel('Metric utilized')
ax.set_ylabel('Area under receiver operating\ncharacteristic curve (AUROC)')
# change name and color
children = ax.legend_.get_children()[0].get_children()[-1].get_children()[0].get_children()
children[0].get_children()[-1].get_children()[0].set_text('Observed')
children[-1].get_children()[-1].get_children()[0].set_text('Randomly\nshuffled')
children[-1].get_children()[0].get_children()[0].set_hatch('//')

In [ ]:
# derive the relevant receptors involved in certain interactions
metab_receptors = df_intrxn_gpcrs['receptor'][~df_intrxn_gpcrs['is_ppi']].unique()
secr_receptors = df_intrxn_gpcrs['receptor'][(df_intrxn_gpcrs['is_ppi'])&(df_intrxn_gpcrs['annotation'] == 'Secretion')&(df_intrxn_gpcrs['annotation'] != 'Adhesion-Adhesion')].unique()
nonsecr_receptors = df_intrxn_gpcrs['receptor'][(df_intrxn_gpcrs['is_ppi'])&(df_intrxn_gpcrs['annotation'] != 'Secretion')&(df_intrxn_gpcrs['annotation'] != 'Adhesion-Adhesion')].unique()
adhes_receptors = df_intrxn_gpcrs['receptor'][(df_intrxn_gpcrs['directionality'] == 'Adhesion-Adhesion')].unique()

In [ ]:
# compute seperately per ligand
df_stat_overall = df_stat.copy()

In [ ]:
# define the lrs
lrs = df_z.columns.to_series().str.split(':', expand=True)
df_stat = pd.DataFrame(columns=['auroc','auprc','auroc_rand','auprc_rand','method','receptor_type'])
params = [[metab_receptors, 'metab'], [secr_receptors, 'secr'],
          [nonsecr_receptors, 'nonsecr'], [adhes_receptors, 'adhes']]
for receptors, name in params:
    # define the data, mean
    mask = lrs[1].isin(receptors)
    xs = df_z.loc[:, mask].mean(0)
    ys = 1 * df_z.columns[mask].isin(df_intrxn.index)
    xs = np.array(xs).reshape(-1, 1); ys = np.array(ys)
    # setup the data splitter
    seed = 0; np.random.seed(seed)
    sss = StratifiedKFold(random_state=0, n_splits=10, shuffle=True)
    # setup the random seed and loop
    for idxs, idys in tqdm(sss.split(xs, ys), total=10):
        data = xs[idxs].copy(), ys[idxs].copy()
        # train the model
        clf = LogisticRegression(random_state=seed)
        clf.fit(*data)
        # get the predictions
        preds = clf.predict_proba(xs[idys])
        preds = preds[:, clf.classes_ == 1][:, 0]
        preds_bin = clf.predict(xs[idys])
        auroc = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
        auprc = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
        # retrieve random predictions
        np.random.shuffle(preds)
        np.random.shuffle(preds_bin)
        auroc_rand = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
        auprc_rand = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
        df_stat.loc[df_stat.shape[0]] = auroc, auprc, auroc_rand, auprc_rand, 'mean', name

In [ ]:
# melt the data and plot accordingly
data = df_stat.melt(id_vars=['receptor_type','method'])
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
sns.boxplot(x='receptor_type', y='value', hue='variable', data=data, hue_order=['auprc','auprc_rand'],
            palette=['xkcd:bright lilac','lightgray'], saturation=1, linewidth=1.5, linecolor='k')
ax.legend(bbox_to_anchor=(.99, .5), bbox_transform=ax.transAxes, loc='center left', frameon=False)
# add some extra space on top and compute p-values
ylim = ax.get_ylim(); yrange = ylim[1] - ylim[0]; ax.set_ylim(ylim[0], ylim[1]+yrange*0.1)
# compute p-values
for name in ['metab','secr','nonsecr','adhes']:
    p = ss.mannwhitneyu(data.loc[(data['receptor_type'] == name)&(data['variable'] == 'auprc'), 'value'],
                        data.loc[(data['receptor_type'] == name)&(data['variable'] == 'auprc_rand'), 'value'])[1]
    print(name, p)
for n, c in zip(range(4), ['#cc78bc', '#56b4e9', '#de8f05', '#0173b2']):
    ax.patches[n].set_color(c)
    ax.patches[n].set_edgecolor('k')
for n in range(4, 8):
    ax.patches[n].set_hatch('//')
ax.set_xticklabels(['Metabolite\n(Secreted)','Protein\n(Secreted)','Protein\n(Non-Secreted)', 'Protein\n(Adhesion)'])
ax.tick_params(axis='x', labelrotation=90)
ax.set(xlabel='Ligand-receptor class', ylabel='Area under precision-\nrecall curve (AUPRC)')

In [ ]:
# melt the data and plot accordingly
data = df_stat.melt(id_vars=['receptor_type','method'])
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
sns.boxplot(x='receptor_type', y='value', hue='variable', data=data, hue_order=['auroc','auroc_rand'],
            palette=['xkcd:bright lilac','lightgray'], saturation=1, linewidth=1.5, linecolor='k')
ax.legend(bbox_to_anchor=(.99, .5), bbox_transform=ax.transAxes, loc='center left', frameon=False)
# add some extra space on top and compute p-values
ylim = ax.get_ylim(); yrange = ylim[1] - ylim[0]; ax.set_ylim(ylim[0], ylim[1]+yrange*0.1)
# compute p-values
for name in ['metab','secr','nonsecr','adhes']:
    p = ss.mannwhitneyu(data.loc[(data['receptor_type'] == name)&(data['variable'] == 'auroc'), 'value'],
                        data.loc[(data['receptor_type'] == name)&(data['variable'] == 'auroc_rand'), 'value'])[1]
    print(name, p)
for n, c in zip(range(4), ['#cc78bc', '#56b4e9', '#de8f05', '#0173b2']):
    ax.patches[n].set_color(c)
    ax.patches[n].set_edgecolor('k')
for n in range(4, 8):
    ax.patches[n].set_hatch('//')
ax.set_xticklabels(['Metabolite\n(Secreted)','Protein\n(Secreted)','Protein\n(Non-Secreted)', 'Protein\n(Adhesion)'])
ax.tick_params(axis='x', labelrotation=90)
ax.set(xlabel='Ligand-receptor class', ylabel='Area under receiver operating\ncharacteristic curve (AUROC)')

In [ ]:
df_stat.groupby(['method','receptor_type']).median()

In [ ]:
df_stat.groupby(['method','receptor_type']).mean()

In [ ]:
df_stat.groupby(['method','receptor_type']).std() / np.sqrt(10)

In [ ]:
# save receptor type
df_stat_rtype = df_stat.copy()

In [ ]:
# keep track of predictions
df_stat = pd.DataFrame(columns=['auroc','auprc','auroc_rand','auprc_rand','method','dataset']); preds_ = []
for dataset in tqdm(df_z.index):
    # define the data, mean
    xs = df_z.loc[dataset]
    ys = 1 * df_z.columns.isin(df_intrxn.index)
    xs = np.array(xs).reshape(-1, 1); ys = np.array(ys)
    # setup the data splitter
    seed = 0; np.random.seed(seed)
    sss = StratifiedKFold(random_state=0, n_splits=10, shuffle=True)
    # setup the random seed and loop
    for idxs, idys in sss.split(xs, ys):
        data = xs[idxs].copy(), ys[idxs].copy()
        # train the model
        clf = LogisticRegression(random_state=seed)
        clf.fit(*data)
        # get the predictions
        preds = clf.predict_proba(xs[idys])
        preds = preds[:, clf.classes_ == 1][:, 0]
        preds_.append(pd.Series(preds, index=df_z.columns[idys], name=dataset))
        preds_bin = clf.predict(xs[idys])
        auroc = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
        auprc = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
        # retrieve random predictions
        np.random.shuffle(preds)
        np.random.shuffle(preds_bin)
        auroc_rand = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
        auprc_rand = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
        df_stat.loc[df_stat.shape[0]] = auroc, auprc, auroc_rand, auprc_rand, 'mean', dataset

In [ ]:
from matplotlib.cm import get_cmap
from matplotlib.colors import to_hex
# derive a color palette
cmap = get_cmap('Purples')
# melt the data and plot accordingly
fig, ax = plt.subplots(figsize=[8, 4]); ax.grid(False)
values = df_stat.groupby('dataset').mean(numeric_only=True)['auprc'].sort_values()
# derive colors and order
order = values.index; vmin = values.min(); vmax = values.max(); print(vmin, vmax)
sns.boxplot(x='dataset', y='auprc', data=df_stat, saturation=1, linewidth=1.5, linecolor='k',
            order=order[::-1], palette=[to_hex(cmap((x-vmin)/(vmax-vmin))) for x in values[::-1]])
ax.legend(bbox_to_anchor=(.99, .5), bbox_transform=ax.transAxes, loc='center left', frameon=False)
# add some extra space on top and compute p-values
ax.set_xlim(-1, df_z.shape[0]); ax.tick_params(axis='x', labelrotation=90)
ylim = ax.get_ylim(); yrange = ylim[1] - ylim[0]; ax.set_ylim(ylim[0], ylim[1]+yrange*0.1)
ax.set(xlabel='Tissue', ylabel='Area under precision-\nrecall curve (AUPRC)')
labels = values.index.str.capitalize().str.replace(' – ', ' in ')\
.str.replace('Cns','CNS').str.replace('avn','atrioventricular node').str.replace('Prostate','Reproductive (prostate)')
ax.set_xticklabels(labels[::-1])

In [ ]:
# melt the data and plot accordingly
fig, ax = plt.subplots(figsize=[8, 4]); ax.grid(False)
values = df_stat.groupby('dataset').mean(numeric_only=True)['auroc'].sort_values()
# derive colors and order
order = values.index; vmin = values.min(); vmax = values.max(); print(vmin, vmax)
sns.boxplot(x='dataset', y='auroc', data=df_stat, saturation=1, linewidth=1.5, linecolor='k',
            order=order[::-1], palette=[to_hex(cmap((x-vmin)/(vmax-vmin))) for x in values[::-1]])
ax.legend(bbox_to_anchor=(.99, .5), bbox_transform=ax.transAxes, loc='center left', frameon=False)
# add some extra space on top and compute p-values
ax.plot([-1, df_z.shape[0]], [0.5]*2, color='k', linestyle='--')
ax.set_xlim(-1, df_z.shape[0]); ax.tick_params(axis='x', labelrotation=90)
ylim = ax.get_ylim(); yrange = ylim[1] - ylim[0]; ax.set_ylim(ylim[0], ylim[1]+yrange*0.1)

In [ ]:
# look at the predictions
preds = pd.concat(preds_, axis=1).T.reset_index()
preds = preds.groupby('index').mean().T
df_p = preds.corr(method=get_p).astype(float)
for idx in df_p.index: df_p.loc[idx, idx] = 0
mask = df_p.astype(float) >= 0.05

In [ ]:
# demostrate organ to organ similarity
g = sns.clustermap(preds.corr(), cmap='magma', method='ward', cbar_pos=(0, 1, .01, .1), mask=mask,
                   figsize=[8, 8], dendrogram_ratio=.2, xticklabels=1, yticklabels=1, vmax=0.8, vmin=-0.2)
g.ax_heatmap.grid(False); g.ax_cbar.grid(False)
g.ax_heatmap.tick_params(labelsize=10); g.ax_heatmap.set_facecolor('lightgray')

In [ ]:
# add on the extra information
anno_tmp = anno.copy()
anno_tmp['tmp'] = anno[['tag','organ']].value_counts().reset_index().set_index('tag')['organ'].values
anno_tmp['tmp'] += ':' + anno[['tag','age_cat']].value_counts().reset_index().set_index('tag')['age_cat'].values
anno_tmp['tmp'] = anno_tmp['tmp'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')

In [ ]:
# derive coefficients of variation
df_info['coef_of_var'] = np.nan
for idx in tqdm(range(len(adatas)), total=len(adatas)):
    try: coef_of_var = np.nanmean(np.sqrt(adatas[idx].X.mean(axis=0).A1) / adatas[idx].X.toarray().var(axis=0))
    except: coef_of_var = np.nanmean(np.sqrt(adatas[idx].X.mean(axis=0)) / adatas[idx].X.var(axis=0))
    df_info.loc[idx, 'coef_of_var'] = coef_of_var
anno_tmp['coef_of_var'] = anno_tmp['tag'].map(df_info.set_index('tag')['coef_of_var'])

In [ ]:
# calculate sparsity
df_info['sparsity'] = np.nan
for idx in tqdm(range(len(adatas)), total=len(adatas)):
    df_info.loc[idx, 'sparsity'] = 1 - np.mean(adatas[idx].X != 0)
anno_tmp['sparsity'] = anno_tmp['tag'].map(df_info.set_index('tag')['sparsity'])

In [ ]:
from scipy.spatial.distance import pdist
# calculate connectivity
df_info['connectivity'] = np.nan
for idx in tqdm(range(len(adatas)), total=len(adatas)):
    df_info.loc[idx, 'connectivity'] = np.mean(pdist(adatas[idx].obsm['spatial']) < 2.5)
anno_tmp['connectivity'] = anno_tmp['tag'].map(df_info.set_index('tag')['connectivity'])

In [ ]:
# compile the results for z-score of known LRs with rand LRs as control
df_zs = []
for result, tag in zip(results, df_info['tag']):
    if result is None: continue
    # grab the file id
    file_id = list(result.keys())[0]
    # grab the result object
    result = result[file_id]
    # retrieve the proper dataframe
    known, rand = result['known']['z'].copy(), result['rand']['z'].copy()
    known.index += '}{KNOWN'; rand.index += '}{RAND'
    df_z = pd.concat([known, rand], axis=0).copy()
    df_z.name = tag
    df_zs.append(df_z)

In [ ]:
# compute the expression profiles
df_z = pd.concat(df_zs, axis=1).T.fillna(0)
df_z_known, df_z_test = df_z.loc[:, df_z.columns.str.endswith('}{KNOWN')], df_z.loc[:, df_z.columns.str.endswith('}{RAND')]
df_z_known.columns = df_z_known.columns.str.slice(0, -7)
df_z_test.columns = df_z_test.columns.str.slice(0, -6)
# derive a baseline based on random
baseline_random = df_z_test.mean(1)
# and normalize data with it
df_z_known = (df_z_known.T / baseline_random).T

In [ ]:
# keep track of predictions
df_stat_hires = pd.DataFrame(columns=['auroc','auprc','auroc_rand','auprc_rand','method','dataset']); preds_ = []
for dataset in tqdm(df_z_known.index):
    # define the data, mean
    xs = df_z_known.loc[dataset]
    ys = 1 * df_z_known.columns.isin(df_intrxn.index)
    xs = np.array(xs).reshape(-1, 1); ys = np.array(ys)
    # setup the data splitter
    seed = 0; np.random.seed(seed)
    sss = StratifiedKFold(random_state=0, n_splits=10, shuffle=True)
    # setup the random seed and loop
    for idxs, idys in sss.split(xs, ys):
        data = xs[idxs].copy(), ys[idxs].copy()
        # train the model
        clf = LogisticRegression(random_state=seed)
        clf.fit(*data)
        # get the predictions
        preds = clf.predict_proba(xs[idys])
        preds = preds[:, clf.classes_ == 1][:, 0]
        preds_.append(pd.Series(preds, index=df_z_known.columns[idys], name=dataset))
        preds_bin = clf.predict(xs[idys])
        auroc = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
        auprc = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
        # retrieve random predictions
        np.random.shuffle(preds)
        np.random.shuffle(preds_bin)
        auroc_rand = auc(*roc_curve(1 * (ys[idys] == 1), preds)[:-1])
        auprc_rand = auc(*precision_recall_curve(1 * (ys[idys] == 1), preds)[:-1][::-1])
        df_stat_hires.loc[df_stat_hires.shape[0]] = auroc, auprc, auroc_rand, auprc_rand, 'mean', dataset

In [ ]:
# look at correlations
anno_mean = anno_tmp[anno_tmp['tag'].isin(df_stat_hires['dataset'])].copy()
anno_mean['auroc'] = anno_mean['tag'].map(df_stat_hires.groupby('dataset').mean(numeric_only=True)['auroc']).values
anno_mean['auprc'] = anno_mean['tag'].map(df_stat_hires.groupby('dataset').mean(numeric_only=True)['auprc']).values
# check the correlation
cols = ['n-spots', 'coef_of_var',  'sparsity', 'connectivity', 'auroc', 'auprc']
g = sns.clustermap(anno_mean[cols].corr(method='pearson'), vmin=-0.5, vmax=1, cmap='magma', method='ward', figsize=[5, 5],
                   dendrogram_ratio=.2, cbar_pos=(0, 1, .01, .1))
g.ax_heatmap.grid(False)

In [ ]:
# measure against n-spots
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
xs, ys = anno_mean['auroc'], anno_mean['n-spots']
ax.scatter(xs, ys, lw=1.5, alpha=0.75,
           edgecolor='xkcd:bright lilac', color='xkcd:pale lavender', s=20)
model = np.polynomial.Polynomial(0)
model = model.fit(xs, ys, 1)
xlim, ylim = ax.get_xlim(), ax.get_ylim()
xl, yl = model.linspace(domain=xlim)
ax.plot(xl, yl, color='k')
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set(xlabel='AUROC', ylabel='# of spots')
ss.pearsonr(xs, ys)

In [ ]:
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
xs, ys = anno_mean['auprc'], anno_mean['n-spots']
ax.scatter(xs, ys, lw=1.5, alpha=0.75,
           edgecolor='xkcd:bright lilac', color='xkcd:pale lavender', s=20)
model = np.polynomial.Polynomial(0)
model = model.fit(xs, ys, 1)
xlim, ylim = ax.get_xlim(), ax.get_ylim()
xl, yl = model.linspace(domain=xlim)
ax.plot(xl, yl, color='k')
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set(xlabel='Area under precision-\nrecall curve (AUPRC)', ylabel='# of observations / dataset')
ss.pearsonr(xs, ys)

In [ ]:
# as well as sparsity
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
xs, ys = anno_mean['auroc'], anno_mean['sparsity']
ax.scatter(xs, ys, lw=1.5, alpha=0.75,
           edgecolor='xkcd:bright lilac', color='xkcd:pale lavender', s=20)
model = np.polynomial.Polynomial(0)
model = model.fit(xs, ys, 1)
xlim, ylim = ax.get_xlim(), ax.get_ylim()
xl, yl = model.linspace(domain=xlim)
ax.plot(xl, yl, color='k')
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set(xlabel='AUROC', ylabel='sparsity')
ss.pearsonr(xs, ys)

In [ ]:
# as well as sparsity
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
xs, ys = anno_mean['auprc'], anno_mean['sparsity']
ax.scatter(xs, ys, lw=1.5, alpha=0.75,
           edgecolor='xkcd:bright lilac', color='xkcd:pale lavender', s=20)
model = np.polynomial.Polynomial(0)
model = model.fit(xs, ys, 1)
xlim, ylim = ax.get_xlim(), ax.get_ylim()
xl, yl = model.linspace(domain=xlim)
ax.plot(xl, yl, color='k')
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set(xlabel='Area under precision-\nrecall curve (AUPRC)', ylabel='Sparsity of counts')
ss.pearsonr(xs, ys)

In [ ]:
# compile the results for z-score of gpcr screen LRs
df_zs = []
for result, tag in zip(results, df_info['tag']):
    if result is None: continue
    # grab the file id
    file_id = list(result.keys())[0]
    # grab the result object
    result = result[file_id]
    # retrieve the proper dataframe
    df_z = result['known']['z'].copy()
    df_z.name = tag
    df_zs.append(df_z)
# average per organ
df_z = pd.concat(df_zs, axis=1).T.fillna(0)
df_z['organ'] = df_z.index.map(anno[['tag','organ']].value_counts().reset_index().set_index('tag')['organ'])
df_z['organ'] += ':' + df_z.index.map(anno[['tag','age_cat']].value_counts().reset_index().set_index('tag')['age_cat'])
df_z['organ'] = df_z['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
df_z = df_z.groupby('organ').mean()
df_z = (df_z.T / baseline_random_organ).T

In [ ]:
from scipy.spatial.distance import cdist, pdist, squareform
# retrieve ligands with multiple receptors
ligands = df_intrxn_gpcrs['ligand'].value_counts().index[df_intrxn_gpcrs['ligand'].value_counts() >= 2]
# compute the distance in spatial patterning
df_dist = pd.DataFrame(columns=['ligand','in','out','n'])
for ligand in ligands:
    if sum(df_z.columns.str.startswith(ligand) & df_z.columns.isin(df_intrxn.index)) < 2: continue
    if sum(df_z.columns.str.startswith(ligand) & (~df_z.columns.isin(df_intrxn.index))) < 2: continue
    d_in = pdist(df_z[df_z.columns[df_z.columns.str.startswith(ligand) & df_z.columns.isin(df_intrxn.index)]].T).mean()
    d_ou = cdist(df_z[df_z.columns[df_z.columns.str.startswith(ligand) & df_z.columns.isin(df_intrxn.index)]].T,
                 df_z[df_z.columns[df_z.columns.str.startswith(ligand) & (~df_z.columns.isin(df_intrxn.index))]].T).mean()
    df_dist.loc[df_dist.shape[0]] = ligand, d_in, d_ou, sum(df_z.columns.str.startswith(ligand) & df_z.columns.isin(df_intrxn.index))

In [ ]:
# derive the relevant ligands involved in certain interactions
metab_ligands = df_intrxn_gpcrs['ligand'][~df_intrxn_gpcrs['is_ppi']].unique()
secr_ligands = df_intrxn_gpcrs['ligand'][(df_intrxn_gpcrs['is_ppi'])&(df_intrxn_gpcrs['annotation'] == 'Secretion')&(df_intrxn_gpcrs['annotation'] != 'Adhesion-Adhesion')].unique()
nonsecr_ligands = df_intrxn_gpcrs['ligand'][(df_intrxn_gpcrs['is_ppi'])&(df_intrxn_gpcrs['annotation'] != 'Secretion')&(df_intrxn_gpcrs['annotation'] != 'Adhesion-Adhesion')].unique()
adhes_ligands = df_intrxn_gpcrs['ligand'][(df_intrxn_gpcrs['directionality'] == 'Adhesion-Adhesion')].unique()

In [ ]:
# adhesions do not have enough 1LMR
df_metab = df_dist[df_dist['ligand'].isin(metab_ligands)].melt(id_vars=['ligand'])
df_metab['annotation'] = 'Metabolite (Secreted)'
df_secr = df_dist[df_dist['ligand'].isin(secr_ligands)].melt(id_vars=['ligand'])
df_secr['annotation'] = 'Protein (Secreted)'
df_nonsecr = df_dist[df_dist['ligand'].isin(nonsecr_ligands)].melt(id_vars=['ligand'])
df_nonsecr['annotation'] = 'Protein (Non-Secreted)'
df_adhes = df_dist[df_dist['ligand'].isin(adhes_ligands)].melt(id_vars=['ligand'])
df_adhes['annotation'] = 'Protein (Adhesion)'
df = pd.concat([df_metab, df_secr, df_nonsecr, df_adhes], axis=0)

In [ ]:
# save the values
df_dist_spatial, df_spatial = df_dist.copy(), df.copy()

In [ ]:
# retrieve ligands with multiple receptors
ligands = df_intrxn_gpcrs['ligand'].value_counts().index[df_intrxn_gpcrs['ligand'].value_counts() >= 2]
# compare maximal interaction scores
df_dist = pd.DataFrame(columns=['ligand','in','out','n'])
for ligand in tqdm(ligands):
    if sum(df_z.columns.str.startswith(ligand) & df_z.columns.isin(df_intrxn.index)) < 2: continue
    if sum(df_z.columns.str.startswith(ligand) & (~df_z.columns.isin(df_intrxn.index))) < 2: continue
    d_in = pdist(df_z[df_z.columns[df_z.columns.str.startswith(ligand) & df_z.columns.isin(df_intrxn.index)]].max(0).values.reshape(-1, 1)).mean()
    d_ou = cdist(df_z[df_z.columns[df_z.columns.str.startswith(ligand) & df_z.columns.isin(df_intrxn.index)]].max(0).values.reshape(-1, 1),
                 df_z[df_z.columns[df_z.columns.str.startswith(ligand) & (~df_z.columns.isin(df_intrxn.index))]].max(0).values.reshape(-1, 1)).mean()
    df_dist.loc[df_dist.shape[0]] = ligand, d_in, d_ou, sum(df_z.columns.str.startswith(ligand) & df_z.columns.isin(df_intrxn.index))

In [ ]:
# adhesions do not have enough 1LMR
df_metab = df_dist[df_dist['ligand'].isin(metab_ligands)].melt(id_vars=['ligand'])
df_metab['annotation'] = 'Metabolite (Secreted)'
df_secr = df_dist[df_dist['ligand'].isin(secr_ligands)].melt(id_vars=['ligand'])
df_secr['annotation'] = 'Protein (Secreted)'
df_nonsecr = df_dist[df_dist['ligand'].isin(nonsecr_ligands)].melt(id_vars=['ligand'])
df_nonsecr['annotation'] = 'Protein (Non-Secreted)'
df_adhes = df_dist[df_dist['ligand'].isin(adhes_ligands)].melt(id_vars=['ligand'])
df_adhes['annotation'] = 'Protein (Adhesion)'
df = pd.concat([df_metab, df_secr, df_nonsecr, df_adhes], axis=0)

In [ ]:
# save the values
df_dist_max, df_max = df_dist.copy(), df.copy()

In [ ]:
# look at the broad distribution
fig, ax = plt.subplots(figsize=[1, 4]); ax.grid(False)
sns.boxplot(np.log2(df_dist_spatial['in'] / df_dist_spatial['out']), color='xkcd:bright lilac', linecolor='k', linewidth=1.5)
ax.plot([-1, 1], [0]*2, color='k', linestyle='--', zorder=0); ax.set_xlim(-1, 1)
ax.tick_params(axis='x', bottom=False, which='both')
ax.set_ylabel('Cross-organ interaction\npattern distance')
# calculate statistics
counts = (1 * (np.log2(df_dist_spatial['in'] / df_dist_spatial['out']) < 0)).value_counts().loc[[1, 0]]
exp_avg = counts.sum() / 2
exp = [exp_avg] * 2
p = ss.fisher_exact(np.array([exp, counts]))[1]
print(p)
# show the percent that are less than 0
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
_ = ax.pie(counts, colors=['xkcd:bright lilac','lightgray'], hatch=['','//'])
print(np.mean(np.log2(df_dist_spatial['in'] / df_dist_spatial['out']) < 0)); np.log2(df_dist_spatial['in'] / df_dist_spatial['out']).describe()

In [ ]:
# look at the broad distribution
fig, ax = plt.subplots(figsize=[1, 4]); ax.grid(False)
sns.boxplot(np.log2(df_dist_max['in'] / df_dist_max['out']), color='xkcd:bright lilac', linecolor='k', linewidth=1.5)
ax.plot([-1, 1], [0]*2, color='k', linestyle='--', zorder=0); ax.set_xlim(-1, 1)
ax.tick_params(axis='x', bottom=False, which='both')
ax.set_ylabel('Interaction intensity distance')
# calculate statistics
counts = (1 * (np.log2(df_dist_max['in'] / df_dist_max['out']) < 0)).value_counts().loc[[1, 0]]
exp_avg = counts.sum() / 2
exp = [exp_avg] * 2
p = ss.fisher_exact(np.array([exp, counts]))[1]
print(p)
# show the percent that are less than 0
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
_ = ax.pie(counts, colors=['xkcd:bright lilac','lightgray'], hatch=['','//'])
print(np.mean(np.log2(df_dist_max['in'] / df_dist_max['out']) < 0)); np.log2(df_dist_max['in'] / df_dist_max['out']).describe()

In [ ]:
# compare intra and inter differences in terms of spatial patterning
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
sns.boxplot(x='annotation', y='value', hue='variable', data=df_spatial[df_spatial['variable'].isin(['in','out'])],
            saturation=1, hue_order=['in','out'], linewidth=1.5, linecolor='k', palette=['xkcd:bright lilac','lightgray'])
ax.set_xlim(-1, 4); ax.tick_params(axis='x', labelrotation=90)
for x in df_spatial['annotation'].unique():
    p = ss.mannwhitneyu(df_spatial.loc[(df_spatial['annotation'] == x)&(df_spatial['variable'] == 'in'), 'value'],
                        df_spatial.loc[(df_spatial['annotation'] == x)&(df_spatial['variable'] == 'out'), 'value'])[1]
    print(method, p)
for n in range(4, 8):
    ax.patches[n].set_hatch('//')
ylim = ax.get_ylim(); yrange = ylim[1] - ylim[0]; ax.set_ylim(ylim[0], ylim[1]+yrange*0.15)
ax.set(xlabel='Ligand-receptor class', ylabel='Cross-organ interaction\npattern distance')
ax.legend(labels=['Binds the\nsame ligand','Binds any\nother ligands'],
          bbox_to_anchor=(.99, 1.05), bbox_transform=ax.transAxes, frameon=False, loc='upper left')

In [ ]:
# compare intra and inter differences in terms of spatial patterning
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
sns.boxplot(x='annotation', y='value', hue='variable', data=df_max[df_max['variable'].isin(['in','out'])],
            saturation=1, hue_order=['in','out'], linewidth=1.5, linecolor='k', palette=['xkcd:bright lilac','lightgray'])
ax.set_xlim(-1, 4); ax.tick_params(axis='x', labelrotation=90)
ax.legend(bbox_to_anchor=(.99, .5), bbox_transform=ax.transAxes, frameon=False, loc='center left')
for x in df_max['annotation'].unique():
    p = ss.mannwhitneyu(df_max.loc[(df_max['annotation'] == x)&(df_max['variable'] == 'in'), 'value'],
                        df_max.loc[(df_max['annotation'] == x)&(df_max['variable'] == 'out'), 'value'])[1]
    print(method, p)
for n in range(4, 8):
    ax.patches[n].set_hatch('//')
ylim = ax.get_ylim(); yrange = ylim[1] - ylim[0]; ax.set_ylim(ylim[0], ylim[1]+yrange*0.15)
ax.set(xlabel='Ligand-receptor class', ylabel='Interaction intensity distance')
ax.legend(labels=['Binds the\nsame ligand','Binds any\nother ligands'],
          bbox_to_anchor=(.99, 1.05), bbox_transform=ax.transAxes, frameon=False, loc='upper left')

### comparison with orphans

In [ ]:
# compile the results for z-score of orphan LRs
df_zs = []
for result, tag in zip(results, df_info['tag']):
    if result is None: continue
    # grab the file id
    file_id = list(result.keys())[0]
    # grab the result object
    result = result[file_id]
    # retrieve the proper dataframe
    df_z = result['orphan']['z'].copy()
    df_z.name = tag
    df_zs.append(df_z)

In [ ]:
# average per organ
df_z = pd.concat(df_zs, axis=1).T.fillna(0)
df_z['organ'] = df_z.index.map(anno[['tag','organ']].value_counts().reset_index().set_index('tag')['organ'])
df_z['organ'] += ':' + df_z.index.map(anno[['tag','age_cat']].value_counts().reset_index().set_index('tag')['age_cat'])
df_z['organ'] = df_z['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
df_z = df_z.groupby('organ').mean()
df_z = (df_z.T / baseline_random_organ).T

In [ ]:
# groupby receptor
df_z_r = df_z.T.copy()
df_z_r['receptor'] = df_z_r.index.to_series().str.split(':', expand=True)[1]
df_z_rmean = df_z_r.groupby('receptor').mean()

In [ ]:
# highlight the blank
g = sns.clustermap(df_z_rmean.T, vmin=-5, vmax=5, cmap='RdBu_r', method='ward', cbar_pos=(0, 1, .01, .1),
                   xticklabels=1, yticklabels=1, figsize=[6, 15][::-1], dendrogram_ratio=(.2, .05)[::-1])
g.ax_heatmap.grid(False); g.ax_heatmap.tick_params(labelsize=10)
g.ax_heatmap.tick_params(axis='y', labelrotation=180, labelsize=10)
labels = pd.Series([x.get_text() for x in g.ax_heatmap.get_yticklabels()])
_ = g.ax_heatmap.set_yticklabels(labels.str.capitalize().str.replace(' – ', ' in ')\
.str.replace('Cns','CNS').str.replace('avn','atrioventricular node').str.replace('Prostate','Reproductive (prostate)'))
g.ax_heatmap.set_xlabel('Orphan GPCR', fontsize=10, rotation=180)
g.ax_heatmap.set_ylabel('Tissue', fontsize=10)

In [ ]:
from sklearn.decomposition import PCA
data = df_z_rmean.copy().T
data -= data.mean()
data /= data.std()
pca = PCA(n_components=5)
pca.fit(data.T)
comps = pd.DataFrame(pca.transform(data.T), index=df_z_rmean.index)
evr = pca.explained_variance_ratio_

In [ ]:
# color on the data
fig, ax = plt.subplots(figsize=[2.5, 4]); ax.grid(False)
ax.bar([f'PC{x}' for x in range(1, 5+1)], evr, edgecolor='xkcd:bright lilac', color='xkcd:pale lavender', lw=1.5)
ax.set_ylabel('Explained variance ratio'); ax.tick_params(axis='x', labelrotation=90)

In [ ]:
# report the weights
weights = pd.DataFrame(pca.components_, columns=df_z_rmean.columns).T

In [ ]:
# figure out the order
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
ax.scatter(comps.loc['GPR85', 0], comps.loc['GPR85', 1], s=1e2, label='GPR85', lw=1.5,
           color='xkcd:pale lavender', edgecolor='xkcd:bright lilac', marker='s')
ax.scatter(comps.loc[comps.index != 'GPR85', 0], comps.loc[comps.index != 'GPR85', 1],
           s=50, color='lightgray', edgecolor='grey', label='Orphan\nGPCR', lw=1.5)
ax.set(xlabel='PC1', ylabel='PC2')
ax.legend(bbox_to_anchor=(.99, 1.05), bbox_transform=ax.transAxes, frameon=False)

In [ ]:
# figure out the order
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
ax.scatter(comps[0], comps[1], s=50, c=df_z_rmean['heart (right atrium) – adult'], vmax=5, vmin=-5, cmap='RdBu_r', edgecolor='grey')
ax.set(xlabel='PC1', ylabel='PC2', title='Heart (right atrium) – adult')

In [ ]:
# figure out the order
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
ax.scatter(comps[0], comps[1], s=50, c=df_z_rmean['cns (cerebral cortex) – adult'], vmax=2, vmin=-2, cmap='RdBu_r', edgecolor='grey')
ax.set(xlabel='PC1', ylabel='PC2', title='CNS (cerebral cortex) in adult')

In [ ]:
# figure out the order
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
ax.scatter(comps[0], comps[1], s=50, c=df_z_rmean['lung – adult'], vmax=2, vmin=-2, cmap='RdBu_r', edgecolor='grey')
ax.set(xlabel='PC1', ylabel='PC2', title='Lung in adult')

In [ ]:
# figure out the order
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
ax.scatter(comps[0], comps[1], s=50, c=df_z_rmean['thymus – fetal'], vmax=5, vmin=-5, cmap='RdBu_r', edgecolor='grey')
ax.set(xlabel='PC1', ylabel='PC2', title='Thymus in fetal')

In [ ]:
# plot the data
fig, ax = plt.subplots(figsize=[8, 4]); ax.grid(False)
data = weights[1].sort_values()
ax.bar(data.index, data, edgecolor='xkcd:bright lilac', color='xkcd:pale lavender', lw=1.5)
ax.set(xlabel='Tissue', ylabel='Contribution to PC2')
ax.plot([-1, len(data)], [0]*2, color='k', linestyle='--'); ax.set_xlim(-1, len(data))
# retrieve labels
ax.tick_params(axis='x', labelrotation=90)
ax.set_xticklabels(data.index.str.capitalize().str.replace(' – ', ' in ')\
.str.replace('Cns','CNS').str.replace('avn','atrioventricular node').str.replace('Prostate','Reproductive (prostate)'))

In [ ]:
# average per organ
df_z = pd.concat(df_zs, axis=1).T.fillna(0)
df_z['organ'] = df_z.index.map(anno[['tag','organ']].value_counts().reset_index().set_index('tag')['organ'])
df_z['organ'] += ':' + df_z.index.map(anno[['tag','age_cat']].value_counts().reset_index().set_index('tag')['age_cat'])
df_z['organ'] = df_z['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
df_z = df_z.groupby('organ').mean()
df_z = (df_z.T / baseline_random_organ).T
df_z = df_z.loc[:, df_z.columns.str.endswith(':GPR85')]
df_z.columns = df_z.columns.str.replace(':GPR85','')
# calculate linkages
from scipy.cluster.hierarchy import linkage, fcluster
print('calculating linkages for rows')
Z_row = linkage(df_z, method='ward')
Z_row_organ_gpr85 = Z_row.copy()
print('calculating linkages for columns')
Z_col = linkage(df_z.T, method='ward')
Z_col_organ_gpr85 = Z_col.copy()

In [ ]:
# create clustermap
g = sns.clustermap(df_z, figsize=[20, 8], vmin=-15, vmax=15, cmap='RdBu_r', cbar_pos=(0, 1, .01, .1), dendrogram_ratio=(.05, .1),
                   row_linkage=Z_row_organ_gpr85, col_linkage=Z_col_organ_gpr85, xticklabels=0, yticklabels=1)
g.ax_heatmap.grid(False)

In [ ]:
# create clustermap
col_colors = df_z.columns.to_series().str.startswith('Acetylcholine').map({True:'tab:red'}).fillna('lightgray')
g = sns.clustermap(df_z, figsize=[20, 8], vmin=-15, vmax=15, cmap='RdBu_r', cbar_pos=(0, 1, .01, .1), dendrogram_ratio=(.05, .1),
                   row_linkage=Z_row_organ_gpr85, col_linkage=Z_col_organ_gpr85, xticklabels=0, yticklabels=1,
                   col_colors=col_colors, colors_ratio=0.02)
g.ax_heatmap.grid(False); g.ax_col_colors.grid(False)

In [ ]:
# melt the gpr85 dataframe
df_z_melt = df_z.reset_index().melt(id_vars='organ')
# rename accordingly
df_z_melt['organ_renamed'] = df_z_melt['organ'].str.capitalize().str.replace(' – ', ' in ')\
.str.replace('Cns','CNS').str.replace('avn','atrioventricular node').str.replace('Prostate','Reproductive (prostate)')
# create a bar plot of the expression
fig, ax = plt.subplots(figsize=[10, 4]); ax.grid(False)
order = df_z_melt.groupby('organ_renamed').mean(numeric_only=True)['value'].sort_values().index
sns.barplot(x='organ_renamed', y='value', data=df_z_melt, color='xkcd:pale lavender',
            edgecolor='xkcd:bright lilac', errcolor='xkcd:bright lilac', ci=68,
            linewidth=1.5, capsize=0.3, errwidth=1.5, saturation=1, order=order)
ax.tick_params(axis='x', labelrotation=90)
ax.plot([-1, len(order)], [0]*2, color='k')
ax.set_xlim(-1, len(order))
ax.set(xlabel='Tissue', ylabel='Interaction Z-score (normalized)')

In [ ]:
# compare the CNS and non-CNS adult and non-adult
df_z_melt['cat_simple'] = 'CNS'
df_z_melt.loc[~df_z_melt['organ'].str.startswith('cns'), 'cat_simple'] = 'Non-CNS'
df_z_melt.loc[df_z_melt['organ'].str.endswith('adult'), 'cat_simple'] += ' in adult'
df_z_melt.loc[df_z_melt['organ'].str.endswith('fetal'), 'cat_simple'] += ' in fetal'
# get the color of the different organs
fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
order = sorted(df_z_melt['cat_simple'].unique())
sns.barplot(x='cat_simple', y='value', data=df_z_melt, order=order, color='xkcd:pale lavender',
            ci=68, errwidth=1.5, capsize=0.3, edgecolor='xkcd:bright lilac', errcolor='xkcd:bright lilac', linewidth=1.5)
ax.tick_params(axis='x', labelrotation=90)
ax.set_xlim(-1, len(order))
ax.set(xlabel='', ylabel='Interaction Z-score (normalized)')
ax.legend(bbox_to_anchor=(1.05, .99), bbox_transform=ax.transAxes, frameon=False, loc='upper left')
# perform all p-value comparisons
for tag in df_z_melt['cat_simple'].unique():
    if tag == 'CNS in adult': continue
    p = ss.mannwhitneyu(df_z_melt.loc[df_z_melt['cat_simple'] == 'CNS in adult', 'value'],
                        df_z_melt.loc[df_z_melt['cat_simple'] == tag, 'value'])[1]
    print(tag, p)

In [ ]:
# read in the data from HPA
df = pd.read_table('/home/dchen2/LITERATURE/HPA/rna_tissue_gtex.tsv.zip')
# retrieve the data and order
data = df.loc[df['Gene name'] == 'GPR85']
data = data.sort_values('TPM')
# create CNS buckets
non_cns = ['pancreas','skeletal muscle','liver','ovary','esophagus','stomach','skin','adrenal gland','small intestine','cervix','vagina','thyroid gland','colon','breast','salivary gland','kidney','fallopian tube','prostate','adipose tissue','lung','endometrium','urinary bladder','heart muscle','pituitary gland','spleen','testis']
cns = ['amygdala','spinal cord','hypothalamus','substantia nigra','nucleus accumbens','hippocampus','cerebral cortex','caudate','putamen','cerebellum','retina']
data['is_cns'] = 'CNS'
data.loc[data['Tissue'].isin(non_cns), 'is_cns'] = 'Non-CNS'
# plot the data
fig, ax = plt.subplots(figsize=[12, 4]); ax.grid(False)
ax.bar(data['Tissue'].str.capitalize(), data['TPM'], edgecolor='xkcd:bright lilac', color='xkcd:pale lavender', lw=1.5)
ax.set(xlabel='Tissue', ylabel='Transcripts per million from GTEx')
# retrieve labels
ax.tick_params(axis='x', labelrotation=90)

In [ ]:
# get the color of the different organs
fig, ax = plt.subplots(figsize=[1.5, 4]); ax.grid(False)
order = ['CNS','Non-CNS']
# purely set for visualization purposes
np.random.seed(3)
# derive the bar and stripplots
sns.barplot(x='is_cns', y='TPM', data=data, order=order, color='xkcd:pale lavender',
            ci=68, errwidth=1.5, capsize=0.3, edgecolor='xkcd:bright lilac', errcolor='xkcd:bright lilac', linewidth=1.5)
sns.stripplot(x='is_cns', y='TPM', data=data, order=order, jitter=0.4, alpha=0.5,
              color='xkcd:pale lavender', edgecolor='xkcd:bright lilac', linewidth=1.5)
ax.tick_params(axis='x', labelrotation=90)
ax.set_xlim(-1, len(order))
ax.legend(bbox_to_anchor=(1.05, .99), bbox_transform=ax.transAxes, frameon=False, loc='upper left')
ax.set_ylim(0, 20)
ss.mannwhitneyu(data['TPM'][data['is_cns'] == 'CNS'], data['TPM'][data['is_cns'] == 'Non-CNS'])[1]

In [ ]:
# derive an order of expression for GPR85
order = df_z_melt.loc[df_z_melt['organ'].str.startswith('cns')].groupby('variable').mean(numeric_only=True)['value'].sort_values()[::-1]
# look through neurotransmitters
neurotransmitters = ['Acetylcholine','Glutamate','GABA','Adenosine','Histamine']
diffs = pd.Series(); ps = pd.Series()
for neurotransmitter in neurotransmitters:
    # gather the data
    df_z_tmp = df_z_plot.loc[df_z_plot['protein_vs_metabolite'] != 'Random']
    # look at muscarinic receptors
    mask = df_z_tmp.index == ''
    for receptor in [neurotransmitter]:
        mask = mask | df_z_tmp['variable'].str.startswith(receptor) | df_z_tmp['variable'].str.endswith(receptor)
    df_z_tmp = df_z_tmp.loc[mask].groupby(['organ_simple','variable']).mean(numeric_only=True)['value'].astype(float).reset_index()
    df_z_tmp = df_z_tmp.loc[df_z_tmp['organ_simple'].str.startswith('cns')].groupby('variable').mean(numeric_only=True)['value'].sort_values()[::-1]
    # examine both distributions
    x1s = order.loc[order.index.str.startswith(neurotransmitter)][::-1].tolist()
    x2s = df_z_tmp.tolist()
    diffs.loc[neurotransmitter] = np.mean(x1s) - np.mean(x2s)
    # plot the data and report the pvalue
    fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
    sns.barplot(x=['GPR85']*len(x1s)+['Known']*len(x2s), y=x1s+x2s,
                palette=['xkcd:bright lilac', 'lightgray'], errwidth=1.5,
                errcolor='k', edgecolor='k', linewidth=1.5, capsize=0.3)
    ax.set_xlim(-1, 2); ax.tick_params(axis='x', labelrotation=90)
    ax.set_ylim(np.array(ax.get_ylim())*1.3)
    p = ss.mannwhitneyu(x1s, x2s)[1]; ps.loc[neurotransmitter] = p
    ax.text(0.5, ax.get_ylim()[1]/1.3*1.125, 'p=%.2e' % p, va='center', ha='center')
    ax.set_ylabel(f'Average interaction Z-score\n(normalized) against\n{neurotransmitter.lower()} in the CNS')

In [ ]:
# color on the data
fig, ax = plt.subplots(figsize=[3, 4]); ax.grid(False)
diffs = diffs.sort_values()
ax.bar(diffs.index, 1 / -diffs, edgecolor='xkcd:bright lilac', color='xkcd:pale lavender', lw=1.5)
ax.tick_params(axis='x', labelrotation=90)
ax.set_ylabel('Interaction strength similarity\nto known (1 / (Known - Obs))')

In [ ]:
from adjustText import adjust_text
# color on the data
fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
diffs = diffs.sort_values()
xs, ys = -np.log10(ps.loc[diffs.index]), 1 / -diffs
avoid = ax.scatter(xs, ys, edgecolor='xkcd:bright lilac', color='xkcd:pale lavender', lw=1.5)
ax.set_ylabel('Interaction strength similarity\nto known (1 / (Known - Obs))')
ax.set_xlabel('Interaction strength difference\nto known (-log10(p-value))')
# annotate the texts of interest
TEXTS = []
for text in xs.index:
    TEXTS.append(ax.text(xs.loc[text], ys.loc[text], text))
adjust_text(TEXTS, arrowprops=dict(arrowstyle='-', color='k', lw=1), ax=ax, objects=avoid, expand_axes=True, expand=(1.7, 1.7))

In [ ]:
# assemble dataframe
df_stat = pd.DataFrame(columns=['gpr85','known','diff','pval'])
for neurotransmitter in neurotransmitters:
    # gather the data
    df_z_tmp = df_z_plot.loc[df_z_plot['protein_vs_metabolite'] != 'Random']
    # look at muscarinic receptors
    mask = df_z_tmp.index == ''
    for receptor in [neurotransmitter]:
        mask = mask | df_z_tmp['variable'].str.startswith(receptor) | df_z_tmp['variable'].str.endswith(receptor)
    df_z_tmp = df_z_tmp.loc[mask].groupby(['organ_simple','variable']).mean(numeric_only=True)['value'].astype(float).reset_index()
    df_z_tmp = df_z_tmp.loc[df_z_tmp['organ_simple'].str.startswith('cns')].groupby('variable').mean(numeric_only=True)['value'].sort_values()[::-1]
    # examine both distributions
    x1s = order.loc[order.index.str.startswith(neurotransmitter)][::-1].tolist()
    x2s = df_z_tmp.tolist()
    gpr85, known = np.mean(x1s), np.mean(x2s)
    p = ss.mannwhitneyu(x1s, x2s)[1]
    df_stat.loc[neurotransmitter] = gpr85, known, gpr85 - known, p
df_stat.to_csv('~/TMP/SI_Table.ST_RINTENSITY.csv')

### pathways for interaction scores

In [ ]:
from matplotlib.colors import to_hex
from matplotlib.cm import get_cmap
# read in the enrichments (for non-enrichments)
pathways = '''Inflammatory Response (GO:0006954)
Granulocyte Chemotaxis (GO:0071621)
Lymphocyte Migration (GO:0072676)
Chemokine-Mediated Signaling Pathway (GO:0070098)
Cellular Response To Chemokine (GO:1990869)
Positive Regulation Of ERK1 And ERK2 Cascade (GO:0070374)
Positive Regulation Of Calcium Ion Transport (GO:0051928)
Cellular Response To Type II Interferon (GO:0071346)
Macrophage Chemotaxis (GO:0048246)
Response To Tumor Necrosis Factor (GO:0034612)'''.split('\n')
fdrs = [float(x) for x in '''1.51E-18
2.61E-10
2.61E-10
1.28E-09
1.45E-09
6.43E-08
8.78E-08
1.03E-07
1.28E-07
1.31E-06'''.split('\n')]

# retrieve the colors
ys = -np.log10(fdrs)
colors = [y/15 for y in ys]
cmap = get_cmap('Reds')
colors = [to_hex(cmap(c)) for c in colors]
# plot the colored bars
fig, ax = plt.subplots(figsize=[3, 2]); ax.grid(False)
ax.bar(pathways, ys, edgecolor='k', linewidth=1.5, color=colors)
ax.tick_params(axis='x', labelrotation=90)

In [ ]:
# read in the enrichments (for non-enrichments)
pathways = '''Inflammatory Response (GO:0006954)
Granulocyte Chemotaxis (GO:0071621)
Lymphocyte Chemotaxis (GO:0048247)
Chemokine-Mediated Signaling Pathway (GO:0070098)
Response To Interleukin-1 (GO:0070555)
Cellular Response To Prostaglandin Stimulus (GO:0071379)
Macrophage Chemotaxis (GO:0048246)
Regulation Of ERK1 And ERK2 Cascade (GO:0070372)
Response To Tumor Necrosis Factor (GO:0034612)
Positive Regulation Of MAPK Cascade (GO:0043410)'''.split('\n')
fdrs = [float(x) for x in '''1.51E-18
2.61E-10
3.32E-10
1.28E-09
1.15E-08
6.20E-08
1.28E-07
4.87E-07
1.31E-06
2.39E-06'''.split('\n')]

# retrieve the colors
ys = -np.log10(fdrs)
colors = [y/15 for y in ys]
cmap = get_cmap('Reds')
colors = [to_hex(cmap(c)) for c in colors]
# plot the colored bars
fig, ax = plt.subplots(figsize=[3, 2]); ax.grid(False)
ax.bar(pathways, ys, edgecolor='k', linewidth=1.5, color=colors)
ax.tick_params(axis='x', labelrotation=90)

### save scores for SI tables

In [ ]:
# compile the results for z-score of known LRs
df_zs = []
for result, tag in zip(results, df_info['tag']):
    if result is None: continue
    # grab the file id
    file_id = list(result.keys())[0]
    # grab the result object
    result = result[file_id]
    # retrieve the proper dataframe
    df_z = result['known']['z']
    df_z.name = tag
    df_zs.append(df_z)
# compute the expression profiles
df_z = pd.concat(df_zs, axis=1).T.fillna(0)
df_z['organ'] = df_z.index.map(anno[['tag','organ']].value_counts().reset_index().set_index('tag')['organ'])
df_z['organ'] += ':' + df_z.index.map(anno[['tag','age_cat']].value_counts().reset_index().set_index('tag')['age_cat'])
df_z['organ'] = df_z['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
# save the results
df_z.T.to_csv('/path/to/SI_Table_Known.csv')

In [ ]:
# compile the results for z-score of rand LRs
df_zs = []
for result, tag in zip(results, df_info['tag']):
    if result is None: continue
    # grab the file id
    file_id = list(result.keys())[0]
    # grab the result object
    result = result[file_id]
    # retrieve the proper dataframe
    df_z = result['rand']['z']
    df_z.name = tag
    df_zs.append(df_z)
# compute the expression profiles
df_z = pd.concat(df_zs, axis=1).T.fillna(0)
df_z['organ'] = df_z.index.map(anno[['tag','organ']].value_counts().reset_index().set_index('tag')['organ'])
df_z['organ'] += ':' + df_z.index.map(anno[['tag','age_cat']].value_counts().reset_index().set_index('tag')['age_cat'])
df_z['organ'] = df_z['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
# save the results
df_z.T.to_csv('/path/to/SI_Table_Rand.csv')

In [ ]:
# compile the results for z-score of orphan LRs
df_zs = []
for result, tag in zip(results, df_info['tag']):
    if result is None: continue
    # grab the file id
    file_id = list(result.keys())[0]
    # grab the result object
    result = result[file_id]
    # retrieve the proper dataframe
    df_z = result['orphan']['z']
    df_z.name = tag
    df_zs.append(df_z)
# compute the expression profiles
df_z = pd.concat(df_zs, axis=1).T.fillna(0)
df_z['organ'] = df_z.index.map(anno[['tag','organ']].value_counts().reset_index().set_index('tag')['organ'])
df_z['organ'] += ':' + df_z.index.map(anno[['tag','age_cat']].value_counts().reset_index().set_index('tag')['age_cat'])
df_z['organ'] = df_z['organ'].str.replace('Adult (≥60)', 'adult').str.replace('Adult (<60)', 'adult').str.replace('Unknown (likely adult)','adult').str.replace('Fetal','fetal').str.replace(':', ' – ')
# save the results
df_z.T.to_csv('/path/to/SI_Table_Orphan.csv')

### receptor and ACh contact and conservation analysis

In [ ]:
# read in the conservation information
cf = pd.read_table('</path/to/conservation_data>/output.txt', header=0, index_col=0)
cf.index = cf.index.astype(str)

In [ ]:
# read in the contacts, #1 is GPR85
df = pd.read_table('</path/to/contact_data>/contacts_letFOUR.tsv', skiprows=1, header=None)
df[['start_model','rest']] = df[0].str.split('/', expand=True)
df[['start_chain','rest']] = df['rest'].str.split(':', expand=True)
df[['start_position','start_atom']] = df['rest'].str.split('@', expand=True)
df[['end_model','rest']] = df[1].str.split('/', expand=True)
df[['end_chain','rest']] = df['rest'].str.split(':', expand=True)
df[['end_position','end_atom']] = df['rest'].str.split('@', expand=True)
del df['rest']
df.shape

In [ ]:
# restrict to models #1 and #2 (7t94)
df = df.loc[(df[['start_model','end_model']].isin(['#1','#2'])).all(axis=1)]
df = df.loc[(df[['start_position','end_position']] == '501').sum(1) == 1]
df[['start_position','end_position']] = df[['start_position','end_position']].astype(int)
df = df.loc[(df[['start_position','end_position']] <= 501).all(axis=1)]
df.shape

In [ ]:
# retrieve GPCRdb positions
df['GPCRdb position'] = ''; df['Ligand atom'] = ''
# create a custom mapping for GPR85
mapping = {'102':'3.33x33','105':'3.36x36','106':'3.37x37', '190':'U45.68',
           '298':'6.48x48','301':'6.51x51', '302':'6.52x52','325':'7.39x38',
           '328':'7.42x41','329':'7.43x42'}
backgak = {v:k for k,v in mapping.items()}
mask = df['end_model'] == '#1'
df.loc[mask, 'GPCRdb position'] = df.loc[mask, 'end_position'].astype(str).map(mapping)
mapping = {'C6':'C3','C5':'C2','C3':'C1','C2':'C7', 'O7':'O2','O4':'O1','N1':'N1', 'C8':'C4','C9':'C5','C10':'C6'}
assert len(list(mapping.values())) == len(set(list(mapping.values())))
df.loc[mask, 'Ligand atom'] = df.loc[mask, 'start_atom'].astype(str).map(mapping)
# create a custom mapping for the crystal
mapping = {'72':'2.53x53','103':'3.32x32','104':'3.33x33','107':'3.36x36',
           '108':'3.37x37','111':'3.40x40','155':'4.57x57','194':'5.46x461',
           '400':'6.48x48','403':'6.51x51','426':'7.39x38','429':'7.42x41',
           '430':'7.43x42'}
mask = df['end_position'] == 501
df.loc[mask, 'GPCRdb position'] = df.loc[mask, 'start_position'].astype(str).map(mapping)
assert df.loc[df['end_model'] == '#2', 'end_position'].unique().tolist() == [501]
assert df.index[mask].intersection(df.index[df['end_model'] == '#1']).shape[0] == 0
assert df.index[mask].union(df.index[df['end_model'] == '#1']).shape[0] == df.shape[0]
mapping = {'C6':'C3','C5':'C2','C3':'C1','C2':'C7', 'O7':'O2','O4':'O1','N1':'N1', 'C8':'C4','C9':'C5','C10':'C6'}
df.loc[mask, 'Ligand atom'] = df.loc[mask, 'end_atom'].astype(str).map(mapping)
df['count'] = 1

In [ ]:
# create the two structures
mask = (df[['start_model','end_model']] == '#2').all(axis=1)
df_crystal = df.loc[mask].pivot_table(index='GPCRdb position', columns='Ligand atom', values='count', aggfunc=np.sum).fillna(0)
mask = (df[['start_model','end_model']] == '#1').any(axis=1)
df_gpr85 = df.loc[mask].pivot_table(index='GPCRdb position', columns='Ligand atom', values='count', aggfunc=np.sum).fillna(0)

In [ ]:
from scipy.cluster.hierarchy import linkage
# perform plotting
idxs = df_crystal.index.union(df_gpr85.index); idys = df_crystal.columns.union(df_gpr85.columns)
Z = linkage(pd.concat([df_gpr85.reindex(idxs).T.reindex(idys).T.fillna(0),
                       df_crystal.reindex(idxs).T.reindex(idys).T.fillna(0)], axis=0).T.fillna(0), method='ward')
# > in silico gpr85
g = sns.clustermap(df_gpr85.reindex(idxs).fillna(0).T.reindex(idys).T.fillna(0),
                   cmap='Blues', method='ward', row_cluster=False, col_linkage=Z, dendrogram_ratio=.1,
                   xticklabels=1, yticklabels=1, figsize=[4, 3.4], cbar_pos=(0, 1, .01, .08), vmax=3)
g.ax_heatmap.grid(False); g.ax_heatmap.tick_params(axis='x', labelrotation=90)
g = sns.clustermap(df_crystal.reindex(idxs).fillna(0).T.reindex(idys).T.fillna(0),
                   cmap='Blues', method='ward', row_cluster=False, col_linkage=Z, dendrogram_ratio=.1,
                   xticklabels=1, yticklabels=1, figsize=[4, 3.4], cbar_pos=(0, 1, .01, .08), vmax=3)
g.ax_heatmap.grid(False); g.ax_heatmap.tick_params(axis='x', labelrotation=90)

In [ ]:
# retrieve the conservation for these sites
xs = df_gpr85.sum(1)
xs.index = xs.index.map(backgak)
xs = cf.loc[xs.index]
# retrieve the expected conservation for this residue
avg_score_per_residue = cf.groupby('SEQ').mean(numeric_only=True)['SCORE']
diffs = xs['SCORE'].values - avg_score_per_residue.loc[xs['SEQ']].values
# calculate the p-value
p = ss.wilcoxon(xs['SCORE'].values, avg_score_per_residue.loc[xs['SEQ']].values)[1]
print(p)
# plot the boxplot
fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
ys = xs['SCORE'].values.tolist() + avg_score_per_residue.loc[xs['SEQ']].values.tolist()
sns.boxplot(x=['Obs']*xs.shape[0]+['Exp']*xs.shape[0], y=ys, saturation=1, showfliers=False, zorder=0,
            linewidth=1.5, linecolor='k', order=['Obs','Exp'], palette=['xkcd:bright lilac', 'lightgray'])
ax.scatter(x=['Exp']*xs.shape[0], y=ys[xs.shape[0]:], zorder=2, edgecolor='k', lw=1.5, facecolors='grey', alpha=0.5)
ax.scatter(x=['Obs']*xs.shape[0], y=ys[:xs.shape[0]], zorder=2, edgecolor='k', lw=1.5, facecolors='xkcd:bright lilac', alpha=0.5)
for idx in range(xs.shape[0]):
    idx1, idx2 = idx, idx + xs.shape[0]
    ax.plot(['Obs', 'Exp'], [ys[idx1], ys[idx2]], lw=1, color='k', zorder=1, alpha=0.5)
ax.plot([-1, 2], [0]*2, color='k', linestyle='--', lw=1.5, zorder=2)
ax.set_xlim(-1, 2)
ax.set(xlabel='Contacts', ylabel='Conservation score')

In [ ]:
# color the average profile with lines at the contact sites
fig, ax = plt.subplots(figsize=[12, 4]); ax.grid(False)
xs = cf.index.astype(int)
ax.scatter(xs, cf['SCORE'], s=5, color='xkcd:pale lavender')
ax.plot(xs, cf['AVG_W5'], color='xkcd:bright lilac', lw=1.5)
xlim, ylim = ax.get_xlim(), ax.get_ylim()
ax.plot(xlim, [0]*2, color='k', lw=1.5, linestyle='-')
positions = df_gpr85.index.map(backgak).astype(int)
for pos in positions:
    ax.plot([pos]*2, ylim, color='r', lw=1.5, linestyle='--')
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set(xlabel='Position in GPR85', ylabel='Conservation score')

In [ ]:
# list out positions so we may search them up for annotation
positions

In [ ]:
# given M total objects, n is the number from truth, drawing a sample of N from the population then k are matching
# thus M is the total possible interactions observed in both structures
# given a set of n true interactions from CHRM2:ACh crystal
# if we are to draw N interactions observed in GPR85:ACh in silico
# how many of N, k, are matching those from the crystal
tags1, tags2 = [], []
for idx, idy in zip(*np.where(df_crystal > 0)):
    tag = ']-['.join([df_crystal.index[idx], df_crystal.columns[idy]])
    tags1.append(tag)
for idx, idy in zip(*np.where(df_gpr85 > 0)):
    tag = ']-['.join([df_gpr85.index[idx], df_gpr85.columns[idy]])
    tags2.append(tag)
k = len(set(tags1) & set(tags2))
M = df_crystal.index.union(df_gpr85.index).shape[0] * df_crystal.columns.union(df_gpr85.columns).shape[0]
n = len(tags2)
N = len(tags1)
p = 1 - ss.hypergeom.cdf(k, M, n, N)
print('p=%.4e considering total population as true interactions in both' % p)

In [ ]:
# library
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

# call it out via a venn
x1 = len([x for x in tags1 if x not in tags2])
x2 = len([x for x in tags1 if x in tags2])
x3 = len([x for x in tags2 if x in tags1])
venn2(subsets = (x1, x3, x2), set_labels = ('GPR85:ACh', 'CHRM2:ACh'))

In [ ]:
# read in the data
ef = pd.read_table('</path/to/sasa_data>/gpr85_to_7t94_gpcr.sasa.txt', sep=' ', header=None)[[2, 4]]
ef['position'] = ef[2].str.split(':', expand=True).iloc[:, 1]
ef[4] = ef[4].astype(float)
# retrieve the conservation for these sites
xs = df_gpr85.sum(1)
xs.index = xs.index.map(backgak)
xs = cf.loc[xs.index]
# derive the expected
xs['exp'] = np.nan
for pos in xs.index:
    # derive SASA for this residue's pos
    SASA_pos = ef.loc[ef['position'] == pos, 4].iloc[0]
    # convert the SASA distance from others to a weight
    ef['abs_dist'] = (ef[4] - SASA_pos).abs()
    cf['abs_dist'] = ef.set_index('position')['abs_dist']
    subset = cf.loc[cf['SEQ'] == cf.loc[pos, 'SEQ']].dropna()[['abs_dist','SCORE']]
    subset['abs_dist'] -= subset['abs_dist'].min(); subset['abs_dist'] /= subset['abs_dist'].sum()
    xs.loc[pos, 'exp'] = (subset['SCORE'] * (1 - subset['abs_dist'])).mean()
# retrieve the expected conservation for this residue
diffs = xs['SCORE'].values - xs['exp'].values
# calculate the p-value
p = ss.wilcoxon(xs['SCORE'].values, xs['exp'].values)[1]
print(p)
# plot the boxplot
fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
ys = xs['SCORE'].values.tolist() + xs['exp'].values.tolist()
sns.boxplot(x=['Obs']*xs.shape[0]+['Exp']*xs.shape[0], y=ys, saturation=1, showfliers=False, zorder=0,
            linewidth=1.5, linecolor='k', order=['Obs','Exp'], palette=['xkcd:bright lilac', 'lightgray'])
ax.scatter(x=['Exp']*xs.shape[0], y=ys[xs.shape[0]:], zorder=2, edgecolor='k', lw=1.5, facecolors='grey', alpha=0.5)
ax.scatter(x=['Obs']*xs.shape[0], y=ys[:xs.shape[0]], zorder=2, edgecolor='k', lw=1.5, facecolors='xkcd:bright lilac', alpha=0.5)
for idx in range(xs.shape[0]):
    idx1, idx2 = idx, idx + xs.shape[0]
    ax.plot(['Obs', 'Exp'], [ys[idx1], ys[idx2]], lw=1, color='k', zorder=1, alpha=0.5)
ax.plot([-1, 2], [0]*2, color='k', linestyle='--', lw=1.5, zorder=2)
ax.set_xlim(-1, 2)
ax.set(xlabel='Contacts', ylabel='Conservation score')

In [ ]:
# read in the contacts, #1 is GPR85
df = pd.read_table('</path/to/contact_data>/contacts_letFIVE.tsv', skiprows=1, header=None)
df[['start_model','rest']] = df[0].str.split('/', expand=True)
df[['start_chain','rest']] = df['rest'].str.split(':', expand=True)
df[['start_position','start_atom']] = df['rest'].str.split('@', expand=True)
df[['end_model','rest']] = df[1].str.split('/', expand=True)
df[['end_chain','rest']] = df['rest'].str.split(':', expand=True)
df[['end_position','end_atom']] = df['rest'].str.split('@', expand=True)
del df['rest']
df.shape

In [ ]:
# restrict to models #1 and #2 (7t94)
df = df.loc[(df[['start_model','end_model']].isin(['#1','#2'])).all(axis=1)]
df = df.loc[(df[['start_position','end_position']] == '501').sum(1) == 1]
df[['start_position','end_position']] = df[['start_position','end_position']].astype(int)
df = df.loc[(df[['start_position','end_position']] <= 501).all(axis=1)]
df.shape

In [ ]:
# retrieve GPCRdb positions
df['GPCRdb position'] = ''; df['Ligand atom'] = ''
# create a custom mapping for GPR85
mapping = {'102':'3.33x33','105':'3.36x36','106':'3.37x37', '190':'U45.68',
           '298':'6.48x48','301':'6.51x51', '302':'6.52x52','325':'7.39x38',
           '328':'7.42x41','329':'7.43x42'}
backgak = {v:k for k,v in mapping.items()}
mask = df['end_model'] == '#1'
df.loc[mask, 'GPCRdb position'] = df.loc[mask, 'end_position'].astype(str).map(mapping)
mapping = {'C6':'C3','C5':'C2','C3':'C1','C2':'C7', 'O7':'O2','O4':'O1','N1':'N1', 'C8':'C4','C9':'C5','C10':'C6'}
assert len(list(mapping.values())) == len(set(list(mapping.values())))
df.loc[mask, 'Ligand atom'] = df.loc[mask, 'start_atom'].astype(str).map(mapping)
# create a custom mapping for the crystal
mapping = {'72':'2.53x53','103':'3.32x32','104':'3.33x33','107':'3.36x36',
           '108':'3.37x37','111':'3.40x40','155':'4.57x57','194':'5.46x461',
           '400':'6.48x48','403':'6.51x51','426':'7.39x38','429':'7.42x41',
           '430':'7.43x42'}
mask = df['end_position'] == 501
df.loc[mask, 'GPCRdb position'] = df.loc[mask, 'start_position'].astype(str).map(mapping)
assert df.loc[df['end_model'] == '#2', 'end_position'].unique().tolist() == [501]
assert df.index[mask].intersection(df.index[df['end_model'] == '#1']).shape[0] == 0
assert df.index[mask].union(df.index[df['end_model'] == '#1']).shape[0] == df.shape[0]
mapping = {'C6':'C3','C5':'C2','C3':'C1','C2':'C7', 'O7':'O2','O4':'O1','N1':'N1', 'C8':'C4','C9':'C5','C10':'C6'}
df.loc[mask, 'Ligand atom'] = df.loc[mask, 'end_atom'].astype(str).map(mapping)
df['count'] = 1

In [ ]:
# create the two structures
mask = (df[['start_model','end_model']] == '#2').all(axis=1)
df_crystal = df.loc[mask].pivot_table(index='GPCRdb position', columns='Ligand atom', values='count', aggfunc=np.sum).fillna(0)
mask = (df[['start_model','end_model']] == '#1').any(axis=1)
df_gpr85 = df.loc[mask].pivot_table(index='GPCRdb position', columns='Ligand atom', values='count', aggfunc=np.sum).fillna(0)

In [ ]:
from scipy.cluster.hierarchy import linkage
# perform plotting
idxs = df_crystal.index.union(df_gpr85.index); idys = df_crystal.columns.union(df_gpr85.columns)
idxs = ['2.53x53', '3.32x32', '3.33x33', '3.36x36', '3.37x37', '3.40x40',
       '4.57x57', 'U45.68', '5.46x461', '6.48x48', '6.51x51', '6.52x52', '7.39x38',
       '7.42x41', '7.43x42']
Z = linkage(pd.concat([df_gpr85.reindex(idxs).T.reindex(idys).T.fillna(0),
                       df_crystal.reindex(idxs).T.reindex(idys).T.fillna(0)], axis=0).T.fillna(0), method='ward')
# > in silico gpr85
g = sns.clustermap(df_gpr85.reindex(idxs).fillna(0).T.reindex(idys).T.fillna(0),
                   cmap='Blues', method='ward', row_cluster=False, col_linkage=Z, dendrogram_ratio=.1,
                   xticklabels=1, yticklabels=1, figsize=[4, 4.5], cbar_pos=(0, 1, .01, .08), vmax=4)
g.ax_heatmap.grid(False); g.ax_heatmap.tick_params(axis='x', labelrotation=90)
g = sns.clustermap(df_crystal.reindex(idxs).fillna(0).T.reindex(idys).T.fillna(0),
                   cmap='Blues', method='ward', row_cluster=False, col_linkage=Z, dendrogram_ratio=.1,
                   xticklabels=1, yticklabels=1, figsize=[4, 4.5], cbar_pos=(0, 1, .01, .08), vmax=4)
g.ax_heatmap.grid(False); g.ax_heatmap.tick_params(axis='x', labelrotation=90)

In [ ]:
# > in silico gpr85
g = sns.clustermap(df_gpr85.reindex(idxs).fillna(0).T.reindex(idys).T.fillna(0),
                   cmap='Blues', method='ward', row_cluster=False, col_linkage=Z, dendrogram_ratio=.1,
                   xticklabels=1, yticklabels=1, figsize=[4, 4.5], cbar_pos=(0, 1, .01, .08), vmax=4)
g.ax_heatmap.grid(False); g.ax_heatmap.tick_params(axis='x', labelrotation=270)
g.ax_heatmap.set_ylabel('GPCRdb position', rotation=270, labelpad=16)
g = sns.clustermap(df_crystal.reindex(idxs).fillna(0).T.reindex(idys).T.fillna(0),
                   cmap='Blues', method='ward', row_cluster=False, col_linkage=Z, dendrogram_ratio=.1,
                   xticklabels=1, yticklabels=1, figsize=[4, 4.5], cbar_pos=(0, 1, .01, .08), vmax=4)
g.ax_heatmap.grid(False); g.ax_heatmap.tick_params(axis='x', labelrotation=270)
g.ax_heatmap.set_ylabel('GPCRdb position', rotation=270, labelpad=16)

In [ ]:
# retrieve the conservation for these sites
xs = df_gpr85.sum(1)
xs.index = xs.index.map(backgak)
xs = cf.loc[xs.index]
# retrieve the expected conservation for this residue
avg_score_per_residue = cf.groupby('SEQ').mean(numeric_only=True)['SCORE']
diffs = xs['SCORE'].values - avg_score_per_residue.loc[xs['SEQ']].values
# calculate the p-value
p = ss.wilcoxon(xs['SCORE'].values, avg_score_per_residue.loc[xs['SEQ']].values)[1]
print(p)
# plot the boxplot
fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
ys = xs['SCORE'].values.tolist() + avg_score_per_residue.loc[xs['SEQ']].values.tolist()
sns.boxplot(x=['Obs']*xs.shape[0]+['Exp']*xs.shape[0], y=ys, saturation=1, showfliers=False, zorder=0,
            linewidth=1.5, linecolor='k', order=['Obs','Exp'], palette=['xkcd:bright lilac', 'lightgray'])
ax.scatter(x=['Exp']*xs.shape[0], y=ys[xs.shape[0]:], zorder=2, edgecolor='k', lw=1.5, facecolors='grey', alpha=0.5)
ax.scatter(x=['Obs']*xs.shape[0], y=ys[:xs.shape[0]], zorder=2, edgecolor='k', lw=1.5, facecolors='xkcd:bright lilac', alpha=0.5)
for idx in range(xs.shape[0]):
    idx1, idx2 = idx, idx + xs.shape[0]
    ax.plot(['Obs', 'Exp'], [ys[idx1], ys[idx2]], lw=1, color='k', zorder=1, alpha=0.5)
ax.plot([-1, 2], [0]*2, color='k', linestyle='--', lw=1.5, zorder=2)
ax.set_xlim(-1, 2)
ax.set(xlabel='Contacts', ylabel='Conservation score')
print(ss.pearsonr(df_gpr85.sum(1), xs['SCORE']), ss.spearmanr(df_gpr85.sum(1), xs['SCORE']))
xs_v2 = df_gpr85.sum(1); xs_v2.index = xs_v2.index.map(backgak)
print(ss.pearsonr(xs_v2.reindex(cf.index).fillna(0), cf['SCORE']), \
      ss.spearmanr(xs_v2.reindex(cf.index).fillna(0), cf['SCORE']))

In [ ]:
# color the average profile with lines at the contact sites
fig, ax = plt.subplots(figsize=[12, 4]); ax.grid(False)
xs = cf.index.astype(int)
ax.scatter(xs, cf['SCORE'], s=5, color='xkcd:pale lavender')
ax.plot(xs, cf['AVG_W5'], color='xkcd:bright lilac', lw=1.5)
xlim, ylim = ax.get_xlim(), ax.get_ylim()
ax.plot(xlim, [0]*2, color='k', lw=1.5, linestyle='-')
positions = df_gpr85.index.map(backgak).astype(int)
for pos in positions:
    ax.plot([pos]*2, ylim, color='r', lw=1.5, linestyle='--')
ax.set_xlim(*xlim); ax.set_ylim(*ylim)
ax.set(xlabel='Position in GPR85', ylabel='Conservation score')

In [ ]:
# given M total objects, n is the number from truth, drawing a sample of N from the population then k are matching
# thus M is the total possible interactions observed in both structures
# given a set of n true interactions from CHRM2:ACh crystal
# if we are to draw N interactions observed in GPR85:ACh in silico
# how many of N, k, are matching those from the crystal
tags1, tags2 = [], []
for idx, idy in zip(*np.where(df_crystal > 0)):
    tag = ']-['.join([df_crystal.index[idx], df_crystal.columns[idy]])
    tags1.append(tag)
for idx, idy in zip(*np.where(df_gpr85 > 0)):
    tag = ']-['.join([df_gpr85.index[idx], df_gpr85.columns[idy]])
    tags2.append(tag)
k = len(set(tags1) & set(tags2))
M = df_crystal.index.union(df_gpr85.index).shape[0] * df_crystal.columns.union(df_gpr85.columns).shape[0]
n = len(tags2)
N = len(tags1)
p = 1 - ss.hypergeom.cdf(k, M, n, N)
print('p=%.4e considering total population as true interactions in both' % p)

In [ ]:
# library
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

# call it out via a venn
x1 = len([x for x in tags1 if x not in tags2])
x2 = len([x for x in tags1 if x in tags2])
x3 = len([x for x in tags2 if x in tags1])
venn2(subsets = (x1, x3, x2), set_labels = ('GPR85:ACh', 'CHRM2:ACh'))

In [ ]:
# read in the data
ef = pd.read_table('</path/to/sasa_data>/gpr85_to_7t94_gpcr.sasa.txt', sep=' ', header=None)[[2, 4]]
ef['position'] = ef[2].str.split(':', expand=True).iloc[:, 1]
ef[4] = ef[4].astype(float)
# retrieve the conservation for these sites
xs = df_gpr85.sum(1)
xs.index = xs.index.map(backgak)
xs = cf.loc[xs.index]
# derive the expected
xs['exp'] = np.nan
for pos in xs.index:
    # derive SASA for this residue's pos
    SASA_pos = ef.loc[ef['position'] == pos, 4].iloc[0]
    # convert the SASA distance from others to a weight
    ef['abs_dist'] = (ef[4] - SASA_pos).abs()
    cf['abs_dist'] = ef.set_index('position')['abs_dist']
    subset = cf.loc[cf['SEQ'] == cf.loc[pos, 'SEQ']].dropna()[['abs_dist','SCORE']]
    subset['abs_dist'] -= subset['abs_dist'].min(); subset['abs_dist'] /= subset['abs_dist'].sum()
    xs.loc[pos, 'exp'] = (subset['SCORE'] * (1 - subset['abs_dist'])).mean()
# retrieve the expected conservation for this residue
diffs = xs['SCORE'].values - xs['exp'].values
# calculate the p-value
p = ss.wilcoxon(xs['SCORE'].values, xs['exp'].values)[1]
print(p)
# plot the boxplot
fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
ys = xs['SCORE'].values.tolist() + xs['exp'].values.tolist()
sns.boxplot(x=['Obs']*xs.shape[0]+['Exp']*xs.shape[0], y=ys, saturation=1, showfliers=False, zorder=0,
            linewidth=1.5, linecolor='k', order=['Obs','Exp'], palette=['xkcd:bright lilac', 'lightgray'])
ax.scatter(x=['Exp']*xs.shape[0], y=ys[xs.shape[0]:], zorder=2, edgecolor='k', lw=1.5, facecolors='grey', alpha=0.5)
ax.scatter(x=['Obs']*xs.shape[0], y=ys[:xs.shape[0]], zorder=2, edgecolor='k', lw=1.5, facecolors='xkcd:bright lilac', alpha=0.5)
for idx in range(xs.shape[0]):
    idx1, idx2 = idx, idx + xs.shape[0]
    ax.plot(['Obs', 'Exp'], [ys[idx1], ys[idx2]], lw=1, color='k', zorder=1, alpha=0.5)
ax.plot([-1, 2], [0]*2, color='k', linestyle='--', lw=1.5, zorder=2)
ax.set_xlim(-1, 2)
ax.set(xlabel='Contacts', ylabel='Conservation score')

### in vitro validation of receptor internalization and cAMP quantification

In [ ]:
from scipy.optimize import curve_fit
# create sigmoidal modeling
def sigmoid(x, L ,x0, k, b):
    y = L / (1 + np.exp(-k*(x-x0))) + b
    return (y)

In [ ]:
# create array to capture the data for cAMP ACh+F
arr = ['2.33E+06	2.42E+06	3.26E+06',
'3.77E+06	2.62E+06	2.30E+06',
'2.37E+06	3.32E+06	3.03E+06',
'2.71E+06	3.51E+06	2.78E+06',
'3.21E+06	3.86E+06	3.24E+06',
'3.44E+06	4.71E+06	4.18E+06',
'4.00E+06	4.13E+06	3.75E+06',
'4.30E+06	4.04E+06	3.97E+06',
'3.73E+06	4.07E+06	4.32E+06']
# convert into array
arr = [np.array(row.split('\t')).astype(float) for row in arr]
# plot the results
arr = pd.DataFrame(arr, columns=['GPR85_1','GPR85_2','GPR85_3'],
                   index=np.arange(-12, -4)[::-1].tolist()+[0]).T

# set up the figure for GPR85
for gene in ['GPR85']:
    fig, ax = plt.subplots(figsize=[5, 4]); ax.grid(False)
    mask = arr.index.str.startswith(gene)
    # plot the average
    ax.scatter(arr.columns[:-1], arr.loc[mask, arr.columns[:-1]].mean(0),
               color='xkcd:pale lavender', edgecolor='xkcd:bright lilac', s=50, zorder=1, lw=1.5)
    # plot the standard error
    ci_pad = 0.1
    for col in arr.columns[:-1]:
        x = col; y = arr.loc[mask, col].mean()
        ci = arr[col].std() / np.sqrt(4)
        ax.plot([x]*2, [y-ci, y+ci], color='xkcd:bright lilac', zorder=0, lw=1.5)
        ax.plot([x-ci_pad, x+ci_pad], [y-ci]*2, color='xkcd:bright lilac', zorder=0, lw=1.5)
        ax.plot([x-ci_pad, x+ci_pad], [y+ci]*2, color='xkcd:bright lilac', zorder=0, lw=1.5)
        
    # define the parameters
    xdata = arr.columns[:-1].tolist()*3
    ydata = arr.loc[mask, arr.columns[:-1]].values.flatten()
    vmin, vmax = ydata.min(), ydata.max() - ydata.min()
    ydata = (ydata - vmin) / vmax
    # model the sigmoidal curve
    p0 = [max(ydata), np.median(xdata), 1, min(ydata)] # this is an mandatory initial guess
    popt, pcov = curve_fit(sigmoid, xdata, ydata, p0, method='dogbox')
    # set up the fit
    xl = np.linspace(-13, -3, 1000)
    yl = sigmoid(xl, *popt)
    ax.plot(xl, yl * vmax + vmin, color='k', linestyle='--', lw=2)
    # utilize the ten closest values to estimate an EC50
    xs_ = xl[np.argsort(abs(yl - 0.5))[:10]]
    ys_ = yl[np.argsort(abs(yl - 0.5))[:10]]
    weights = abs(ys_ - 0.5); weights /= weights.sum()
    estimated_ec50 = sum(xs_ * weights)
    print(f'EC50={round(10**estimated_ec50 * 1e9,2)}nM via dogbox')

    # plot the non ligand
    ax.plot([-13, -3], [arr.loc[mask, 0].mean()]*2, color='xkcd:bright lilac', lw=2, linestyle='--')
    
    # setup the ticks
    ax.set_xticks(arr.columns)
    ax.set_xlim(-12.5, -4.5)
    ax.set(xlabel='log$_{10}$(ACh conc.) in M')


In [ ]:
# create array to capture the data for cAMP Glu+F
arr = ['5.77E+05	340706	349120	293306',
'4.29E+05	4.10E+05	349120	314468',
'7.16E+05	6.30E+05	4.90E+05	358991',
'7.04E+05	4.23E+05	4.46E+05	4.29E+05',
'5.83E+05	4.75E+05	4.11E+05	329038',
'4.16E+05	4.52E+05	358991	380908',
'4.61E+05	5.21E+05	364032	367858',
'4.66E+05	5.21E+05	4.26E+05	373024',
'4.85E+05	4.76E+05	5.02E+05	4.19E+05']
# convert into array
arr = [np.array(row.split('\t')).astype(float) for row in arr]
# plot the results
arr = pd.DataFrame(arr, columns=['GPR85_1','GPR85_2','GPR85_3','GPR85_4'],
                   index=np.arange(-12, -4)[::-1].tolist()+[0]).T

# set up the figure for GPR85
for gene in ['GPR85']:
    fig, ax = plt.subplots(figsize=[5, 4]); ax.grid(False)
    mask = arr.index.str.startswith(gene)
    # plot the average
    ax.scatter(arr.columns[:-1], arr.loc[mask, arr.columns[:-1]].mean(0),
               color='xkcd:pale lavender', edgecolor='xkcd:bright lilac', s=50, zorder=1, lw=1.5)
    # plot the standard error
    ci_pad = 0.1
    for col in arr.columns[:-1]:
        x = col; y = arr.loc[mask, col].mean()
        ci = arr[col].std() / np.sqrt(4)
        ax.plot([x]*2, [y-ci, y+ci], color='xkcd:bright lilac', zorder=0, lw=1.5)
        ax.plot([x-ci_pad, x+ci_pad], [y-ci]*2, color='xkcd:bright lilac', zorder=0, lw=1.5)
        ax.plot([x-ci_pad, x+ci_pad], [y+ci]*2, color='xkcd:bright lilac', zorder=0, lw=1.5)
        
    # plot the non ligand
    ax.plot([-13, -3], [arr.loc[mask, 0].mean()]*2, color='xkcd:bright lilac', lw=2, linestyle='--')
    
    # setup the ticks
    ax.set_xticks(arr.columns)
    ax.set_xlim(-12.5, -4.5)
    ax.set(xlabel='log$_{10}$(Glu conc.) in M')

In [ ]:
# create array to capture the data for RI ACh
arr = ['12237	11925	11622	12080	12929	12396	12158	14347',
'14253	12477	12277	12929	13055	13978	13618	14441',
'11964	11182	14441	11660	13618	13183	12277	12080',
'13355	13140	13442	13978	13183	14023	14823	16957',
'15067	14630	15166	17295	14394	14969	16462	19288',
'15067	13842	13399	18972	15878	16790	16516	19611',
'15067	16462	14969	17875	16846	18537	16790	17353',
'15366	17069	16247	16735	16462	15826	15266	16957',
'16194	16408	14871	17239	17699	15983	15266	19352']
# convert into array
arr = [np.array(row.split('\t')).astype(float) for row in arr]
# plot the results
arr = pd.DataFrame(arr, columns=['GPR85_1','GPR85_2','GPR85_3','GPR85_4',
                                 'CHRM2_1','CHRM2_2','CHRM2_3','CHRM2_4'],
                   index=np.arange(-12, -4)[::-1].tolist()+[0]).T

# set up the figure for GPR85 and CHRM2
for gene in ['GPR85','CHRM2']:
    fig, ax = plt.subplots(figsize=[5, 4]); ax.grid(False)
    mask = arr.index.str.startswith(gene)
    # plot the average
    ax.scatter(arr.columns[:-1], arr.loc[mask, arr.columns[:-1]].mean(0),
               color='xkcd:pale lavender', edgecolor='xkcd:bright lilac', s=50, zorder=1, lw=1.5)
    # plot the standard error
    ci_pad = 0.1
    for col in arr.columns[:-1]:
        x = col; y = arr.loc[mask, col].mean()
        ci = arr[col].std() / np.sqrt(4)
        ax.plot([x]*2, [y-ci, y+ci], color='xkcd:bright lilac', zorder=0, lw=1.5)
        ax.plot([x-ci_pad, x+ci_pad], [y-ci]*2, color='xkcd:bright lilac', zorder=0, lw=1.5)
        ax.plot([x-ci_pad, x+ci_pad], [y+ci]*2, color='xkcd:bright lilac', zorder=0, lw=1.5)
        
    # define the parameters
    xdata = arr.columns[:-1].tolist()*4
    ydata = arr.loc[mask, arr.columns[:-1]].values.flatten()
    vmin, vmax = ydata.min(), ydata.max() - ydata.min()
    ydata = (ydata - vmin) / vmax
    # model the sigmoidal curve
    p0 = [max(ydata), np.median(xdata), 1, min(ydata)] # this is an mandatory initial guess
    popt, pcov = curve_fit(sigmoid, xdata, ydata, p0, method='dogbox')
    # set up the fit
    xl = np.linspace(-13, -3, 1000)
    yl = sigmoid(xl, *popt)
    ax.plot(xl, yl * vmax + vmin, color='k', linestyle='--', lw=2)
    # utilize the ten closest values to estimate an EC50
    xs_ = xl[np.argsort(abs(yl - 0.5))[:10]]
    ys_ = yl[np.argsort(abs(yl - 0.5))[:10]]
    weights = abs(ys_ - 0.5); weights /= weights.sum()
    estimated_ec50 = sum(xs_ * weights)
    print(f'EC50={round(10**estimated_ec50 * 1e9,2)}nM via dogbox')

    # plot the non ligand
    ax.plot([-13, -3], [arr.loc[mask, 0].mean()]*2, color='xkcd:bright lilac', lw=2, linestyle='--')
    
    # setup the ticks
    ax.set_xticks(arr.columns)
    ax.set_xlim(-12.5, -4.5)
    ax.set(xlabel='log$_{10}$(ACh conc.) in M')


In [ ]:
# create array to capture the data for RI Glu
arr = ['11811	13013	12971	13226	14969	14871	13752	16194',
'11005	12763	12477	14394	15316	14969	12158	15826',
'12003	12639	12356	14253	12680	16035	14583	14347',
'12396	15117	12003	12396	12804	13618	13312	16247',
'12277	12558	11811	11697	12558	14488	13932	15774',
'12119	16301	12598	12598	15166	13355	12080	13797',
'11925	13752	15878	12763	13486	13055	15723	13978',
'12639	12558	12680	12436	13269	13486	13098	17641',
'13797	13887	12804	12929	13752	13355	14023	17069']
# convert into array
arr = [np.array(row.split('\t')).astype(float) for row in arr]
# plot the results
arr = pd.DataFrame(arr, columns=['GPR85_1','GPR85_2','GPR85_3','GPR85_4',
                                 'CHRM2_1','CHRM2_2','CHRM2_3','CHRM2_4'],
                   index=np.arange(-12, -4)[::-1].tolist()+[0]).T

# set up the figure for GPR85 and CHRM2
for gene in ['GPR85','CHRM2']:
    fig, ax = plt.subplots(figsize=[5, 4]); ax.grid(False)
    mask = arr.index.str.startswith(gene)
    # plot the average
    ax.scatter(arr.columns[:-1], arr.loc[mask, arr.columns[:-1]].mean(0),
               color='xkcd:pale lavender', edgecolor='xkcd:bright lilac', s=50, zorder=1, lw=1.5)
    # plot the standard error
    ci_pad = 0.1
    for col in arr.columns[:-1]:
        x = col; y = arr.loc[mask, col].mean()
        ci = arr[col].std() / np.sqrt(4)
        ax.plot([x]*2, [y-ci, y+ci], color='xkcd:bright lilac', zorder=0, lw=1.5)
        ax.plot([x-ci_pad, x+ci_pad], [y-ci]*2, color='xkcd:bright lilac', zorder=0, lw=1.5)
        ax.plot([x-ci_pad, x+ci_pad], [y+ci]*2, color='xkcd:bright lilac', zorder=0, lw=1.5)
        
    # plot the non ligand
    ax.plot([-13, -3], [arr.loc[mask, 0].mean()]*2, color='xkcd:bright lilac', lw=2, linestyle='--')
    
    # setup the ticks
    ax.set_xticks(arr.columns)
    ax.set_xlim(-12.5, -4.5)
    ax.set(xlabel='log$_{10}$(Glu conc.) in M')

### verify GPR85 surface expression upon transduction

In [ ]:
# transduced, from ACh RI
transduced = [float(x) for x in '16194	16408	14871	17239'.split('\t')]
untransduced = [float(x) for x in '4326	3597	3977	4377'.split('\t')]
ys = transduced + untransduced
xs = ['Tx']*4+['UnTx']*4
# construct the plot
fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
sns.boxplot(x=xs, y=ys, order=['UnTx','Tx'], palette=['lightgray','xkcd:bright lilac'],
            linewidth=1.5, linecolor='k')
ax.scatter(x=['UnTx']*4, y=ys[-4:], zorder=2, edgecolor='k', lw=1.5, facecolors='grey', alpha=0.5)
ax.scatter(x=['Tx']*4, y=ys[:4], zorder=2, edgecolor='k', lw=1.5, facecolors='xkcd:bright lilac', alpha=0.5)
for idx in range(4):
    ax.plot(['UnTx','Tx'], [untransduced[idx], transduced[idx]], lw=1, color='k', zorder=1, alpha=0.5)
ylim = ax.get_ylim()
ax.set_ylim(ylim[0], ylim[1]+(ylim[1]-ylim[0])*0.3)
ax.set_xlim(-1, 2)
# calculate the p-value
print('shapiro', ss.shapiro(untransduced)[1], ss.shapiro(transduced)[1])
print('wilcoxon', ss.wilcoxon(transduced, untransduced)[1])
print('mannwhitneyu', ss.mannwhitneyu(transduced, untransduced)[1])
print('ttest_rel', ss.ttest_rel(transduced, untransduced)[1])
print('ttest_ind', ss.ttest_ind(transduced, untransduced)[1])

### download single-cell atlases

In [ ]:
# Dementia https://cellxgene.cziscience.com/collections/c53573b2-eff4-4c5e-9ad0-b24d422dfd9b
# read in the data, subset for GPR85, write and delete to save space
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
print('reading in data...')
adata = sc.read_h5ad(f'{DN}/ext_data/cxg_dementia_snrnaseq/2808a16d-64c5-451b-91a5-c1a2d9f270d5.h5ad')
# reassign the variable names
print('reassigning variable names...')
adata.var['ensembl_id'] = adata.var.index
adata.var.index = adata.var['feature_name'].astype(str)
adata.var_names_make_unique()
# notes: ethnicity is self reported, suspension type indicates snRNA-seq
print('keeping relevant variables...')
assert adata.raw.shape[1] ==  adata.shape[1]
org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
               'assay','tissue','ct_subcluster','cell_type']
obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
               'sequencing_kit','tissue','cell_type_broad','cell_type_fine']
adata.obs = adata.obs[org_columns]
adata.obs.columns = obs_columns
adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
adata.obs['study'] = 'Rexach et al. 2024 (Cell)'
adata.var = adata.var[['ensembl_id']]
# store the raw counts
print('cleaning up raw counts...')
adata.X = adata.raw.X
adata.layers['raw_counts'] = adata.X
# clean the raw loadings
adata.raw.var.index = adata.var.index
adata.raw = adata
import seaborn as sns
# retrieve the frequency of the GPR85 gene
print('finding GPR85+ cells...')
freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
# subset the data by cutoff to only get high confidence GPR85 positive cells
fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
ax.set(xlabel='GPR85+ % of counts')
# perform the cutting
np.random.seed(0)
idxs_pos = adata.obs.index[freq >= 5e-4].copy()
idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
# save the data
adata = adata[idxs_pos.union(idxs_neg)].copy()
adata.write(f'{DN}/ext_data/cxg_dementia_snrnaseq/GPR85.mixed.h5ad')

In [ ]:
# Brain census V1.0 https://cellxgene.cziscience.com/collections/283d65eb-dd53-496d-adb7-7570c7caa443
# read in the data, subset for GPR85, write and delete to save space
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_brain_census'
fns = [f'{DN}/ext_data/{group_name}/c2f66cd5-4ff4-4578-876c-55783a57cf8f.h5ad',
       f'{DN}/ext_data/{group_name}/99f27be8-9fac-451e-9723-9e4c7191589e.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.neuronal.h5ad',
           f'{DN}/ext_data/{group_name}/GPR85.mixed.nonneuronal.h5ad']
for fn, out_fn in zip(fns[1:], out_fns[1:]):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: dissection added to tissue for more information, X is raw
    print('keeping relevant variables...')
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','supercluster_term','cell_type','dissection']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine','dissection']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['tissue'] = adata.obs[['tissue','dissection']].astype(str).agg(':'.join, axis=1)
    del adata.obs['dissection']
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Siletti et al. 2023 (Science)'
    adata.var = adata.var[['ensembl_id']]
    adata.obsm['X_umap'] = adata.obsm['X_UMAP']
    adata.obsm['X_tsne'] = adata.obsm['X_tSNE']
    del adata.obsm['X_UMAP'], adata.obsm['X_tSNE']    
    # store the raw counts
    print('cleaning up raw counts...')
    adata.layers['raw_counts'] = adata.X
    # clean the raw loadings
    adata.raw = adata
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# Neocortex https://cellxgene.cziscience.com/collections/d17249d2-0e6e-4500-abb8-e6c93fa1ac6f
# read in the data, subset for GPR85, write and delete to save space
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_neocortex_organization'
fns = [f'{DN}/ext_data/{group_name}/b33ff5a3-527e-41c3-83ce-012a8ad2d2c1.h5ad',
       f'{DN}/ext_data/{group_name}/cc14c567-83a5-4e24-974e-9dfdb14dee5a.h5ad',
       f'{DN}/ext_data/{group_name}/35df8878-f72d-400b-9626-58062590ef22.h5ad',
       f'{DN}/ext_data/{group_name}/a8f46fa6-3d16-4a86-865e-5611a9dcf0ae.h5ad',
       f'{DN}/ext_data/{group_name}/bdfa0cf3-7e2a-4794-90d0-7ef337011766.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.itprojex.h5ad',
           f'{DN}/ext_data/{group_name}/GPR85.mixed.mgein.h5ad',
           f'{DN}/ext_data/{group_name}/GPR85.mixed.cgein.h5ad',
           f'{DN}/ext_data/{group_name}/GPR85.mixed.nonneuronal.h5ad',
           f'{DN}/ext_data/{group_name}/GPR85.mixed.deepnonitex.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: suspension type indicates snRNA-seq, generating tissue with region and subregion
    # to add more information on cell type broad we add in the x area subclass/cluster then the w/in area
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','Class','cell_type','Region','Subregion',
                   'CrossArea_subclass','CrossArea_cluster','WithinArea_subclass','WithinArea_cluster']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine',
                   'Region','Subregion','CrossArea_subclass','CrossArea_cluster','WithinArea_subclass','WithinArea_cluster']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['tissue'] = adata.obs[['tissue','Region','Subregion']].astype(str).agg(':'.join, axis=1)
    adata.obs['cell_type_broad'] = adata.obs[['cell_type_broad','CrossArea_subclass','CrossArea_cluster','WithinArea_subclass','WithinArea_cluster']].astype(str).agg(':'.join, axis=1)
    del adata.obs['Region'], adata.obs['Subregion'], adata.obs['CrossArea_subclass'], adata.obs['CrossArea_cluster'], adata.obs['WithinArea_subclass'], adata.obs['WithinArea_cluster']
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Jorstad et al. 2023 (Science)'
    adata.var = adata.var[['ensembl_id']]    
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    adata.layers['raw_counts'] = adata.X
    # clean the raw loadings
    adata.raw = adata
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    adata.obsm['X_umap'] = adata.obsm['X_UMAP']
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# Seattle AD https://cellxgene.cziscience.com/collections/1ca90a2d-2943-483d-b678-b809bf464c30
# read in the data, subset for GPR85, write and delete to save space
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_alzheimers_seattle'
fns = [f'{DN}/ext_data/{group_name}/c32964d2-3339-441f-8e56-7177234c7876.h5ad',
       f'{DN}/ext_data/{group_name}/d3427e8c-c55d-4d4e-b15b-1a8774cd3a4b.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.mtg.h5ad',
           f'{DN}/ext_data/{group_name}/GPR85.mixed.dlpfc.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: suspension type indicates snRNA-seq, cell type broad becomes finer
    assert adata.raw.shape[1] ==  adata.shape[1]
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','Class','cell_type','Age at death',
                   'Subclass','Supertype','Years of education','Cognitive status','ADNC','Braak stage',
                   'Thal phase','CERAD score','APOE4 status','Lewy body disease pathology','LATE-NC stage','Microinfarct pathology',]
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine','Age at death',
                   'Subclass','Supertype','Years of education','Cognitive status','ADNC','Braak stage',
                   'Thal phase','CERAD score','APOE4 status','Lewy body disease pathology','LATE-NC stage','Microinfarct pathology',]
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['cell_type_broad'] = adata.obs[['cell_type_broad','Subclass','Supertype']].astype(str).agg(':'.join, axis=1)
    adata.obs['developmental_stage'] = adata.obs[['developmental_stage','Age at death']].astype(str).agg(':'.join, axis=1)
    del adata.obs['Age at death'], adata.obs['Subclass'], adata.obs['Supertype']
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Gabitto and Travaglini et al. 2024 (bioRxiv)'
    adata.var = adata.var[['ensembl_id']]   
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    # do delayed cleaning to avoid memory issues
    adata.layers['raw_counts'] = adata.X
    # clean the raw loadings
    adata.raw.var.index = adata.var.index
    adata.raw = adata
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# Brain matters https://cellxgene.cziscience.com/collections/9d63fcf1-5ca0-4006-8d8f-872f3327dbe9
# read in the data, subset for GPR85, write and delete to save space
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_brain_matters'
fns = [f'{DN}/ext_data/{group_name}/4bb40321-d632-474d-805d-6b665b70a7cf.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: snRNAseq
    assert adata.raw.shape[1] == adata.shape[1]
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','broad_cell_type','cell_type','author_cell_type','donor_cause_of_death','CauseOfDeath_category']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine','author_cell_type','donor_cause_of_death','CauseOfDeath_category']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['tissue'] = adata.obs[['tissue']].astype(str).agg(':'.join, axis=1)
    adata.obs['cell_type_broad'] = adata.obs[['cell_type_broad','author_cell_type']].astype(str).agg(':'.join, axis=1)
    del adata.obs['author_cell_type']
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Seeker et al. 2023 (Acta Neuropathol. Commun.)'
    adata.var = adata.var[['ensembl_id']]    
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    adata.layers['raw_counts'] = adata.X
    # clean the raw loadings
    adata.raw = adata
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# photoreceptor https://cellxgene.cziscience.com/collections/e9c73b68-e980-49c2-8d0d-1c9cab21507e
# NOT UTILIZED IN THIS STUDY (for future extensions, but we read it in nonetheless so kept for transparency)
# read in the data, subset for GPR85, write and delete to save space
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_photoreceptor'
fns = [f'{DN}/ext_data/{group_name}/3ba8c0d4-89c2-4c84-9d66-bb8e1a816286.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # the rest have been checked for this
    assert adata.obs['is_primary_data'].sum() == adata.shape[0]
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: snRNAseq
    assert adata.raw.shape[1] == adata.shape[1]
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','author_cell_type','cell_type']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Voigt et al. 2021 (Hum. Mol. Genet.)'
    adata.var = adata.var[['ensembl_id']]    
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    adata.layers['raw_counts'] = adata.X
    # clean the raw loadings
    adata.raw = adata
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# fetal retina https://cellxgene.cziscience.com/collections/5900dda8-2dc3-4770-b604-084eac1c2c82
# NOT UTILIZED IN THIS STUDY (for future extensions, but we read it in nonetheless so kept for transparency)
# read in the data, subset for GPR85, write and delete to save space
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_fetal_retina'
fns = [f'{DN}/ext_data/{group_name}/8aa25b63-8f8e-47ef-af65-7ad951169e93.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # the rest have been checked for this
    assert adata.obs['is_primary_data'].sum() == adata.shape[0]
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: snRNAseq
    assert adata.raw.shape[1] == adata.shape[1]
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','author_cell_type','cell_type','majorclass','subclass']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine','majorclass','subclass']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['cell_type_broad'] = adata.obs[['cell_type_broad','majorclass','subclass']].astype(str).agg(':'.join, axis=1)
    del adata.obs['majorclass'], adata.obs['subclass']
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Zuo, Cheng, Ferdous et al. 2024 (Nat. Commun.)'
    adata.var = adata.var[['ensembl_id']]    
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    adata.layers['raw_counts'] = adata.X
    # clean the raw loadings
    adata.raw = adata
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# human retina https://cellxgene.cziscience.com/collections/4c6eaf5c-6d57-4c76-b1e9-60df8c655f1e
# NOT UTILIZED IN THIS STUDY (for future extensions, but we read it in nonetheless so kept for transparency)
# this includes the ### adult retina https://cellxgene.cziscience.com/collections/af893e86-8e9f-41f1-a474-ef05359b1fb7
# data and thus we do not include the adult retina data additionally
# read in the data, subset for GPR85, write and delete to save space
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_human_retina'
fns = [f'{DN}/ext_data/{group_name}/488d9376-5b2f-4557-93ed-8524903aa00a.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: snRNAseq
    assert adata.raw.shape[1] == adata.shape[1]
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','author_cell_type','cell_type','study_name','AC_subclass','AC_cluster','BC_subclass','RGC_cluster']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine','study_name','AC_subclass','AC_cluster','BC_subclass','RGC_cluster']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['cell_type_broad'] = adata.obs[['cell_type_broad','AC_subclass','AC_cluster','BC_subclass','RGC_cluster']].astype(str).agg(':'.join, axis=1)
    del adata.obs['AC_subclass'], adata.obs['AC_cluster'], adata.obs['BC_subclass'], adata.obs['RGC_cluster']
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Li, Wang, Ibarra, and Cheng et al. 2023 (Bioarxiv)'
    adata.obs.loc[adata.obs['study_name'] == 'Shekhar_GSE237204','study'] = 'Hahn and Monavarfeshani et al. 2023 (Nature)'
    adata.obs.loc[adata.obs['study_name'] == 'Chen_b_GSE226108','study'] = 'Liang et al. 2023 (Cell Genom.)'
    adata.obs.loc[adata.obs['study_name'] == 'Chen_c_GSE247157','study'] = 'Wang and Cheng et al. 2023 (Genome Biol.)'
    del adata.obs['study_name']
    adata.var = adata.var[['ensembl_id']]  
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    # clean the raw loadings
    adata.raw = adata
    adata.layers['raw_counts'] = adata.X
    adata.write(out_fns[0])
    !rm -rf '{fn}'

In [ ]:
# NOT UTILIZED IN THIS STUDY (for future extensions, but we read it in nonetheless so kept for transparency)
# this includes the ### adult retina https://cellxgene.cziscience.com/collections/af893e86-8e9f-41f1-a474-ef05359b1fb7
# data and thus we do not include the adult retina data additionally
# read in the data, subset for GPR85, write and delete to save space
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_human_retina'
fns = [f'{DN}/ext_data/{group_name}/64175889-d600-4b58-97ea-e74be80206e5.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.supp.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: scRNAseq
    assert adata.raw.shape[1] == adata.shape[1]
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','author_cell_type','cell_type','majorclass','study_name']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine','majorclass','study_name']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['cell_type_broad'] = adata.obs[['cell_type_broad','majorclass']].astype(str).agg(':'.join, axis=1)
    del adata.obs['majorclass']
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' scRNAseq'
    adata.obs['study'] = adata.obs['study_name'].astype(str)
    adata.obs.loc[adata.obs['study_name'] == 'Roska_EGAS00001004561','study'] = 'Cowan and Renner et al. 2020 (Cell)'
    adata.obs.loc[adata.obs['study_name'] == 'Sanes_GSE148077','study'] = 'Yang, Peng, and van Zyl et al. 2020 (Sci. Rep.)'
    adata.obs.loc[adata.obs['study_name'] == 'Wong_E-MTAB-7316','study'] = 'Lukowski and Lo et al. 2019 (EMBO J)'
    adata.obs.loc[adata.obs['study_name'] == 'Hafler_GSE137537','study'] = 'Menon et al. 2019 (Nat. Commun.)'
    adata.obs.loc[adata.obs['study_name'] == 'Scheetz_GSE130636','study'] = 'Voigt et al. 2019 (Exp. Eye Res.)'
    del adata.obs['study_name']
    adata.var = adata.var[['ensembl_id']]  
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    # clean the raw loadings
    adata.raw = adata
    adata.layers['raw_counts'] = adata.X
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# developing neocortex https://cellxgene.cziscience.com/collections/c8565c6a-01a1-435b-a549-f11b452a83a8
# define the inputs
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_dev_neoc'
fns = [f'{DN}/ext_data/{group_name}/50d3a96e-b0cc-4c40-a105-cd4ca0dbab78.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: snRNAseq
    assert adata.raw.shape[1] == adata.shape[1]
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','cluster_label','cell_type','brain_region','cortical_area']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine','brain_region','cortical_area']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['tissue'] = adata.obs[['tissue','brain_region','cortical_area']].astype(str).agg(':'.join, axis=1)
    del adata.obs['brain_region'], adata.obs['cortical_area']
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' scRNAseq'
    adata.obs['study'] = 'Bhaduri and Saandoval-Espinosa et al. 2021 (Nature)'
    adata.var = adata.var[['ensembl_id']]  
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    # clean the raw loadings
    adata.raw = adata
    adata.layers['raw_counts'] = adata.X
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# developing cerebral cortex https://cellxgene.cziscience.com/collections/ceb895f4-ff9f-403a-b7c3-187a9657ac2c
# define the inputs
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_dev_cc'
fns = [f'{DN}/ext_data/{group_name}/484dbc33-c7dc-4e5e-9954-7f2a1cc849bc.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: snRNAseq
    assert adata.raw.shape[1] == adata.shape[1]
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','author_cell_type','cell_type']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Zhu and Bendl et al. 2023 (Sci. Adv.)'
    adata.var = adata.var[['ensembl_id']]  
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    # clean the raw loadings
    adata.raw = adata
    adata.layers['raw_counts'] = adata.X
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# astrocytoma https://cellxgene.cziscience.com/collections/10bf5c50-8d85-4c5f-94b4-22c1363d9f31
# define the inputs
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_astrocytoma'
fns = [f'{DN}/ext_data/{group_name}/c3d6b166-5864-49d2-a843-b2993f0885ed.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: snRNAseq
    assert adata.raw.shape[1] == adata.shape[1]
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','customclassif','cell_type']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Hernández-Hernández et al. 2022 (HCA)'
    adata.var = adata.var[['ensembl_id']]  
    # store the raw counts
    print('cleaning up raw counts...')
    adata.X = adata.raw.X
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    # clean the raw loadings
    adata.raw = adata
    adata.layers['raw_counts'] = adata.X
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# first trimester brain https://cellxgene.cziscience.com/collections/4d8fed08-2d6d-4692-b5ea-464f1d072077
# define the inputs
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_first_trim'
fns = [f'{DN}/ext_data/{group_name}/58f2940e-49f7-4b5e-87b1-0e191af1fbfd.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: we check the suspension type to determine
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','CellClass','cell_type','Region','Subregion']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine','Region','Subregion']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['tissue'] = adata.obs['tissue'].astype(str)
    adata.obs['tissue'] = adata.obs[['tissue','Region','Subregion']].astype(str).agg(':'.join, axis=1)
    del adata.obs['Region'], adata.obs['Subregion']
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' scRNAseq'
    adata.obs['study'] = 'Braun and Danan-Gottthold et al. 2023 (Science)'
    adata.var = adata.var[['ensembl_id']]  
    # store the raw counts
    print('cleaning up raw counts...')
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    # clean the raw loadings
    adata.raw = adata
    adata.layers['raw_counts'] = adata.X
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# mammalian cerebellum https://cellxgene.cziscience.com/collections/72d37bc9-76cc-442d-9131-da0e273862db
# define the inputs
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
group_name = 'cxg_mam_cer'
fns = [f'{DN}/ext_data/{group_name}/b23a3e02-d6f9-43a3-bb1b-38af6860c092.h5ad']
out_fns = [f'{DN}/ext_data/{group_name}/GPR85.mixed.h5ad']
for fn, out_fn in zip(fns, out_fns):
    print('reading in data...')
    adata = sc.read_h5ad(fn)
    # reassign the variable names
    print('reassigning variable names...')
    adata.var['ensembl_id'] = adata.var.index
    adata.var.index = adata.var['feature_name'].astype(str)
    adata.var_names_make_unique()
    # notes: we check the suspension type to determine
    org_columns = ['donor_id','disease','development_stage','self_reported_ethnicity','sex',
                   'assay','tissue','author_cell_type','cell_type']
    obs_columns = ['donor','disease_state','developmental_stage','reported_ethnicity','reported_sex',
                   'sequencing_kit','tissue','cell_type_broad','cell_type_fine']
    adata.obs = adata.obs[org_columns]
    adata.obs.columns = obs_columns
    adata.obs['sequencing_kit'] = adata.obs['sequencing_kit'].astype(str) + ' snRNAseq'
    adata.obs['study'] = 'Sepp and Leiss et al. 2023 (Nature)'
    adata.var = adata.var[['ensembl_id']]
    # store the raw counts
    print('cleaning up raw counts...')
    # retrieve the frequency of the GPR85 gene
    print('finding GPR85+ cells...')
    freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
    # subset the data by cutoff to only get high confidence GPR85 positive cells
    fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
    sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
    ax.set(xlabel='GPR85+ % of counts')
    # perform the cutting
    np.random.seed(0)
    idxs_pos = adata.obs.index[freq >= 5e-4].copy()
    idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
    idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
    # save the data
    adata = adata[idxs_pos.union(idxs_neg)].copy()
    # clean the raw loadings
    adata.raw = adata
    adata.layers['raw_counts'] = adata.X
    adata.write(out_fn)
    !rm -rf '{fn}'

In [ ]:
# brain vasculture mixture https://cells-test.gi.ucsc.edu/?ds=brain-vasc-atlas https://www.nature.com/articles/s41586-021-04369-3
from scipy.sparse import csr_matrix
from scipy.io import mmread
# read in the data accordingly
dn = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85/ext_data/ucsc_brain_vasc/'
X = csr_matrix(mmread(f'{dn}/matrix.mtx.gz'))
obs_names = pd.read_table(f'{dn}/barcodes.tsv.gz', header=None, index_col=None)[0]
var_names = pd.read_table(f'{dn}/features.tsv.gz', header=None, index_col=None)[0]
meta = pd.read_table(f'{dn}/meta.tsv', index_col=0)
umap = pd.read_table(f'{dn}/UMAP.coords.tsv.gz', header=None, index_col=0)

In [ ]:
from anndata import AnnData
# convert into anndata object and rename accordingly
adata = AnnData(X.T)
adata.obs_names = obs_names
adata.var_names = var_names
adata.obs = meta
adata.obsm['X_umap'] = umap.values
adata.obs = adata.obs[['Gender','Sample','Treat','Region','Cell_Type','Age','APOE4','APOE34','APOE_Number']]
adata.obs.columns = ['reported_sex','donor','disease_state','tissue','cell_type_fine','developmental_stage','APOE4','APOE34','APOE_Number']
adata.obs['study'] = 'Yang et al. 2022 (Nature)'
adata.obs['sequencing_kit'] = '10x 3ʹ v3 snRNAseq'

In [ ]:
# store the raw counts
print('cleaning up raw counts...')
# retrieve the frequency of the GPR85 gene
print('finding GPR85+ cells...')
freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
# subset the data by cutoff to only get high confidence GPR85 positive cells
fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
ax.set(xlabel='GPR85+ % of counts')
# perform the cutting
np.random.seed(0)
idxs_pos = adata.obs.index[freq >= 5e-4].copy()
idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
# save the data
adata = adata[idxs_pos.union(idxs_neg)].copy()
# clean the raw loadings
adata.raw = adata
adata.layers['raw_counts'] = adata.X
adata.write(f'{dn}/GPR85.mixed.h5ad')

In [ ]:
# brain endothelial cell https://cells-test.gi.ucsc.edu/?ds=brain-vasc https://www.nature.com/articles/s41586-024-07493-y/figures/1
from scipy.sparse import csr_matrix
from scipy.io import mmread
# read in the data accordingly
dn = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85/ext_data/ucsc_brain_ec_sorted'
X = csr_matrix(mmread(f'{dn}/counts_matrix.mtx.gz'))
obs_names = pd.read_table(f'{dn}/counts_barcodes.tsv.gz', header=None, index_col=None)[0]
var_names = pd.read_table(f'{dn}/counts_features.tsv.gz', header=None, index_col=None)[0]
meta = pd.read_table(f'{dn}/meta.tsv', index_col=0)
umap = pd.read_table(f'{dn}/UMAP.coords.tsv.gz', header=None, index_col=0)

In [ ]:
from anndata import AnnData
# convert into anndata object and rename accordingly
adata = AnnData(X.T)
adata.obs_names = obs_names
adata.var_names = var_names
adata.obs = meta
adata.obsm['X_umap'] = umap.values
del adata.obs['PATH2'], adata.obs['PATH3']
adata.obs.columns = ['disease_state','cell_type_fine','donor','reported_sex','developmental_stage']
adata.obs['study'] = 'Wälchli and Ghobrial et al. 2024 (Nature)'
adata.obs['sequencing_kit'] = '10x 3ʹ v3 scRNAseq'

In [ ]:
# store the raw counts
print('cleaning up raw counts...')
# retrieve the frequency of the GPR85 gene
print('finding GPR85+ cells...')
freq = adata[:, adata.var.index == 'GPR85'].X.sum(1).A1 / adata.X.sum(1).A1
# subset the data by cutoff to only get high confidence GPR85 positive cells
fig, ax = plt.subplots(figsize=[6, 3]); ax.grid(False)
sns.kdeplot(freq * 100, fill=True, bw_adjust=0.1, color='k', lw=1.5)
ax.set(xlabel='GPR85+ % of counts')
# perform the cutting
np.random.seed(0)
idxs_pos = adata.obs.index[freq >= 5e-4].copy()
idxs_leftover = adata.obs.index[~adata.obs.index.isin(idxs_pos)]
idxs_neg = np.random.choice(idxs_leftover, size=len(idxs_pos), replace=False)
# save the data
adata = adata[idxs_pos.union(idxs_neg)].copy()
# clean the raw loadings
adata.raw = adata
adata.layers['raw_counts'] = adata.X
adata.write(f'{dn}/GPR85.mixed.h5ad')

### integrate single-cell atlases - concatenate data

In [ ]:
from glob import glob
# retrieve filenames
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
fns = glob(f'{DN}/ext_data/*/GPR85*.h5ad')
# load in files sequentially creating a batch key called path_to_data
adatas, batch_categories = [], []
for fn in fns:
    adata = sc.read_h5ad(fn)
    print(fn, adata.shape, adata.var.index.intersection(['PECAM1','GPR85','ACTA2','GLS','OLIG1','ITGAM']).shape[0])
    adatas.append(adata)
    batch_categories.append(fn)

In [ ]:
# concatentate the objects together
adata = adatas[0].concatenate(adatas[1:], batch_key='path_to_data', batch_categories=batch_categories)
adata.var.columns = adata.var.columns.str.replace('ensembl_id-/lustre/scratch126/cellgen/team205/dc20/proj_GPR85/ext_data/','')
adata.var.columns = adata.var.columns.str.replace('/GPR85.mixed','').str.replace('.h5ad','')
adata.obs['developmental_stage'] = adata.obs['developmental_stage'].astype(str)
adata

In [ ]:
# confirm there are all raw counts, get total cell number
adata.obs['path_to_data'].value_counts()

In [ ]:
# confirm there are all raw counts, get non-integer cell number
adata.obs.loc[(adata.X.sum(1) * 10).A1 % 10 != 0, 'path_to_data'].value_counts()

### integrate single-cell atlases – preprocessing for dimension reduction

In [ ]:
# check for any outliers and unannotated prior and remove them
outliers = [x for x in adata.obs['cell_type_broad'].astype(str).unique() if 'outlier' in x.lower()]
unknowns = [x for x in adata.obs['cell_type_fine'].astype(str).unique() if 'unknown' in x.lower()]
mask = adata.obs['cell_type_broad'].isin(outliers) | adata.obs['cell_type_fine'].isin(unknowns)
adata = adata[~mask].copy()
adata

In [ ]:
# filter for robustly expressed genes
sc.pp.filter_genes(adata, min_cells=1)
# calculate qc metrics
adata.obs['total_counts'] = adata.X.sum(1).A1
# quantify the percentages
mt_mask = adata.var.index.str.startswith('MT-')
rp_mask = adata.var.index.str.startswith('RPS') | adata.var.index.str.startswith('RPL')
adata.obs['perc_mito'] = adata[:, mt_mask].X.sum(1).A1 / adata.obs['total_counts']
adata.obs['perc_ribo'] = adata[:, rp_mask].X.sum(1).A1 / adata.obs['total_counts']
# normalize the data to 1e4-summed ln1p
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [ ]:
# define the suspension type
adata.obs['suspension_type'] = 'cell'
adata.obs.loc[adata.obs['sequencing_kit'].str.endswith('snRNAseq'), 'suspension_type'] = 'nuclei'
# find highly variable genes
adata_ = adata[adata.obs['suspension_type'] == 'cell'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_cells = adata_.var.index[adata_.var['highly_variable']]
adata_ = adata[adata.obs['suspension_type'] == 'nuclei'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_nuclei = adata_.var.index[adata_.var['highly_variable']]
hvgs = hvgs_cells.union(hvgs_nuclei)
adata.var['highly_variable'] = adata.var.index.isin(hvgs)
print(adata.var['highly_variable'].sum())

### integrate single-cell atlases – dimension reduction and batch correction via scVI initial review

In [ ]:
# use scVI to compute dimension reduction
import scvi
# derive a batch agg variable to account for donor differences due to issues in large-scale integration without it
adata.obs['batch_agg'] = adata.obs[['study','sequencing_kit']].astype(str).agg(':'.join, axis=1)
# subset for highly variable genes
adata_scvi = adata[:, adata.var['highly_variable']].copy()
del adata_scvi.raw
# setup the parameters
params = {
    'encode_covariates': True,
    'dropout_rate': 0.2,
    'n_layers': 3,
    'n_latent': 50,
    'dispersion': 'gene-batch'
}
# create the actual model
scvi.model.SCVI.setup_anndata(adata_scvi, batch_key='suspension_type', categorical_covariate_keys=['batch_agg'],
                              layer='raw_counts', continuous_covariate_keys=['total_counts','perc_mito','perc_ribo'])
vae = scvi.model.SCVI(adata_scvi, **params)
# train the model
vae.train(early_stopping=True, accelerator='gpu', max_epochs=100)
adata.obsm['X_scvi'] = vae.get_latent_representation()
adata.write(f'{DN}/out_data/h5ads/int.nrm.h5ad')

In [ ]:
# plot the training and validation loss
for loss in ['train_loss_epoch','validation_loss']:
    fig, ax = plt.subplots()
    ax.grid(False)
    ax.plot(vae.history_[loss], label=loss)
    ax.set(xlabel='Epoch', ylabel='Loss')
    ax.legend(frameon=False)

In [ ]:
# compute knn, umap, and leiden
print('computing neighbors...')
sc.pp.neighbors(adata, random_state=0, use_rep='X_scvi')
print('computing umap...')
sc.tl.umap(adata, random_state=0)
print('computing leiden...')
sc.tl.leiden(adata, random_state=0, key_added='leiden_scvi')
adata.write(f'{DN}/out_data/h5ads/int.nrm.h5ad')

In [ ]:
# plot the final projection
sc.pl.umap(adata, color=['sequencing_kit', 'study'], ncols=1)

In [ ]:
# look at the expression
sc.pl.umap(adata, color=['PECAM1','ACTA2','GPR85'], use_raw=False)

In [ ]:
# look at the expression
sc.pl.umap(adata, color=['PECAM1','ACTA2','GPR85'], use_raw=False, vmin=2, vmax=3, cmap='Reds')

In [ ]:
# remove retinal atlases for this study
retinal_studies = ['Li, Wang, Ibarra, and Cheng et al. 2023 (Bioarxiv)',
                   'Hahn and Monavarfeshani et al. 2023 (Nature)',
                   'Liang et al. 2023 (Cell Genom.)',
                   'Wang and Cheng et al. 2023 (Genome Biol.)',
                   'Cowan and Renner et al. 2020 (Cell)',
                   'Yang, Peng, and van Zyl et al. 2020 (Sci. Rep.)',
                   'Lukowski and Lo et al. 2019 (EMBO J)',
                   'Menon et al. 2019 (Nat. Commun.)',
                   'Voigt et al. 2019 (Exp. Eye Res.)',
                   'Zuo, Cheng, Ferdous et al. 2024 (Nat. Commun.)',
                   'Voigt et al. 2021 (Hum. Mol. Genet.)']
adata_ = adata[adata.obs['study'].isin(retinal_studies)].copy()
adata_.write(f'{DN}/out_data/h5ads/int.nrm.retinal.h5ad')
adata_ = adata[~adata.obs['study'].isin(retinal_studies)].copy()
adata_.write(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')
del adata_

### integrate single-cell atlases – whole brain batch correction and dimension reduction via scVI

In [ ]:
# let's try this again
adata = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')
# remove the poor mitochondrial cells
adata = adata[adata.obs['cell_type_fine'] != 'Mitochondrial'].copy()

In [ ]:
# define the suspension type
adata.obs['suspension_type'] = 'cell'
adata.obs.loc[adata.obs['sequencing_kit'].str.endswith('snRNAseq'), 'suspension_type'] = 'nuclei'
# find highly variable genes
adata_ = adata[adata.obs['suspension_type'] == 'cell'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_cells = adata_.var.index[adata_.var['highly_variable']]
adata_ = adata[adata.obs['suspension_type'] == 'nuclei'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_nuclei = adata_.var.index[adata_.var['highly_variable']]
hvgs = hvgs_cells.union(hvgs_nuclei)
adata.var['highly_variable'] = adata.var.index.isin(hvgs)
print(adata.var['highly_variable'].sum())

In [ ]:
# use scVI to compute dimension reduction
import scvi
# derive a batch agg variable to account for donor differences due to issues in large-scale integration without it
adata.obs['batch_agg'] = adata.obs[['study','sequencing_kit']].astype(str).agg(':'.join, axis=1)
# subset for highly variable genes
adata_scvi = adata[:, adata.var['highly_variable']].copy()
del adata_scvi.raw
# setup the parameters
params = {
    'encode_covariates': True,
    'dropout_rate': 0.2,
    'n_layers': 3,
    'n_latent': 50,
    'dispersion': 'gene-batch'
}
# create the actual model
scvi.model.SCVI.setup_anndata(adata_scvi, batch_key='suspension_type', categorical_covariate_keys=['batch_agg'],
                              layer='raw_counts', continuous_covariate_keys=['total_counts','perc_mito','perc_ribo'])
vae = scvi.model.SCVI(adata_scvi, **params)
# train the model
vae.train(early_stopping=True, accelerator='gpu', max_epochs=100)
adata.obsm['X_scvi'] = vae.get_latent_representation()
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')

In [ ]:
# plot the training and validation loss
for loss in ['train_loss_epoch','validation_loss']:
    fig, ax = plt.subplots()
    ax.grid(False)
    ax.plot(vae.history_[loss], label=loss)
    ax.set(xlabel='Epoch', ylabel='Loss')
    ax.legend(frameon=False)

In [ ]:
# compute knn, umap, and leiden
print('computing neighbors...')
sc.pp.neighbors(adata, random_state=0, use_rep='X_scvi')
print('computing umap...')
sc.tl.umap(adata, random_state=0)
print('computing leiden...')
sc.tl.leiden(adata, random_state=0, key_added='leiden_scvi')
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')

In [ ]:
# plot the final projection
sc.pl.umap(adata, color=['sequencing_kit', 'study'], ncols=1)

### harmonize single-cell annotations

In [ ]:
import cellhint
alignment = cellhint.harmonize(adata, 'study', 'cell_type_fine', use_rep='X_scvi',
                               use_pct=True, p_thres=1e-3, random_state=0,
                               minimum_unique_percents=(0.75, 0.80, 0.85, 0.90, 0.95),
                               minimum_divide_percents=(0.010, 0.025, 0.050, 0.075, 0.100),
                               maximum_novel_percent=(0.50))

In [ ]:
# add on the cellhint annotation
adata.obs['cellhint_reannotation'] = alignment.reannotation['reannotation']
# resolve the mapping
mapping = {'UNRESOLVED = UNRESOLVED = UNRESOLVED = glutamatergic neuron = UNRESOLVED ∋ L2/3-6 intratelencephalic projecting glutamatergic neuron = NONE = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'NONE = NONE = NONE = NONE = NONE = pvalb GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = capillary endothelial cell = cerebral cortex endothelial cell = cerebral cortex endothelial cell = endothelial cell = NONE = NONE = NONE = NONE = NONE':'endothelial',
'astrocyte = astrocyte = Astrocyte = astrocyte = astrocyte of the cerebral cortex = astrocyte of the cerebral cortex = astrocyte = NONE = NONE = NONE = NONE = NONE':'astrocyte',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = oligodendrocyte precursor cell = oligodendrocyte precursor cell = oligodendrocyte precursor cell = NONE = NONE = NONE = NONE = NONE':'oligodendrocyte',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = GABAergic neuron = UNRESOLVED ∋ microglial cell = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron/microglia',
'NONE = NONE = NONE = NONE = NONE = VIP GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = caudal ganglionic eminence derived interneuron = caudal ganglionic eminence derived interneuron = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'oligodendrocyte = oligodendrocyte = Oligo = oligodendrocyte = oligodendrocyte = oligodendrocyte = oligodendrocyte = UNRESOLVED = NONE = NONE = NONE = NONE':'oligodendrocyte',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = GABAergic neuron = UNRESOLVED ∋ sncg GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = chandelier pvalb GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = lamp5 GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = GABAergic neuron = UNRESOLVED ∋ sst GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = glutamatergic neuron = UNRESOLVED ∋ corticothalamic-projecting glutamatergic cortical neuron = NONE = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = glutamatergic neuron = UNRESOLVED ∋ L6b glutamatergic cortical neuron = NONE = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'NONE = NONE = NONE = NONE = NONE = near-projecting glutamatergic cortical neuron = NONE = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = mural cell = vascular leptomeningeal cell = vascular leptomeningeal cell = UNRESOLVED = NONE = NONE = NONE = NONE = NONE':'VLMC',
'NONE = NONE = NONE = NONE = NONE = L5 extratelencephalic projecting glutamatergic cortical neuron = NONE = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'glutamatergic neuron = neuron = NONE = UNRESOLVED = NONE = NONE ∋ glutamatergic neuron = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'glutamatergic neuron = neuron = NONE = UNRESOLVED = NONE = NONE ∋ inhibitory interneuron = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron/GABAergic interneuron',
'NONE = leukocyte = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'leukocyte',
'microglial cell = central nervous system macrophage = Microglia/Mφ = microglial cell = microglial cell = NONE = microglial cell = UNRESOLVED = NONE = NONE = NONE = NONE':'microglia',
'endothelial cell of vascular tree = endothelial cell = Capillary = UNRESOLVED = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'endothelial',
'oligodendrocyte precursor cell = oligodendrocyte precursor cell = OPC = oligodendrocyte precursor cell = UNRESOLVED = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'oligodendrocyte precursor',
'NONE = Bergmann glial cell = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'bergmann glia',
'pericyte = fibroblast = NONE = NONE = NONE = NONE = UNRESOLVED = NONE = NONE = NONE = NONE = NONE':'VSMC/fibroblast',
'NONE = pericyte = NONE = NONE = NONE = NONE = NONE = UNRESOLVED = NONE = NONE = NONE = NONE':'VSMC',
'NONE = vascular associated smooth muscle cell ∋ P. Fibro = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'VSMC',
'NONE = choroid plexus epithelial cell = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'choroid plexus epithelial',
'NONE = ependymal cell = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'ependymal',
'NONE = vascular associated smooth muscle cell ∋ Pericyte = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'VSMC',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = endothelial cell of artery = NONE = NONE = NONE = UNRESOLVED = NONE = NONE = NONE = NONE':'endothelial',
'NONE = NONE = NONE = vascular associated smooth muscle cell = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'VSMC',
'NONE = NONE = NONE = cerebellar granule cell = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'cerebellar granule cell',
'NONE = NONE = NONE = differentiation-committed oligodendrocyte precursor = NONE = NONE = NONE = UNRESOLVED = NONE = NONE = NONE = NONE':'oligodendrocyte precursor (differentiation-committed)',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = leukocyte = NONE = NONE = NONE = UNRESOLVED = NONE = NONE = NONE = NONE':'leukocyte',
'NONE = NONE = NONE = central nervous system macrophage = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'microglia',
'NONE = NONE = NONE = neuron = NONE = NONE = NONE = UNRESOLVED = NONE = NONE = NONE = NONE':'neuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = glutamatergic neuron = NONE':'glutamatergic neuron',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = progenitor cell = NONE':'progenitor',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = Stem-to-EC ∋ forebrain radial glial cell ∈ interneuron':'stem-to-ec/radial glia/interneuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = Stem-to-EC ∋ oligodendrocyte precursor cell ∈ interneuron':'opc/interneuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = Stem-to-EC ∋ microglial cell = NONE':'microglia',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = blood vessel endothelial cell = NONE':'endothelial',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = cerebral cortex GABAergic interneuron = NONE':'GABAergic interneuron',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = cerebral cortex endothelial cell = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Cajal-Retzius cell = NONE':'cajal-retzius',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = neuron':'neuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = macrophage':'macrophage',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = endothelial cell':'endothelial',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = oligodendrocyte precursor cell':'oligodendrocyte precursor',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = T cell':'T cell',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = astrocyte':'astrocyte',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Bergmann glial cell':'bergmann glia',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = oligodendrocyte':'oligodendrocyte',
'NONE = NONE = NONE = NONE = NONE = NONE = radial glial cell = NONE = NONE = NONE = NONE = NONE':'radial glia',
'NONE = NONE = NONE = NONE = NONE = NONE = caudal ganglionic eminence derived interneuron = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = medial ganglionic eminence derived interneuron = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'NONE = NONE = NONE = NONE = NONE = NONE = pericyte = NONE = NONE = NONE = NONE = NONE':'VSMC',
'NONE = NONE = NONE = NONE = NONE = NONE = neural progenitor cell = NONE = NONE = NONE = NONE = NONE':'neural progenitor',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = vascular associated smooth muscle cell = NONE = NONE = NONE = NONE = NONE':'VSMC',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = Purkinje cell = NONE = NONE = NONE = NONE':'purkinje cell',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = glutamatergic neuron = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = macroglial cell ∋ glioblast = NONE = NONE = NONE':'macroglia/glioblast',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = neuroblast (sensu Vertebrata) ∋ neuroblast (sensu Vertebrata) = NONE = NONE = NONE':'neuroblast',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = neuroblast (sensu Vertebrata) ∋ neuron = NONE = NONE = NONE':'neuroblast',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = GABAergic neuron = NONE = NONE = NONE = NONE':'GABAergic neuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = macroglial cell ∋ neuroblast (sensu Nematoda and Protostomia) = NONE = NONE = NONE':'macroglia/neuroblast',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = macroglial cell ∋ radial glial cell = NONE = NONE = NONE':'macroglia/radial glia',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = progenitor cell = NONE = NONE = NONE = NONE':'progenitor',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = microglial cell = NONE = NONE = NONE = NONE':'microglia',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = cerebellar granule cell = NONE = NONE = NONE = NONE':'cerebellar granule cell',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = cerebellar granule cell precursor = NONE = NONE = NONE = NONE':'cerebellar granule cell precursor',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = central nervous system macrophage = NONE = NONE = NONE = NONE':'microglia',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = interneuron = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = oligodendrocyte precursor cell = NONE = NONE = NONE = NONE':'oligodendrocyte precursor',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = unipolar brush cell = NONE = NONE = NONE = NONE':'unipolar brush cell',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = noradrenergic cell = NONE = NONE = NONE = NONE':'noradrenergic cell',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = erythroid lineage cell = NONE = NONE = NONE = NONE':'erythroid',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = immature astrocyte = NONE = NONE = NONE = NONE':'immature astrocyte',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = Bergmann glial cell = NONE = NONE = NONE = NONE':'bergmann glia',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = oligodendrocyte = NONE = NONE = NONE = NONE':'oligodendrocyte',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = brain vascular cell = NONE = NONE = NONE = NONE':'endothelial (?)',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = glioblast = NONE = NONE = NONE = NONE':'glioblast',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = meningeal macrophage = NONE = NONE = NONE = NONE':'macrophage (meningeal)',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = differentiation-committed oligodendrocyte precursor = NONE = NONE = NONE = NONE':'oligodendrocyte precursor (differentiation-committed)',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = leukocyte = NONE = NONE = NONE = NONE':'leukocyte',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = T cell = NONE = NONE = NONE = NONE':'T cell',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = near-projecting glutamatergic cortical neuron = NONE = NONE = UNRESOLVED = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'NONE = NONE = NONE = NONE = corticothalamic-projecting glutamatergic cortical neuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = L6b glutamatergic cortical neuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'NONE = NONE = NONE = NONE = L5 extratelencephalic projecting glutamatergic cortical neuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'NONE = NONE = NONE = NONE = VIP GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'NONE = NONE = NONE = NONE = sncg GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'NONE = NONE = NONE = NONE = lamp5 GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'NONE = NONE = NONE = NONE = sst GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'NONE = NONE = NONE = NONE = pvalb GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'NONE = NONE = NONE = NONE = chandelier pvalb GABAergic cortical interneuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'NONE = NONE = NONE = NONE = L2/3-6 intratelencephalic projecting glutamatergic neuron = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'glutamatergic neuron',
'NONE = NONE = Veinous = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'endothelial',
'NONE = NONE = SMC = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'endothelial',
'NONE = NONE = Arterial = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'endothelial',
'inhibitory interneuron = NONE = Neuron = NONE = NONE = NONE = NONE = UNRESOLVED = NONE = NONE = NONE = NONE':'GABAergic interneuron',
'NONE = NONE = Ependymal = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'ependymal cell',
'NONE = NONE = M. Fibro = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'fibroblast (m.)',
'UNRESOLVED = UNRESOLVED = T cell = UNRESOLVED = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'T cell',
'T cell = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE':'T cell',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Large vein = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Capillary = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Vein = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Angiogenic capillary = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Artery = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Large artery = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Arteriole = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Venule = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Proliferating cell = NONE = NONE':'endothelial',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = EndoMT = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Proliferating Stem-to-EC = NONE = NONE':'endothelial',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Proliferating EndoMT = NONE = NONE':'endothelial',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = cell = NONE = NONE = NONE':'cell',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = erythrocyte = NONE = NONE = NONE':'erythroid',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = oligodendrocyte = NONE = NONE = NONE':'oligodendrocyte',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = macrophage = NONE = NONE = NONE':'macrophage (?)',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = neuroplacodal cell = NONE = NONE = NONE':'neuroplacodal',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = fibroblast = UNRESOLVED = NONE = NONE':'fibroblast',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = neural crest cell = UNRESOLVED = NONE = NONE':'neural crest'}
adata.obs['cell_type_int'] = adata.obs['cellhint_reannotation'].map(mapping)
# these are all endothelial
mask = adata.obs['study'] == 'Wälchli and Ghobrial et al. 2024 (Nature)'
adata.obs.loc[mask, 'cell_type_int'] = 'endothelial'

In [ ]:
# color the data on for the preliminary integration
sc.pl.umap(adata, color=['cell_type_int'])

In [ ]:
# perform dendorgram clustering to confirm our assignment and resolve the unresolved
sc.tl.dendrogram(adata, groupby='cell_type_int', use_rep='X_scvi')
fig, ax = plt.subplots(figsize=[10, 4]); ax.grid(False)
sc.pl.dendrogram(adata, groupby='cell_type_int', ax=ax)

In [ ]:
# validate the main contributors are well annotated
adata.obs['cell_type_int'].value_counts()

In [ ]:
# rename accordingly
renamer = {'neural progenitor':'progenitor', 'oligodendrocyte precursor (differentiation-committed)':'oligodendrocyte precursor',
           'fibroblast (m.)':'fibroblast', 'GABAergic neuron':'GABAergic interneuron', 'endothelial (?)':'VLMC/VSMC',
           'macrophage (?)':'microglia', 'GABAergic interneuron/microglia':'microglia', 'macrophage (meningeal)':'microglia',
           'immature astrocyte':'glioblast', 'ependymal cell':'ependymal', 'cell':'erythroid', 'VLMC':'VLMC/VSMC',
           'VSMC/fibroblast':'VLMC/VSMC', 'neuron':'oligodendrocyte precursor', 'VSMC':'VLMC/VSMC',
           'glutamatergic neuron/GABAergic interneuron':'glutamatergic neuron', 'macroglia/radial glia':'radial glia',
           'stem-to-ec/radial glia/interneuron':'radial glia','macroglia/glioblast':'radial glia',
           'macroglia/neuroblast':'oligodendrocyte', 'macrophage':'microglia', 'T cell':'leukocyte',
           'opc/interneuron':'glioblast'}
adata.obs['cell_type_int'] = adata.obs['cell_type_int'].astype(str)
for k,v in renamer.items():
    adata.obs.loc[adata.obs['cell_type_int'] == k, 'cell_type_int'] = v
# color the data on
del adata.uns['cell_type_int_colors']
sc.pl.umap(adata, color=['cell_type_int'])

### validate annotations via celltypist

In [ ]:
# train a celltypist model to validate these annotations
adata_ref = adata.copy()
# find highly variable genes
sc.pp.filter_genes(adata_ref, min_cells=1)
sc.pp.highly_variable_genes(adata_ref, n_top_genes=10000)
sc.pl.highly_variable_genes(adata_ref)
print(adata_ref.var['highly_variable'].sum())
# sample 500 cells from each cell type for `adata_ref`.
# all cells from a given cell type will be selected if the cell type size is < 500.
sampled_cell_index = celltypist.samples.downsample_adata(adata_ref, mode='each', n_cells=500, by='cell_type_int', return_index=True)
print(f'Number of downsampled cells for training: {len(sampled_cell_index)}')
# use `celltypist.train` to quickly train a rough CellTypist model.
model_fs = celltypist.train(adata_ref[sampled_cell_index], 'cell_type_int', n_jobs=1, max_iter=10, use_SGD=True)

In [ ]:
# use the medium trained model to derive genes
gene_index = np.argpartition(np.abs(model_fs.classifier.coef_), -100, axis = 1)[:, -100:]
gene_index = np.unique(gene_index)
print(f'Number of genes selected: {len(gene_index)}')
# add `check_expression = False` to bypass expression check with only a subset of genes.
model = celltypist.train(adata_ref[sampled_cell_index], 'cell_type_int', check_expression=False, n_jobs=1, max_iter=100)
# save the model.
model.write(f'{DN}/out_data/celltypist/model_from_neural.pkl')

In [ ]:
# perform the predictions
adata_ref = celltypist.annotate(adata_ref, model=f'{DN}/out_data/celltypist/model_from_neural.pkl', majority_voting=True, mode='best match')
adata_ref = adata_ref.to_adata(insert_prob=True)

In [ ]:
# write down the data
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')
adata_ref.write(f'{DN}/out_data/h5ads/int.nrm.neural.w_celltypist.h5ad')

### examine for GPR85 bias

In [ ]:
from tqdm import tqdm
# attempt to examine this on a per donor scale
adata.obs['donor_study'] = adata.obs[['donor','study']].agg(':'.join, axis=1)
diffs = []
for donor in tqdm(adata.obs['donor_study'].unique()):
    mask = adata.obs['donor_study'] == donor
    counts_high = adata.obs.loc[(adata.obs['GPR85_binary'] == 'High')&mask, 'cell_type_int'].value_counts()
    counts_low = adata.obs.loc[(adata.obs['GPR85_binary'] == 'Negative')&mask, 'cell_type_int'].value_counts()
    diff = counts_high - counts_low
    diff = diff / adata.obs.loc[mask, 'cell_type_int'].value_counts()
    diff = pd.DataFrame(diff)
    diff.columns = ['diff']
    diff['donor_study'] = donor
    diff['disease_state'] = adata.obs.loc[mask, 'disease_state'].unique()[0]
    assert len(adata.obs.loc[mask, 'disease_state'].unique()) == 1
    diffs.append(diff)

In [ ]:
# remove cell types that have less than ten cells
mask = adata_ref.obs['disease_state'].isin(['normal','Control','TL'])
counts = adata_ref.obs.loc[mask, 'cell_type_int'].value_counts()
percs = counts / counts.sum()
valids = percs.index[percs >= 0.01]

In [ ]:
# get the plot, for normal conditions
fig, ax = plt.subplots(figsize=[5, 4]); ax.grid(False)
# retrieve the data
data = pd.concat(diffs, axis=0).reset_index()
mask = data['disease_state'].isin(['normal','Control','TL'])
diffs_normal = data[mask].groupby('index').mean(numeric_only=True).iloc[:, 0]
order = [x for x in diffs_normal.sort_values().index if x in valids]
# retrieve the colors
cmap = get_cmap('RdBu_r')
colors = (diffs_normal.loc[order] + 1) / 2
colors = [to_hex(cmap(x)) for x in colors]
sns.barplot(x='index', y='diff', data=data[mask], order=order,
            ci=68, errwidth=1.5, capsize=0.3, errcolor='k',
            linewidth=1.5, edgecolor='k', saturation=1, palette=colors)
ax.set(xlabel='Cell type', ylabel='GPR85 expression')
ax.plot([-1, len(order)], [0]*2, color='k')
ax.set_xlim(-1, len(order))
ax.tick_params(axis='x', labelrotation=90)

### validate celltypist predictions for whole brain

In [ ]:
# read in the data
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
adata = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')
adata_ref = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural.w_celltypist.h5ad')

In [ ]:
# compare the annotations
df = pd.concat([adata.obs['cell_type_int'], adata_ref.obs['predicted_labels']], axis=1)
df.columns = ['manual','automatic']
df = df.value_counts().reset_index()
df.columns = ['manual','automatic','counts']
print(df.loc[df['manual'] == df['automatic'], 'counts'].sum() / df['counts'].sum())
# flip the results
df = df.pivot_table(index='manual', columns='automatic', values='counts', aggfunc=np.sum).fillna(0)
df = (df.T / df.sum(1)).T
g = sns.clustermap(df, cmap='viridis', vmin=0, vmax=1, row_cluster=False, col_cluster=False,
                   xticklabels=1, yticklabels=1, figsize=[10, 10], cbar_pos=(.07, .77, .01, .08))
g.ax_heatmap.grid(False)

### examine for cross tissue cell type GPR85 bias

In [ ]:
from tqdm import tqdm
# prepare the data
adata.obs['tissue_simple'] = adata.obs['tissue'].str.split(':', expand=True).iloc[:, 0].str.lower()
adata.obs['tag'] = adata.obs[['tissue_simple','cell_type_int']].astype(str).agg(':'.join, axis=1)
# examine per donor as per before
adata.obs['donor_study'] = adata.obs[['donor','study']].agg(':'.join, axis=1)
diffs = []
for donor in tqdm(adata.obs['donor_study'].unique()):
    mask = adata.obs['donor_study'] == donor
    counts_high = adata.obs.loc[(adata.obs['GPR85_binary'] == 'High')&mask, 'tag'].value_counts()
    counts_low = adata.obs.loc[(adata.obs['GPR85_binary'] == 'Negative')&mask, 'tag'].value_counts()
    diff = counts_high - counts_low
    diff = diff / adata.obs.loc[mask, 'tag'].value_counts()
    diff = pd.DataFrame(diff)
    diff.columns = ['diff']
    diff['donor_study'] = donor
    diff['disease_state'] = adata.obs.loc[mask, 'disease_state'].unique()[0]
    assert len(adata.obs.loc[mask, 'disease_state'].unique()) == 1
    diffs.append(diff)
data = pd.concat(diffs, axis=0).reset_index()
data[['tissue','cell type']] = data['index'].str.split(':', expand=True)
mask = adata.obs['disease_state'].isin(['normal','Control','TL'])
diffs = data[data['disease_state'].isin(['normal','Control','TL'])].pivot_table(index='tissue', columns='cell type', values='diff')
diffs = diffs.loc[diffs.index.astype(str) != 'nan'].T
diffs = diffs.loc[diffs.index.astype(str) != 'nan'].T

In [ ]:
# create plot
diffs_fillna = diffs.fillna(0)
g = sns.clustermap(diffs_fillna, vmin=-1, vmax=1, cmap='RdBu_r', method='ward',
                   figsize=[9*1.15, 12*1.15], xticklabels=1, yticklabels=1,
                   dendrogram_ratio=(.1, .07), cbar_pos=(0, 1, .01, .04), mask=diffs.isna())
g.ax_heatmap.grid(False); g.ax_heatmap.set_facecolor('lightgray')

In [ ]:
# retrieve the data for statistical plots
mask = (~adata.obs['cell_type_int'].isin(['endothelial','VLMC/VSMC','fibroblast']))&(adata.obs['disease_state'].isin(['normal','Control','TL']))
percs = adata.obs.loc[mask, 'tissue_simple'].value_counts() / sum(mask)
valids = percs.index[percs >= 0.01]
mask = (data['cell type'] == 'glutamatergic neuron')&(data['disease_state'].isin(['normal','Control','TL']))
order = data[mask].groupby('tissue').mean(numeric_only=True)['diff'].sort_values().index
order = [x for x in order if x in valids]

In [ ]:
from matplotlib.cm import get_cmap
from matplotlib.colors import to_hex
# get the plot, for normal conditions
fig, ax = plt.subplots(figsize=[6, 4]); ax.grid(False)
cmap = get_cmap('RdBu_r')
diffs_normal = data[mask].groupby('tissue').mean(numeric_only=True)['diff']
colors = (diffs_normal.loc[order] + 1) / 2
colors = [to_hex(cmap(x)) for x in colors]
sns.barplot(x='tissue', y='diff', data=data[mask], order=order,
            ci=68, errwidth=1.5, capsize=0.3, errcolor='k',
            linewidth=1.5, edgecolor='k', saturation=1, palette=colors)
ax.set(xlabel='Cell type', ylabel='GPR85 expression')
ax.set_xlim(-1, len(order))
ax.tick_params(axis='x', labelrotation=90)

### subset and examine hippocampal related tissues

In [ ]:
# read in the data
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
adata = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')
adata.obs['tissue_simple'] = adata.obs['tissue'].str.split(':', expand=True).iloc[:, 0].str.lower()
# subset for relevant cell type
adata = adata[adata.obs['tissue_simple'].isin(['hippocampal formation','hippocampus'])].copy()
print(adata.obs['tissue_simple'].unique())
adata

In [ ]:
# find highly variable genes
adata_ = adata[adata.obs['suspension_type'] == 'cell'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_cells = adata_.var.index[adata_.var['highly_variable']]
adata_ = adata[adata.obs['suspension_type'] == 'nuclei'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_nuclei = adata_.var.index[adata_.var['highly_variable']]
hvgs = hvgs_cells.union(hvgs_nuclei)
adata.var['highly_variable'] = adata.var.index.isin(hvgs)
print(adata.var['highly_variable'].sum())

In [ ]:
# use scVI to compute dimension reduction
import scvi
# derive a batch agg variable to account for donor differences due to issues in large-scale integration without it
adata.obs['batch_agg'] = adata.obs[['study','sequencing_kit']].astype(str).agg(':'.join, axis=1)
# subset for highly variable genes
adata_scvi = adata[:, adata.var['highly_variable']].copy()
del adata_scvi.raw
# setup the parameters
params = {
    'encode_covariates': True,
    'dropout_rate': 0.2,
    'n_layers': 2,
    'n_latent': 50,
    'dispersion': 'gene-batch'
}
# create the actual model
scvi.model.SCVI.setup_anndata(adata_scvi, batch_key='suspension_type', categorical_covariate_keys=['batch_agg'],
                              layer='raw_counts', continuous_covariate_keys=['total_counts','perc_mito','perc_ribo'])
vae = scvi.model.SCVI(adata_scvi, **params)
# train the model
vae.train(early_stopping=True, accelerator='gpu', max_epochs=200)
adata.obsm['X_scvi'] = vae.get_latent_representation()
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural_hippocampal.h5ad')

In [ ]:
# plot the training and validation loss
for loss in ['train_loss_epoch','validation_loss']:
    fig, ax = plt.subplots()
    ax.grid(False)
    ax.plot(vae.history_[loss], label=loss)
    ax.set(xlabel='Epoch', ylabel='Loss')
    ax.legend(frameon=False)

In [ ]:
# compute knn, umap, and leiden
print('computing neighbors...')
sc.pp.neighbors(adata, random_state=0, use_rep='X_scvi')
print('computing umap...')
sc.tl.umap(adata, random_state=0)
print('computing leiden...')
sc.tl.leiden(adata, random_state=0, key_added='leiden_scvi')
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural_hippocampal.h5ad')

### examine hippocampal expression of ACh receptors

In [ ]:
# show dot plot expression
genes = adata.var.index[adata.var.index.str.startswith('CHRM') | adata.var.index.str.startswith('CHRN')].tolist() + ['GPR85']
sc.pl.dotplot(adata, groupby='cell_type_int', var_names=genes, use_raw=False)

### examine GPR85 bias within cells of the hippocampal formation

In [ ]:
from tqdm import tqdm
# attempt to examine this on a per donor scale
adata.obs['donor_study'] = adata.obs[['donor','study']].agg(':'.join, axis=1)
diffs = []
for donor in tqdm(adata.obs['donor_study'].unique()):
    mask = adata.obs['donor_study'] == donor
    counts_high = adata.obs.loc[(adata.obs['GPR85_binary'] == 'High')&mask, 'cell_type_int'].value_counts()
    counts_low = adata.obs.loc[(adata.obs['GPR85_binary'] == 'Negative')&mask, 'cell_type_int'].value_counts()
    diff = counts_high - counts_low
    diff = diff / adata.obs.loc[mask, 'cell_type_int'].value_counts()
    diff = pd.DataFrame(diff)
    diff.columns = ['diff']
    diff['donor_study'] = donor
    diff['disease_state'] = adata.obs.loc[mask, 'disease_state'].unique()[0]
    assert len(adata.obs.loc[mask, 'disease_state'].unique()) == 1
    diffs.append(diff)

In [ ]:
# get the plot, for normal conditions
fig, ax = plt.subplots(figsize=[6, 4]); ax.grid(False)
cmap = get_cmap('RdBu_r')
colors = (diffs_normal.loc[order] + 1) / 2
colors = [to_hex(cmap(x)) for x in colors]
sns.barplot(x='index', y='diff', data=data[mask], order=order,
            ci=68, errwidth=1.5, capsize=0.3, errcolor='k',
            linewidth=1.5, edgecolor='k', saturation=1, palette=colors)
ax.set(xlabel='Cell type', ylabel='GPR85 expression')
ax.set_xlim(-1, len(order))
ax.tick_params(axis='x', labelrotation=90)

### subset for microglia populations

In [ ]:
# read in the data
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
adata = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')
# subset for relevant cell type
adata = adata[adata.obs['cell_type_int'].isin(['microglia'])].copy()
adata

In [ ]:
# find highly variable genes
adata_ = adata[adata.obs['suspension_type'] == 'cell'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_cells = adata_.var.index[adata_.var['highly_variable']]
adata_ = adata[adata.obs['suspension_type'] == 'nuclei'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_nuclei = adata_.var.index[adata_.var['highly_variable']]
hvgs = hvgs_cells.union(hvgs_nuclei)
adata.var['highly_variable'] = adata.var.index.isin(hvgs)
print(adata.var['highly_variable'].sum())

In [ ]:
# use scVI to compute dimension reduction
import scvi
# derive a batch agg variable to account for donor differences due to issues in large-scale integration without it
adata.obs['batch_agg'] = adata.obs[['study','sequencing_kit']].astype(str).agg(':'.join, axis=1)
# subset for highly variable genes
adata_scvi = adata[:, adata.var['highly_variable']].copy()
del adata_scvi.raw
# setup the parameters
params = {
    'encode_covariates': True,
    'dropout_rate': 0.2,
    'n_layers': 2,
    'n_latent': 50,
    'dispersion': 'gene-batch'
}
# create the actual model
scvi.model.SCVI.setup_anndata(adata_scvi, batch_key='suspension_type', categorical_covariate_keys=['batch_agg'],
                              layer='raw_counts', continuous_covariate_keys=['total_counts','perc_mito','perc_ribo'])
vae = scvi.model.SCVI(adata_scvi, **params)
# train the model
vae.train(early_stopping=True, accelerator='gpu', max_epochs=200)
adata.obsm['X_scvi'] = vae.get_latent_representation()
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural_microglia.h5ad')

In [ ]:
# plot the training and validation loss
for loss in ['train_loss_epoch','validation_loss']:
    fig, ax = plt.subplots()
    ax.grid(False)
    ax.plot(vae.history_[loss], label=loss)
    ax.set(xlabel='Epoch', ylabel='Loss')
    ax.legend(frameon=False)

In [ ]:
# compute knn, umap, and leiden
print('computing neighbors...')
sc.pp.neighbors(adata, random_state=0, use_rep='X_scvi')
print('computing umap...')
sc.tl.umap(adata, random_state=0)
print('computing leiden...')
sc.tl.leiden(adata, random_state=0, key_added='leiden_scvi')
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural_microglia.h5ad')

### examine for GPR85 correlated genes in microglia

In [ ]:
# plot the relevant data
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
adata = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural_microglia.h5ad')

In [ ]:
from tqdm import tqdm
import scipy.stats as ss
# look at the correlative profiles
df = sc.get.obs_df(adata, keys=adata.var.index.tolist())
df = df.loc[:, df.nunique(0) > 1]
df_corr = pd.DataFrame(columns=['gene','r','p'])
for x in tqdm(df.columns):
    if x == 'GPR85': continue
    r, p = ss.spearmanr(df['GPR85'], df[x])
    df_corr.loc[df_corr.shape[0]] = x, r, p

In [ ]:
# define mask
mask = df_corr['gene'].str.startswith('MT-') | df_corr['gene'].str.startswith('MRPS')
mask = mask | df_corr['gene'].str.startswith('RPS') | df_corr['gene'].str.startswith('MRPL')
mask = mask | df_corr['gene'].str.startswith('RPL')
for x in df_corr[~mask].sort_values('r')['gene'][::-1][:100]:
    print(x)

In [ ]:
# define mask
mask = df_corr['gene'].str.startswith('MT-') | df_corr['gene'].str.startswith('MRPS')
mask = mask | df_corr['gene'].str.startswith('RPS') | df_corr['gene'].str.startswith('MRPL')
mask = mask | df_corr['gene'].str.startswith('RPL')
for x in df_corr[~mask].sort_values('r')['gene'][:100]:
    print(x)

### examine for ACh receptor expression in microglia

In [ ]:
# show dot plot expression
genes = adata.var.index[adata.var.index.str.startswith('CHRM') | adata.var.index.str.startswith('CHRN')].tolist() + ['GPR85']
sc.pl.dotplot(adata, groupby='cell_type_int', var_names=genes, use_raw=False)

### subset for vasculature related populations and create new dimension reduction

In [ ]:
# read in the data
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
adata = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')
# subset for relevant cell type
adata = adata[adata.obs['cell_type_int'].isin(['fibroblast','VLMC/VSMC','endothelial'])].copy()
adata

In [ ]:
# remove the fetal periph samples
adata.obs['disease_simple'] = adata.obs['disease_state'].astype(str)
kvs = {'Alzheimer disease':'AD','normal':'Control','acute myocardial infarction':'Control','heart failure':'Control', 'TL':'Control'}
for k, v in kvs.items():
    adata.obs['disease_simple'] = adata.obs['disease_simple'].str.replace(k, v)
adata.obs['disease_simple'] = adata.obs['disease_simple'].str.capitalize()
adata.obs['disease_simple'] = adata.obs['disease_simple'].str.replace('Ad','AD')
# multiple rounds were used to verify proper renaming
adata.obs['disease_simple'] = adata.obs['disease_simple'].astype(str)
kvs = {'Control':'Control (adult)', 'Fetalperiph':'Control (fetal periph.)',
       'Fetalcns':'Control (fetal CNS)','Met':'Cancer (lung metastasis)',
       'Avm':'Atrioventricular malformation','AD':'Alzheimer\'s disease',
       'Gbm':'Cancer (GBM)','Lgg':'Cancer (lower-grade glioma)','Men':'Cancer (meningioma)',}
for k, v in kvs.items():
    adata.obs['disease_simple'] = adata.obs['disease_simple'].str.replace(k, v)
# actual removal because these are not CNS
adata = adata[adata.obs['disease_simple'] != 'Control (fetal periph.)'].copy()

In [ ]:
# find highly variable genes
adata_ = adata[adata.obs['suspension_type'] == 'cell'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_cells = adata_.var.index[adata_.var['highly_variable']]
adata_ = adata[adata.obs['suspension_type'] == 'nuclei'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_nuclei = adata_.var.index[adata_.var['highly_variable']]
hvgs = hvgs_cells.union(hvgs_nuclei)
adata.var['highly_variable'] = adata.var.index.isin(hvgs)
print(adata.var['highly_variable'].sum())

In [ ]:
# use scVI to compute dimension reduction
import scvi
# derive a batch agg variable to account for donor differences due to issues in large-scale integration without it
adata.obs['batch_agg'] = adata.obs[['study','sequencing_kit']].astype(str).agg(':'.join, axis=1)
# subset for highly variable genes
adata_scvi = adata[:, adata.var['highly_variable']].copy()
del adata_scvi.raw
# setup the parameters
params = {
    'encode_covariates': True,
    'dropout_rate': 0.2,
    'n_layers': 2,
    'n_latent': 50,
    'dispersion': 'gene-batch'
}
# create the actual model
scvi.model.SCVI.setup_anndata(adata_scvi, batch_key='suspension_type', categorical_covariate_keys=['batch_agg'],
                              layer='raw_counts', continuous_covariate_keys=['total_counts','perc_mito','perc_ribo'])
vae = scvi.model.SCVI(adata_scvi, **params)
# train the model
vae.train(early_stopping=True, accelerator='gpu', max_epochs=200)
adata.obsm['X_scvi'] = vae.get_latent_representation()
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural_endothelial.h5ad')

In [ ]:
# plot the training and validation loss
for loss in ['train_loss_epoch','validation_loss']:
    fig, ax = plt.subplots()
    ax.grid(False)
    ax.plot(vae.history_[loss], label=loss)
    ax.set(xlabel='Epoch', ylabel='Loss')
    ax.legend(frameon=False)

In [ ]:
# compute knn, umap, and leiden
print('computing neighbors...')
sc.pp.neighbors(adata, random_state=0, use_rep='X_scvi')
print('computing umap...')
sc.tl.umap(adata, random_state=0)
print('computing leiden...')
sc.tl.leiden(adata, random_state=0, key_added='leiden_scvi')
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural_endothelial.h5ad')

### remove outliers in vasculature embedding

In [ ]:
# read in the data
adata = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural_endothelial.h5ad')
# identify the outlier population needs to be removed
sc.pl.umap(adata, color=['leiden_scvi'])
# remove the outlier
adata = adata[adata.obs['leiden_scvi'] != '27'].copy()

### compute new clean dimension reduction for vasculature populations

In [ ]:
# find highly variable genes
adata_ = adata[adata.obs['suspension_type'] == 'cell'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_cells = adata_.var.index[adata_.var['highly_variable']]
adata_ = adata[adata.obs['suspension_type'] == 'nuclei'].copy()
sc.pp.highly_variable_genes(adata_)
hvgs_nuclei = adata_.var.index[adata_.var['highly_variable']]
hvgs = hvgs_cells.union(hvgs_nuclei)
adata.var['highly_variable'] = adata.var.index.isin(hvgs)
print(adata.var['highly_variable'].sum())

In [ ]:
# use scVI to compute dimension reduction
import scvi
# derive a batch agg variable to account for donor differences due to issues in large-scale integration without it
adata.obs['batch_agg'] = adata.obs[['study','sequencing_kit']].astype(str).agg(':'.join, axis=1)
# subset for highly variable genes
adata_scvi = adata[:, adata.var['highly_variable']].copy()
del adata_scvi.raw
# setup the parameters
params = {
    'encode_covariates': True,
    'dropout_rate': 0.2,
    'n_layers': 2,
    'n_latent': 50,
    'dispersion': 'gene-batch'
}
# create the actual model
scvi.model.SCVI.setup_anndata(adata_scvi, batch_key='suspension_type', categorical_covariate_keys=['batch_agg'],
                              layer='raw_counts', continuous_covariate_keys=['total_counts','perc_mito','perc_ribo'])
vae = scvi.model.SCVI(adata_scvi, **params)
# train the model
vae.train(early_stopping=True, accelerator='gpu', max_epochs=200)
adata.obsm['X_scvi'] = vae.get_latent_representation()
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural_endothelial.h5ad')

In [ ]:
# plot the training and validation loss
for loss in ['train_loss_epoch','validation_loss']:
    fig, ax = plt.subplots()
    ax.grid(False)
    ax.plot(vae.history_[loss], label=loss)
    ax.set(xlabel='Epoch', ylabel='Loss')
    ax.legend(frameon=False)

In [ ]:
# compute knn, umap, and leiden
print('computing neighbors...')
sc.pp.neighbors(adata, random_state=0, use_rep='X_scvi')
print('computing umap...')
sc.tl.umap(adata, random_state=0)
print('computing leiden...')
sc.tl.leiden(adata, random_state=0, key_added='leiden_scvi')
adata.write(f'{DN}/out_data/h5ads/int.nrm.neural_endothelial.h5ad')

### identify differentially expressed genes between cell types of the whole brain

In [ ]:
# read in the data
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
adata = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural.h5ad')

In [ ]:
# compute unbiased differential genes
mask = adata.var_names.str.startswith('MT-')
mask = mask | adata.var_names.str.startswith('MRPS') | adata.var_names.str.startswith('MRPL')
mask = mask | adata.var_names.str.startswith('RPS') | adata.var_names.str.startswith('RPL')
adata_tmp = adata[:, ~mask].copy()
sc.tl.rank_genes_groups(adata_tmp, method='wilcoxon', use_raw=False, n_genes=adata_tmp.shape[1], groupby='cell_type_int')
sc.pl.rank_genes_groups(adata_tmp, n_genes=25, sharey=False)

In [ ]:
# plot these unbiased differential genes and write them to a table
names = pd.DataFrame(adata_tmp.uns['rank_genes_groups']['names'])
scores = pd.DataFrame(adata_tmp.uns['rank_genes_groups']['scores'])
pvals = pd.DataFrame(adata_tmp.uns['rank_genes_groups']['pvals'])
fdrs = pd.DataFrame(adata_tmp.uns['rank_genes_groups']['pvals_adj'])
# derive the plot first
df_gex = sc.get.obs_df(adata, keys=list(set(names.iloc[:5].values.flatten()))+['cell_type_int'])
df_gex = df_gex.groupby('cell_type_int').mean()
df_gex -= df_gex.min(); df_gex /= df_gex.max()
# look at the distribution
g = sns.clustermap(df_gex, cmap='viridis', method='ward', dendrogram_ratio=(.05, .2), figsize=[16*1.5, 5*1.5],
                   cbar_pos=(0, 1.05, .008, .12), xticklabels=1, yticklabels=1)
g.ax_heatmap.grid(False); g.ax_cbar.grid(False)

### harmonize subpopulations of the neurovasculature subset

In [ ]:
# read in the data
DN = '/lustre/scratch126/cellgen/team205/dc20/proj_GPR85'
adata = sc.read_h5ad(f'{DN}/out_data/h5ads/int.nrm.neural_endothelial.h5ad')
adata.obs['tissue_simple'] = adata.obs['tissue'].str.split(':', expand=True).iloc[:, 0].str.lower()
# integrate the data
adata.obs[['study','cell_type_fine']] = adata.obs[['study','cell_type_fine']].astype(str)

In [ ]:
import cellhint
alignment = cellhint.harmonize(adata, 'study', 'cell_type_fine', use_rep='X_scvi',
                               p_thres=1e-3, random_state=0,
                               minimum_unique_percents=(0.75, 0.80, 0.85, 0.90, 0.95),
                               minimum_divide_percents=(0.010, 0.025, 0.050, 0.075, 0.100),
                               maximum_novel_percent=(0.50))
# color on the cellhint annotation
adata.obs['cellhint_reannotation'] = alignment.reannotation['reannotation']

In [ ]:
# resolve the mapping
mapping = {'NONE = NONE = NONE = cerebral cortex endothelial cell ∈ cerebral cortex endothelial cell = endothelial cell of vascular tree = cerebral cortex endothelial cell ∈ endothelial cell = endothelial cell ∋ endothelial cell of artery = Veinous = NONE':'Capillary',
'NONE = NONE = NONE = cerebral cortex endothelial cell ∈ cerebral cortex endothelial cell = endothelial cell of vascular tree = cerebral cortex endothelial cell ∈ endothelial cell = endothelial cell ∋ capillary endothelial cell = Capillary = Capillary':'Capillary',
'NONE = NONE = NONE = NONE = vascular leptomeningeal cell = pericyte = vascular leptomeningeal cell = pericyte = pericyte = UNRESOLVED = NONE = NONE':'Pericyte',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = vascular associated smooth muscle cell = fibroblast = NONE = P. Fibro = UNRESOLVED':'Fibroblast',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = vascular associated smooth muscle cell = vascular associated smooth muscle cell = SMC = NONE':'VSMC',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = mural cell = Pericyte = UNRESOLVED':'Pericyte',
'fibroblast = endothelial cell = brain vascular cell = blood vessel endothelial cell ∈ cerebral cortex endothelial cell = NONE = NONE ∈ endothelial cell = NONE = NONE = NONE = NONE':'Fibroblast',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Arterial = Artery':'Artery',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = M. Fibro = NONE':'Fibroblast',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = Vein':'Vein',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Large artery':'Large artery',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Angiogenic capillary':'Angiogenic capillary',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Arteriole':'Arteriole',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Large vein':'Large vein',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Venule':'Venule',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = Proliferating cell':'Proliferating cell',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Proliferating Stem-to-EC':'Proliferating Stem-to-EC',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Stem-to-EC':'Stem-to-EC',
'UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = UNRESOLVED = EndoMT':'EndoMT',
'NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = NONE = Proliferating EndoMT':'Proliferating EndoMT'}
adata.obs['cell_type_int_v2'] = adata.obs['cellhint_reannotation'].map(mapping).astype(str)

### annotate larger categories

In [ ]:
# derive the categories
adata.obs['cell_type_int_revised'] = 'Endothelial'
adata.obs.loc[adata.obs['cell_type_int_v2'].isin(['Pericyte','VSMC','Fibroblast']), 'cell_type_int_revised'] = 'Mesenchymal'
adata.obs.loc[adata.obs['cell_type_int_v2'].isin(['EndoMT','Proliferating EndoMT']), 'cell_type_int_revised'] = 'Endo-to-Mesen'
adata.obs.loc[adata.obs['cell_type_int_v2'].isin(['Stem-to-EC','Proliferating Stem-to-EC']), 'cell_type_int_revised'] = 'Stem-to-Endo'

In [ ]:
# plot the colors
adata.uns['cell_type_int_revised_colors'] = ['#8da0cb', '#e78ac3', '#66c2a5','#fc8d62',]
sc.pl.umap(adata, color=['cell_type_int_revised'])

In [ ]:
# ^ renamed with coarse and fine suffixes in the public dataset

### examine the fine-grain annotations and verify cell type relationships via dendrogram

In [ ]:
# function for easier color editing
def edit(a, b):
    cats = adata.obs['cell_type_int_v2'].cat.categories.tolist()
    idx1, idx2 = cats.index(a), cats.index(b)
    col1, col2 = adata.uns['cell_type_int_v2_colors'][idx1], adata.uns['cell_type_int_v2_colors'][idx2]
    adata.uns['cell_type_int_v2_colors'][idx1] = col2
    adata.uns['cell_type_int_v2_colors'][idx2] = col1

In [ ]:
# set the colors appropriately
adata.uns['cell_type_int_v2_colors'] = ['xkcd:bright lilac', '#f7b6d2', '#ff9896', 'xkcd:pale lavender',
                                        '#c49c94', '#c5b0d5', '#e377c2', '#1f77b4', 'tomato', 'tab:brown',
                                        'tab:orange', '#aa40fc', '#ffbb78', 'tab:red', '#aec7e8', '#17becf']
sc.pl.umap(adata, color=['cell_type_int_v2'], use_raw=False)
# plot the dendrogram
sc.tl.dendrogram(adata, use_rep='X_scvi', groupby='cell_type_int_v2')
sc.pl.dendrogram(adata, groupby='cell_type_int_v2')

### examine ACh receptor expression in vasculature subpopulations

In [ ]:
# show dot plot expression
genes = adata.var.index[adata.var.index.str.startswith('CHRM') | adata.var.index.str.startswith('CHRN')].tolist() + ['GPR85']
sc.pl.dotplot(adata, groupby='cell_type_int_v2', var_names=genes, use_raw=False)

### identify genes and pathways correlated with GPR85 expression and PLVAP oppositional programs

In [ ]:
import scipy.stats as ss
from tqdm import tqdm
# look at the correlative profiles
mask = adata.obs['cell_type_int_revised'] == 'Endothelial'
df = pd.DataFrame(adata[mask].X.toarray(), columns=adata.var.index, index=adata.obs.index[mask])
df = df.loc[:, df.nunique(0) > 1]
df_corr = pd.DataFrame(columns=['gene','r','p'])
for x in tqdm(df.columns):
    if x == 'GPR85': continue
    r, p = ss.spearmanr(df['GPR85'], df[x])
    df_corr.loc[df_corr.shape[0]] = x, r, p

In [ ]:
# impute the data
adata_m = adata.copy()
sc.external.pp.magic(adata_m, random_state=0)
# look at the correlative profiles
df_m = sc.get.obs_df(adata_m[mask], keys=adata_m.var.index.tolist())

In [ ]:
# find explanations that are sweet
fig, ax = plt.subplots(); ax.grid(False)
sc.pl.scatter(adata_m[mask], x='GPR85', y='PLVAP', color='cell_type_int_v2', use_raw=False, size=10, ax=ax, show=False)
print(ss.spearmanr(df_m['PLVAP'], df_m['GPR85']))
print(ss.spearmanr(df['PLVAP'], df['GPR85']))

### calculate embedding densities for GPR85 vs. PLVAP scatterplots

In [ ]:
# capillary
all_X = df_m[['GPR85','PLVAP']].T
mask = adata_m.obs['cell_type_int_v2'] == 'Capillary'
input_X = df_m.loc[mask, ['GPR85','PLVAP']].T
z_cap = ss.gaussian_kde(input_X, bw_method=0.5)(all_X)
# angiogenic capillary
mask = adata_m.obs['cell_type_int_v2'] == 'Angiogenic capillary'
input_X = df_m.loc[mask, ['GPR85','PLVAP']].T
z_angcap = ss.gaussian_kde(input_X, bw_method=0.5)(all_X)
# normalize
z_cap -= z_cap.mean(); z_cap /= z_cap.std()
z_angcap -= z_angcap.mean(); z_angcap /= z_angcap.std()

In [ ]:
# find explanations that are sweet
mask = adata.obs['cell_type_int_revised'] == 'Endothelial'
adata_m.obs[['z_cap','z_angcap','z_cap-angcap']] = np.nan
adata_m.obs.loc[mask, 'z_cap'] = z_cap; adata_m.obs.loc[mask, 'z_angcap'] = z_angcap
adata_m.obs.loc[mask, 'z_cap-angcap'] = z_cap - z_angcap
# plot the data
fig, ax = plt.subplots(); ax.grid(False)
cs = z_cap - z_angcap; order = np.argsort(abs(cs))
ax.scatter(df_m['GPR85'][order], df_m['PLVAP'][order], c=cs[order], s=0.5, cmap='RdBu_r', vmin=-2, vmax=2)

### examine correlated gene programs amongst endothelial subpopulations

In [ ]:
# define mask
mask = df_corr['gene'].str.startswith('MT-') | df_corr['gene'].str.startswith('MRPS')
mask = mask | df_corr['gene'].str.startswith('RPS') | df_corr['gene'].str.startswith('MRPL')
mask = mask | df_corr['gene'].str.startswith('RPL')
for x in df_corr[~mask].sort_values('r')['gene'][::-1][:100]:
    print(x)

In [ ]:
# define mask
mask = df_corr['gene'].str.startswith('MT-') | df_corr['gene'].str.startswith('MRPS')
mask = mask | df_corr['gene'].str.startswith('RPS') | df_corr['gene'].str.startswith('MRPL')
mask = mask | df_corr['gene'].str.startswith('RPL')
for x in df_corr[~mask].sort_values('r')['gene'][:100]:
    print(x)

### examine genes correlated with GPR85 expression only considering non-angiogenic capillaries

In [ ]:
import scipy.stats as ss
from tqdm import tqdm
# look at the correlative profiles
mask = adata.obs['cell_type_int_v2'] == 'Capillary'
df = pd.DataFrame(adata[mask].X.toarray(), columns=adata.var.index, index=adata.obs.index[mask])
df = df.loc[:, df.nunique(0) > 1]
df_corr = pd.DataFrame(columns=['gene','r','p'])
for x in tqdm(df.columns):
    if x == 'GPR85': continue
    r, p = ss.spearmanr(df['GPR85'], df[x])
    df_corr.loc[df_corr.shape[0]] = x, r, p

In [ ]:
# define mask
mask = df_corr['gene'].str.startswith('MT-') | df_corr['gene'].str.startswith('MRPS')
mask = mask | df_corr['gene'].str.startswith('RPS') | df_corr['gene'].str.startswith('MRPL')
mask = mask | df_corr['gene'].str.startswith('RPL')
for x in df_corr[~mask].sort_values('r')['gene'][::-1][:100]:
    print(x)

In [ ]:
# define mask
mask = df_corr['gene'].str.startswith('MT-') | df_corr['gene'].str.startswith('MRPS')
mask = mask | df_corr['gene'].str.startswith('RPS') | df_corr['gene'].str.startswith('MRPL')
mask = mask | df_corr['gene'].str.startswith('RPL')
for x in df_corr[~mask].sort_values('r')['gene'][:100]:
    print(x)

### validate the annotations between literature and this study

In [ ]:
# compare the annotations
mask = adata.obs['study'] == 'Wälchli and Ghobrial et al. 2024 (Nature)'
df = pd.concat([adata[mask].obs['cell_type_int_v2'], adata_ref[mask].obs['cell_type_fine']], axis=1).astype(str)
df.columns = ['manual','automatic']
df = df.value_counts().reset_index()
df.columns = ['manual','automatic','counts']
print(df.loc[df['manual'] == df['automatic'], 'counts'].sum() / df['counts'].sum())
# flip the results
df = df.pivot_table(index='manual', columns='automatic', values='counts', aggfunc=np.sum).fillna(0)
df = (df.T / df.sum(1)).T
g = sns.clustermap(df, cmap='Purples', vmin=0, vmax=1, row_cluster=False, col_cluster=False,
                   xticklabels=1, yticklabels=1, figsize=[6, 6], cbar_pos=(.07, .77, .01, .08))
g.ax_heatmap.grid(False)

### compute differentially expressed genes between vasculature subpopulations for additional annotation validation

In [ ]:
# compute unbiased differential genes
mask = adata.var_names.str.startswith('MT-')
mask = mask | adata.var_names.str.startswith('MRPS') | adata.var_names.str.startswith('MRPL')
mask = mask | adata.var_names.str.startswith('RPS') | adata.var_names.str.startswith('RPL')
adata_tmp = adata[:, ~mask].copy()
sc.tl.rank_genes_groups(adata_tmp, method='wilcoxon', use_raw=False, n_genes=adata_tmp.shape[1], groupby='cell_type_int_v2')
sc.pl.rank_genes_groups(adata_tmp, n_genes=25, sharey=False)

In [ ]:
# plot these unbiased differential genes and write them to a table
names = pd.DataFrame(adata_tmp.uns['rank_genes_groups']['names'])
scores = pd.DataFrame(adata_tmp.uns['rank_genes_groups']['scores'])
pvals = pd.DataFrame(adata_tmp.uns['rank_genes_groups']['pvals'])
fdrs = pd.DataFrame(adata_tmp.uns['rank_genes_groups']['pvals_adj'])
# derive the plot first
df_gex = sc.get.obs_df(adata, keys=list(set(names.iloc[:5].values.flatten()))+['cell_type_int_v2'])
df_gex = df_gex.groupby('cell_type_int_v2').mean()
df_gex -= df_gex.min(); df_gex /= df_gex.max()
# look at the distribution
g = sns.clustermap(df_gex, cmap='viridis', method='ward', dendrogram_ratio=(.05, .2), figsize=[16*1.5, 5*1.5],
                   cbar_pos=(0, 1.05, .008, .12), xticklabels=1, yticklabels=1)
g.ax_heatmap.grid(False); g.ax_cbar.grid(False)

### produce individual marker gene bar plots between capillary subpopulations

In [ ]:
df = sc.get.obs_df(adata, keys=['GPR85','cell_type_int_v2'])
df['GPR85'] -= df['GPR85'].mean(); df['GPR85'] /= df['GPR85'].std()
fig, ax = plt.subplots(figsize=[1.5, 4]); ax.grid(False)
sns.barplot(x='cell_type_int_v2', y='GPR85', data=df,
            order=['Capillary','Angiogenic capillary'], color='xkcd:pale lavender',
            edgecolor='xkcd:bright lilac', errcolor='xkcd:bright lilac',
            errwidth=1.5, capsize=0.3, linewidth=1.5)
ax.tick_params(axis='x', labelrotation=90)
ax.plot([-1, 2], [0]*2, color='k')
ax.set_xlim(-1, 2)
ylim = ax.get_ylim()
ax.set_ylim(ylim[0], ylim[1]+(ylim[1]-ylim[0])*0.2)
ss.mannwhitneyu(df['GPR85'][df['cell_type_int_v2'] == 'Capillary'],
                df['GPR85'][df['cell_type_int_v2'] == 'Angiogenic capillary'])

In [ ]:
df = sc.get.obs_df(adata, keys=['PLVAP','cell_type_int_v2'])
df['PLVAP'] -= df['PLVAP'].mean(); df['PLVAP'] /= df['PLVAP'].std()
fig, ax = plt.subplots(figsize=[1.5, 4]); ax.grid(False)
sns.barplot(x='cell_type_int_v2', y='PLVAP', data=df,
            order=['Capillary','Angiogenic capillary'], color='xkcd:pale lavender',
            edgecolor='xkcd:bright lilac', errcolor='xkcd:bright lilac',
            errwidth=1.5, capsize=0.3, linewidth=1.5)
ax.tick_params(axis='x', labelrotation=90)
ax.plot([-1, 2], [0]*2, color='k')
ax.set_xlim(-1, 2)
ylim = ax.get_ylim()
ax.set_ylim(ylim[0], ylim[1]+(ylim[1]-ylim[0])*0.2)
ss.mannwhitneyu(df['PLVAP'][df['cell_type_int_v2'] == 'Capillary'],
                df['PLVAP'][df['cell_type_int_v2'] == 'Angiogenic capillary'])

### logistic regression of cell type and marker gene as controlled for sex and age to associate with cancer

In [ ]:
import statsmodels.api as sm
import scipy.stats as ss
def test_output(output, df):
    gene = df.columns[0]
    # examine comparisons
    mask = df['cat'].isin(['Control (adult)','Control (elder)']) | (df[output] == 1)
    assert (df.loc[df[output] == 1, 'age_in_years'] >= 18).all()
    input_X = df.loc[mask, [gene,'older_than_or_equal_to_60','is_female']]
    output_X = df.loc[mask, output]
    
    # no sex consideration, all cases with cancer are male
    mask_sex = input_X['is_female'] == 0
    model = sm.Logit(output_X.loc[mask_sex], sm.add_constant(input_X.loc[mask_sex, [gene,'older_than_or_equal_to_60']]))
    results = model.fit(); summary = results.summary()

    fig, ax = plt.subplots(figsize=[2, 4]); ax.grid(False)
    sns.boxplot(x=output_X, y=results.fittedvalues, color='xkcd:pale lavender', linecolor='xkcd:bright lilac', linewidth=1.5)
    ax.set_xlim(-1, 2); ylim = ax.get_ylim(); ax.set_ylim(ylim[0], ylim[1]+(ylim[1]-ylim[0])*0.2)
    print('M', ss.mannwhitneyu(results.fittedvalues[output_X == 0], results.fittedvalues[output_X == 1]))

    pvalues_m = results.pvalues; coefs_m = results.params; cis_m = results.conf_int()
    print(summary)
    
    # now plot them together
    fig, ax = plt.subplots(figsize=[4, 4]); ax.grid(False)
    for idx_, idx in enumerate([gene,'older_than_or_equal_to_60','const']):
        c, p, cil, cih = coefs_m[idx], pvalues_m[idx], cis_m.loc[idx, 0], cis_m.loc[idx, 1]
        ax.scatter(c, idx_, color='tab:cyan', edgecolor='dodgerblue', s=100, zorder=2, linewidth=2)
        ax.plot([cil, cih], [idx_]*2, color='dodgerblue', lw=2, zorder=0)
        ax.plot([cil]*2, [idx_-0.1, idx_+0.1], color='dodgerblue', lw=2, zorder=0)
        ax.plot([cih]*2, [idx_-0.1, idx_+0.1], color='dodgerblue', lw=2, zorder=0)
    ax.set_yticks([0, 1, 2]); ax.set_yticklabels([gene,'older_than_or_equal_to_60','const'])
    ax.plot([0]*2, [-1, 3], color='k'); ax.set_ylim(-1, 3); xlim = ax.get_xlim()
    
    return pvalues_m, coefs_m, cis_m

In [ ]:
# reannotate the disease state to be sure of correctness
adata.obs['disease_simple'] = adata.obs['disease_state'].astype(str)
kvs = {'Alzheimer disease':'AD','normal':'Control','acute myocardial infarction':'Control','heart failure':'Control', 'TL':'Control'}
for k, v in kvs.items():
    adata.obs['disease_simple'] = adata.obs['disease_simple'].str.replace(k, v)
adata.obs['disease_simple'] = adata.obs['disease_simple'].str.capitalize()
adata.obs['disease_simple'] = adata.obs['disease_simple'].str.replace('Ad','AD')
adata.obs['disease_simple'] = adata.obs['disease_simple'].astype(str)
kvs = {'Control':'Control (adult)', 'Fetalperiph':'Control (fetal periph.)',
       'Fetalcns':'Control (fetal CNS)','Met':'Cancer (lung metastasis)',
       'Avm':'Atrioventricular malformation','AD':'Alzheimer\'s disease',
       'Gbm':'Cancer (GBM)','Lgg':'Cancer (lower-grade glioma)','Men':'Cancer (meningioma)',}
for k, v in kvs.items():
    adata.obs['disease_simple'] = adata.obs['disease_simple'].str.replace(k, v)

In [ ]:
# show associations with age and disease only considering endothelial cells
adata.obs['donor_study'] = adata.obs[['donor','study']].agg(':'.join, axis=1)
def analyze(column, mask):
    df = sc.get.obs_df(adata[mask], keys=['donor_study','disease_simple','developmental_stage','reported_sex',column], use_raw=False)
    df['developmental_stage'] = df['developmental_stage'].str.split(':', expand=True).iloc[:, 0]
    df['developmental_stage'] = df['developmental_stage'].str.replace('-year-old stage','')
    df['developmental_stage'] = df['developmental_stage'].str.replace(' years','')
    df['developmental_stage'] = df['developmental_stage'].str.replace('80 year-old and over stage','65')
    df['developmental_stage'] = df['developmental_stage'].str.replace('60-79 year-old stage','65')
    df['developmental_stage'] = df['developmental_stage'].str.replace('sixth decade stage','65')
    # change the data in accordance
    pediatrics = ['pediatric stage','infant stage']
    fetals = ['14th week post-fertilization stage','16th week post-fertilization stage','18th week post-fertilization stage',
              '19th week post-fertilization stage','20th week post-fertilization stage','22nd week post-fertilization stage',
              '25th week post-fertilization stage','17th week post-fertilization stage','9th week post-fertilization stage',
              '12.4 gestational weeks','14.4 and 16.4 gestational weeks ','14.4 gestational weeks','15.5 gestational weeks',
              '16.4 gestational weeks','18 gestational weeks','9 gestational weeks','10th week post-fertilization stage',
              'Carnegie stage 20','Carnegie stage 15','12th week post-fertilization stage','15th week post-fertilization stage',
              'Carnegie stage 16']
    df['disease_simple'] = df['disease_simple'].astype(str)
    df['disease_simple'][df['developmental_stage'].isin(pediatrics)&df['disease_simple'].str.startswith('Control')] = 'Control (pediatric)'
    df['disease_simple'][df['developmental_stage'].isin(fetals)&df['disease_simple'].str.startswith('Control')] = 'Control (fetal)'
    # control the controls
    mask_base = df['disease_simple'] == 'Control (adult)'
    pediatric_idxs = df.index[mask_base][df['developmental_stage'][mask_base].astype(float) < 18]
    elder_idxs = df.index[mask_base][df['developmental_stage'][mask_base].astype(float) >= 60]
    df.loc[pediatric_idxs, 'disease_simple'] = 'Control (pediatric)'
    df.loc[elder_idxs, 'disease_simple'] = 'Control (elder)'
    cats = df['disease_simple'].unique()
    for disease in cats:
        df[disease] = (df['disease_simple'] == disease) * 1
    df['is_female'] = 1 * (df['reported_sex'] == 'female')
    df['age_in_years'] = np.nan
    mask_age = df['developmental_stage'].str.isnumeric()
    df.loc[mask_age, 'age_in_years'] = df['developmental_stage'][mask_age].astype(float)
    df['older_than_or_equal_to_60'] = np.nan
    df.loc[mask_age, 'older_than_or_equal_to_60'] = 1 * (df.loc[mask_age, 'age_in_years'] >= 60)
    df = df.groupby('donor_study').mean(numeric_only=True)
    assert df[cats.tolist()].sum(1).max() == 1
    assert df[cats.tolist()].sum(1).min() == 1
    df['cat'] = df[cats.tolist()].idxmax(1)

    # plot
    order = ['Control (fetal)','Control (pediatric)','Control (adult)','Control (elder)']
    fig, ax = plt.subplots(figsize=[2.5, 4]); ax.grid(False)
    sns.barplot(x='cat', y=column, data=df, order=order,
                ci=68, errwidth=1.5, capsize=0.3, color='xkcd:pale lavender',
                edgecolor='xkcd:bright lilac', errcolor='xkcd:bright lilac',
                linewidth=1.5)
    ax.tick_params(axis='x', labelrotation=90)
    ax.set_xlim(-1, 4)
    ylim = ax.get_ylim()
    ax.set_ylim(ylim[0], ylim[1]+(ylim[1]-ylim[0])*0.4)
    ps = []
    for cat in order[1:]:
        p = ss.mannwhitneyu(df[column][df['cat'] == 'Control (fetal)'],
                        df[column][df['cat'] == cat])[1]
        print(cat, p)
        ps.append(p)
    print(max(ps))
    df[column] -= df[column].min(); df[column] /= df[column].max()
    df = df.dropna(); df['has_cancer'] = 1 * (df['cat'].str.startswith('Cancer')); assert df['has_cancer'].sum() > 0
    return test_output('has_cancer', df)

In [ ]:
# create quantitative annotations
adata.obs['is_Angiogenic_Capillary'] = 1 * (adata.obs['cell_type_int_v2'] == 'Angiogenic capillary')
adata.obs['is_nonAngiogenic_Capillary'] = 1 * (adata.obs['cell_type_int_v2'] == 'Capillary')

In [ ]:
import scipy.stats as ss
# endothelial populations
mask = adata.obs['cell_type_int_revised'] == 'Endothelial'
data = analyze('is_nonAngiogenic_Capillary', mask)
data = analyze('is_Angiogenic_Capillary', mask)
data = analyze('GPR85', mask)
data = analyze('PLVAP', mask)
# only considering capillaries that are not angiogenic
mask = adata.obs['cell_type_int_v2'] == 'Capillary'
data = analyze('GPR85', mask)
data = analyze('PLVAP', mask)

### specifically interrogate for AD-related endothelial and BBB-related pathways from MSIGDB

In [ ]:
# compute the signaling pathways
pathway_list = ['score_establish_bbb','score_maintain_bbb',
                'score_AD_endothelial_up']
# GOBP_ESTABLISHMENT_OF_BLOOD_BRAIN_BARRIER	https://www.gsea-msigdb.org/gsea/msigdb/human/geneset/GOBP_ESTABLISHMENT_OF_BLOOD_BRAIN_BARRIER	
genes_ebbb = 'ADGRA2	BPGM	CLDN3	CTNNB1	FOXP3	FZD4	LRP5	LSR	MFSD2A	MXRA8	NDP	RECK	TRPV1	TTPA	VPS4B	WNT7A'.split('\t')
sc.tl.score_genes(adata, gene_list=genes_ebbb, use_raw=False, random_state=0, score_name='score_establish_bbb')
# GOBP_MAINTENANCE_OF_BLOOD_BRAIN_BARRIER	https://www.gsea-msigdb.org/gsea/msigdb/human/geneset/GOBP_MAINTENANCE_OF_BLOOD_BRAIN_BARRIER	ABCC8	ACTB	ACTG1	ANGPT1	CD2AP	CDH5	CLDN1	CLDN12	CLDN3	CLDN5	DMD	ESAM	F11R	GJA1	GJB6	IL6	ITGB1	JAM2	JAM3	LAMA2	LAMC1	LSR	MBP	MFSD2A	NAGLU	OCLN	PECAM1	PTGS2	SH3GL2	SLC12A2	SLC1A1	TJP1	TJP2	TJP3	TSPAN12	VCL	VEGFA	WNK3	ZEB2
genes_mbbb = 'ABCC8	ACTB	ACTG1	ANGPT1	CD2AP	CDH5	CLDN1	CLDN12	CLDN3	CLDN5	DMD	ESAM	F11R	GJA1	GJB6	IL6	ITGB1	JAM2	JAM3	LAMA2	LAMC1	LSR	MBP	MFSD2A	NAGLU	OCLN	PECAM1	PTGS2	SH3GL2	SLC12A2	SLC1A1	TJP1	TJP2	TJP3	TSPAN12	VCL	VEGFA	WNK3	ZEB2'.split('\t')
sc.tl.score_genes(adata, gene_list=genes_mbbb, use_raw=False, random_state=0, score_name='score_maintain_bbb')
# WU_ALZHEIMER_DISEASE_UP	https://www.gsea-msigdb.org/gsea/msigdb/human/geneset/WU_ALZHEIMER_DISEASE_UP	
genes_ad_endo_up = 'ABTB2	ADGRB2	APEX2	CBFA2T3	CYP11B1	ENTPD1	FKBP1A	FOXO4	H2AC7	NRG1	PCDH9	PSPHP1	STEAP2-AS1	TGM2'.split('\t')
sc.tl.score_genes(adata, gene_list=genes_ad_endo_up, use_raw=False, random_state=0, score_name='score_AD_endothelial_up')

# plot the scores on the UMAP, convert into z-scores
adata.obs[pathway_list] -= adata.obs[pathway_list].mean()
adata.obs[pathway_list] /= adata.obs[pathway_list].std()

In [ ]:
import scipy.stats as ss
# perform clustering
def get_r(xs, ys): return ss.pearsonr(xs.astype(float), ys.astype(float))[0]
def get_p(xs, ys): return ss.pearsonr(xs.astype(float), ys.astype(float))[1]
data = sc.get.obs_df(adata[adata.obs['cell_type_int_revised'] == 'Endothelial'],
                     keys=pathway_list+['GPR85','PLVAP']).astype(float)
data_p = data.corr(method=get_p)
for idx in data_p.index: data_p.loc[idx, idx] = 0
g = sns.clustermap(data.corr(method=get_r), vmin=-0.5, vmax=0.5,
                   method='ward', figsize=[6, 6], cmap='RdBu_r',
                   mask=data_p>=0.05, cbar_pos=(0, 1, .01, .1))
g.ax_heatmap.grid(False); g.ax_heatmap.set_facecolor('lightgray')

In [ ]:
import scipy.stats as ss
# perform clustering
def get_r(xs, ys): return ss.pearsonr(xs.astype(float), ys.astype(float))[0]
def get_p(xs, ys): return ss.pearsonr(xs.astype(float), ys.astype(float))[1]
data = sc.get.obs_df(adata[adata.obs['cell_type_int_v2'] == 'Capillary'],
                     keys=pathway_list+['GPR85','PLVAP']).astype(float)
data_p = data.corr(method=get_p)
for idx in data_p.index: data_p.loc[idx, idx] = 0
g = sns.clustermap(data.corr(method=get_r), vmin=-0.5, vmax=0.5,
                   method='ward', figsize=[6, 6], cmap='RdBu_r',
                   mask=data_p>=0.05, cbar_pos=(0, 1, .01, .1))
g.ax_heatmap.grid(False); g.ax_heatmap.set_facecolor('lightgray')

### plot pathways enriched in GPR85 correlated genes in endothelial populations

In [ ]:
from matplotlib.colors import to_hex
from matplotlib.cm import get_cmap

# read in the enrichments (for endothelial)
pathways = np.array('''Vascular Transport (GO:0010232)
Transport Across Blood-Brain Barrier (GO:0150104)
Neuromuscular Junction Development (GO:0007528)
Synapse Organization (GO:0050808)
Adherens Junction (GO:0005912)
Cell-Cell Junction (GO:0005911)
Apical Junction Complex (GO:0043296)
Cadherin Binding (GO:0045296)'''.split('\n'))
ys = np.array([float(x) for x in '''0.025983577
0.025983577
0.032788749
0.036858571
0.010101972
0.023840607
0.026572157
0.016710338'''.split('\n')])

# retrieve the colors
ys = -np.log10(ys)
colors = [y/2 for y in ys]
cmap = get_cmap('Purples')
colors = np.array([to_hex(cmap(c)) for c in colors])
order = np.argsort(ys)[::-1]
# plot the colored bars
fig, ax = plt.subplots(figsize=[3, 2]); ax.grid(False)
ax.bar(pathways[order], ys[order], edgecolor='k', linewidth=1.5, color=colors[order])
ax.tick_params(axis='x', labelrotation=90)

In [ ]:
# single-cell HEK data
a = sc.read_10x_h5('/home/dchen2/LITERATURE/10x_hek_nih/20k_hgmm_3p_HT_nextgem_Chromium_X_filtered_feature_bc_matrix.h5', gex_only=False)
a.var_names_make_unique()
# retrieve the species counts
a.obs['n_counts_human'] = a[:, a.var['genome'] == 'GRCh38'].X.sum(1).A1
a.obs['n_counts_mouse'] = a[:, a.var['genome'] == 'mm10'].X.sum(1).A1
# subset the data for human-like cells
mask = (a.obs['n_counts_human'] > 500) & (a.obs['n_counts_mouse'] < 500)
a = a[mask, a.var['genome'] == 'GRCh38'].copy()
# rename the genes and quantify the number of human genes and percent mitochondrial
a.var_names = a.var_names.str.replace('GRCh38_','')
mt_genes = a.var.index.str.startswith('MT-')
a.obs['perc_mito'] = a[:, mt_genes].X.sum(1).A1 / a.obs['n_counts_human']
a.obs['n_genes_human'] = (a.X > 0).sum(1).A1
# perform intense filtering with human and live cells
mask = (a.obs['n_counts_human'] > 1000) & (a.obs['n_counts_human'] < 100000)
mask = mask & (a.obs['n_genes_human'] > 500) & (a.obs['n_genes_human'] < 10000)
mask = mask & (a.obs['perc_mito'] < 0.20)
a = a[mask].copy()
# normalize the data
a.raw = a; sc.pp.normalize_total(a, target_sum=1e4); sc.pp.log1p(a)
# project the data to exclude any non-HEK cells
sc.pp.highly_variable_genes(a)
sc.tl.pca(a, use_highly_variable=True)
sc.pp.neighbors(a, n_pcs=20)
sc.tl.umap(a)
# exclude the low count clusters
sc.tl.leiden(a)
a = a[~a.obs['leiden'].isin(['10','12'])].copy()
# present the normalized counts
df = sc.get.obs_df(a, keys=['CHRM1','CHRM2','CHRM3','CHRM4','CHRM5','GPR85','VIM']).melt()
fig, ax = plt.subplots(figsize=[3.5, 4]); ax.grid(False)
sns.barplot(x='variable', y='value', data=df, saturation=1, linewidth=1.5,
            edgecolor='k', errwidth=1.5, capsize=0.3, errcolor='k')
ax.set_xlim(-1, 7); ax.tick_params(axis='x', labelrotation=90)
ax.set(xlabel='Gene', ylabel='Gene expression (log-norm)', title='scRNA-seq from 10x')

In [ ]:
# examine the bulkRNA-seq data
# https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE186192
df = pd.read_csv('/home/dchen2/LITERATURE/GSE186192_HEK/GSE186192_HEK_wt_kallisto_RNASeq.csv.gz')
data = df.groupby('gene_id').sum()
data = data.loc[data.index.isin(['GPR85','CHRM1','CHRM2','CHRM3','CHRM4','CHRM5','VIM'])]
data = data.loc[:, data.columns.str.startswith('tpm_replicate')].reset_index().melt(id_vars='gene_id')
# present the normalized counts
fig, ax = plt.subplots(figsize=[3.5, 4]); ax.grid(False)
sns.barplot(x='gene_id', y='value', data=data, saturation=1, linewidth=1.5,
            edgecolor='k', errwidth=1.5, capsize=0.3, errcolor='k')
ax.set_xlim(-1, 7); ax.tick_params(axis='x', labelrotation=90)
ax.set(xlabel='Gene', ylabel='Gene expression (TPM)', title='bulkRNA-seq from GSE186192')

In [ ]:
# read in the bulk data from HPA
df = pd.read_table('/home/dchen2/LITERATURE/HPA/rna_celline.tsv.zip')
# retrieve the data and order
data = df.loc[df['Cell line'] == 'HEK293']
data = data.loc[data['Gene name'].isin(['CHRM1','CHRM2','CHRM3','CHRM4','CHRM5','GPR85','VIM'])]
data = data.set_index('Gene name').loc[['CHRM1','CHRM2','CHRM3','CHRM4','CHRM5','GPR85','VIM']].reset_index()
# present the normalized counts
fig, ax = plt.subplots(figsize=[3.5, 4]); ax.grid(False)
sns.barplot(x='Gene name', y='TPM', data=data, saturation=1, linewidth=1.5,
            edgecolor='k', errwidth=1.5, capsize=0.3, errcolor='k')
ax.set_xlim(-1, 7); ax.tick_params(axis='x', labelrotation=90)
ax.set(xlabel='Gene', ylabel='Gene expression (TPM)', title='bulkRNA-seq from HPA')

In [ ]:
print()